In [ ]:
# default_exp export

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Export module

> Provides functionality to export graph to Pantograph data storage.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted 05_export.ipynb.
Converted deBruijnGraphProcessing.ipynb.
Converted graphTesting.ipynb.
Converted index (2).ipynb.
Converted index.ipynb.


In [ ]:
#exporti

import os
import glob
import json
import time
import itertools
import warnings

from collections.abc import Iterable

import pdb

from copy import deepcopy

import numpy as np

from sortedcontainers import SortedSet
from multiprocessing import Process, Manager, cpu_count

from redis import ResponseError

from pangraph_constructor.utils import pathConvert,NpEncoder,adjustZoomLevels
from pangraph_constructor.utils import iset_add,getDBID,resetDB
from pangraph_constructor.graph import GenomeGraph,initialPathAnalysis,calcNodeLengths

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
#export
componentTemplate = {
    "first_bin": 1,
    "last_bin": 1,
    "firstCol": 1,
    "lastCol": 1,
    "occupants": [],
    "matrix": [],
    "larrivals": [],
    "rarrivals": [],
    "ldepartures": [],
    "rdepartures": [],
    "x": 0
}

chunkTemplate = {
    "json_version":19,
    "first_bin": 1,
    "first_col": 1,
    "last_bin": 1,
    "last_col": 1,
    "includes_connectors": True,
    "components": []
}

rootStructTemplate = {
    "json_version": 19,
    "pangenome_length": 0,#pangenomeLength, #total number of bp in pangenome
    "includes_connectors": True,
    "zoom_levels": {}
}


## New version of exporting to Pantograph

**Notation and terminology**

In documentation, we refer to graph nucleotides, columns and components. Components contain columns and columns contain nucleotides.

In the code variable names and comments use slightly different notation. Columns in documentation are bins in code and comments, whereas graph nucleotides in documentation are called columns in the code and comments. This happened for the legacy reasons, i.e. originally there was no nucleotide numbers (columns) in the visualised graph structure and components were split into bins (literally, equal sized bins). It is not true anymore, but old terminology left here.

Ideally all variable names and comments should be changes in line with documentation notation, but I have no idea when this can happen.

In [ ]:
#export
def outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,reason,debug=False,inversionThreshold=0.5):
    
    if (nodeInversionInPath<=inversionThreshold):
#         if debug:
#             print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due to {reason} to node {nextNode}.')
        rightFarLink = True
    else:
#         if debug:
#             print(f'Node {nodeIdx}: Component will be broken before node {nodeIdx} due to {reason} to node {nextNode}.')
        leftFarLink = True
        
    return leftFarLink,rightFarLink

In [ ]:
#export
def recordLinks(nodeIdx,nextNode,pathID,step,nodeInversionInPath,nonLinearCond,pathNodeArray,fromLinks,toLinks,debug=False,inversionThreshold=0.5):
    leftFarLink = False
    rightFarLink = False
    if nonLinearCond:
        if debug:
            print(f'Non-linear link for path {pathID} from node {nodeIdx} to node {nextNode}')

        fromLinks.setdefault(nodeIdx,{}).setdefault(nextNode,[]).append(pathID)
        toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
        leftFarLink,rightFarLink = outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,'backward link',debug=debug,inversionThreshold=0.5)
            
    else:
        if np.any([node in pathNodeArray[pathID,:] for node in range(nodeIdx+1*step,nextNode,step)]):
            if debug:
                print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

            fromLinks.setdefault(nodeIdx,{}).setdefault(nextNode,[]).append(pathID)
            toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
            
            leftFarLink,rightFarLink = outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,'forward jumping link',debug=debug,inversionThreshold=0.5)
        else:
            startNode = None
            endNode = None

            if step==-1:
                startNode = nodeIdx
                endNode = nextNode
                if debug:
                    print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')

            elif step==1:
                startNode = nodeIdx
                endNode = nextNode
                if debug:
                    print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')
            else:
                if debug:
                    print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

                fromLinks.setdefault(nodeIdx,{}).setdefault(nextNode,[]).append(pathID)
                toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                leftFarLink,rightFarLink = outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,'forward jumping link',debug=debug,inversionThreshold=0.5)

            if startNode is not None and endNode is not None and step is not None:
                for intermediateNodeIdx in range(startNode,endNode,step):
                    if debug:
                        print(f'Adding link from node {intermediateNodeIdx} to node {intermediateNodeIdx+1*step} for path {pathID}')
                    fromLinks.setdefault(intermediateNodeIdx,{}).setdefault(intermediateNodeIdx+1*step,[]).append(pathID)
                    toLinks.setdefault(intermediateNodeIdx+1*step,{}).setdefault(intermediateNodeIdx,[]).append(pathID)
    return leftFarLink,rightFarLink,fromLinks,toLinks

In [ ]:
#export
def checkForBreak(nodeIdx,nodeLen,
                  nodePathsIdx,nodeSeqInPath,
                  uniqueNodePathsIDs,pathNodeCount,
                  pathLengths,
                  pathNodeArray,pathDirArray,
                  occupancy,inversion,
                  fromLinks,toLinks,
                  nBins,maxLengthComponent,
                  inversionThreshold=0.5,
                  debug=False):
    '''
    Function to check whether the component should be broken before (left) and/or after (right) it.
    
    Parameters
    ==========
    
    `nodeIdx`: int. Numerical (1 based) node ID within the graph (consecutive number of the node)
               in the graph order.
    
    #######
    The following 5 parameters are calculated in `nodeStat` function externally and passed here:
    `nodeLen`: int. Length of the node in nucleotides
    `nodePathsIdx`: numpy.array. A 1D numpy.array which contain ids of the paths 
                    for each appearance of the node ID in each path.
    `nodeSeqInPath`: numpy.array. A 1D numpy.array with positions of given node in every path. 
                     The path to which this position is related to can be found from `nodePathsIdx`, 
                     e.g. if `nodeSeqInPath[i]` is `k`, then node with node ID `nodeIdx`
                     appear in path `nodePathsIdx[i]` at position `k`.
    `uniqueNodePathsIDs`: Iterable[int]. Usually a list or 1D numpy.array holding unique IDs of paths 
                          which pass this node at least once.
    `pathNodeCount`: numpy.array.: A 1D array (the same length as `uniqueNodePathsIDs`) which 
                     contains number of times the node appear in each path from `uniqueNodePathsIDs`,
                     e.g. if `pathNodeCount[i]` is `k` then node with ID nodeIdx appears in path
                     `uniqueNodePathsIds[i]` `k` times.
    #######
    
    `occupancy`: dict. A dictionary containing occupancy (number of times given path passed the given 
                 node) of the previous node (in terms of graph order) for each path than passed 
                 through previous node. The dictionary has path IDs as keys.
    `inversion`: dict. A dictionary containing inversion (fraction of copies of the node in the path 
                 that are inverted) of the previous node (in terms of graph order) for each path than 
                 passed through previous node. The dictionary has path IDs as keys.
    `fromLinks`: dict[dict[list]]. Dictionary which holds links from each processed node.
    `toLinks`: dict[dict[list]]. Dictionary which holds links to each processed node.
    `nBins`: int. Number of bins recorded in the current component.
    `maxLengthComponent`: int. Maximum number of bins allowed in a component. Used to break too 
                          long components into smaller more manageable blocks.
    `pathDirArray`: 2D numpy.array. Each row (corresponding to specific path in graph) contains directionality
                    of each node in the path. See `initialPathAnalysis` for details.
    `inversionThreshold`: float (default: 0.5). A threshold after which the node is considered inverted.
    
    Returns
    =======
    
    `leftFarLink`: bool. Shows whether there is a far link on the left that will require component break.
    `rightFarLink`: bool. Shows whether there is a far link on the right that will require component break.
    `leftPathStartEnd`: bool. Shows whether there is a start or end of the path on the left of the node.
    `rightPathStartEnd`: bool. Shows whether there is a start or end of the path on the right of the node.
    `isChangedOccupancy`: bool. Shows whether the occupancy of at least one path significantly changed 
                          from previous node. If previous node was missing, and current node has any 
                          occupancy, it will be False as it does not constitute a component break.
    `isChangeInversion`: bool. Shows whether the inversion of at least one path significantly changed 
                          from previous node.
    `isNotFitIntoComponent`: bool. Shows whether this node won't fit into the currently forming component
                             and it should be broken before the current node.
    `pathStarts`: list. Contains list of path IDs which start from this node.
    `pathEnds`: list. Contains list of path IDs which end on this node.
    `fromLinks`: dict[dict[list]]
    `toLinks`: 
    `occupancy`: dict. A dictionary containing occupancy (number of times given path passed the given 
                 node) of the current node (in terms of graph order) for each path than passed 
                 through previous node. The dictionary has path IDs as keys.
    `inversion`: dict. A dictionary containing inversion (fraction of copies of the node in the path 
                 that are inverted) of the current node (in terms of graph order) for each path than 
                 passed through previous node. The dictionary has path IDs as keys.
    
    '''
    
    # Boolean indicator of far incoming link on the left and right
    leftFarLink = False
    rightFarLink = False

    # Boolean if at least one path starts from this node and start is on the left or right
    leftPathStart = False
    rightPathStart = False
    # Boolean if at least one path ends on this node and end is on the left or right
    leftPathEnd = False
    rightPathEnd = False

    # Indicator whether occupancy changed from one node to the next
    isChangedOccupancy = False
    
    # Indicator whether inversion is changed from one side of
    # `inversionThreshold` to the other.
    isChangeInversion = False
    
    # Indicator whether this node does not fit into current component.
    isNotFitIntoComponent = False

    pathEnds = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if nodeInSeq==pathLengths[pathIdx]-1]))
    pathStarts = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if nodeInSeq==0]))
    
    invertedEnds = set()
    invertedStarts = set()
    
    # Check whether adding this node to the current component will violate the limitation
    # for maximum component length. Is it still needed with proper treatment of partially
    # visible components in the front-end???
    if nBins+nodeLen>maxLengthComponent:
        isNotFitIntoComponent = True
    
    # Check whether component should be broken before current node.
    for j, pathID in enumerate(uniqueNodePathsIDs):
        
        # Check whether occupancy changed from previous node.
        # That will require breaking component before current node.
        if nodeIdx>1:
            if np.abs(pathNodeCount[j]-occupancy.get(pathID,0))>0 and occupancy.get(pathID,0)>0:
                isChangedOccupancy = True
        
        # Calculate occupancy of current node for path
        # It is simply how many times this node is passed by current path.
        occupancy[pathID] = pathNodeCount[j]
        
        # Calculate inversion rate of current node for path
        # It is ratio of how many times the node is passed in reversed direction
        # by the path to overall occupancy of the node by the path.
        nodePositions = np.where(nodePathsIdx==pathID)[0]
        
        nodeInversionInPath = np.sum([int(pathDirArray[pathID,nodeNumInPath]) \
                          for nodeNumInPath in nodeSeqInPath[nodePositions]])\
                          /occupancy[pathID]
        
        # Check whether inversion is switched from one side of `inversionThreshold` to the other side.
        if nodeIdx>1 and (nodeInversionInPath-inversionThreshold)*(inversion.get(pathID, nodeInversionInPath)-inversionThreshold)<0:
            isChangeInversion = True
        # Record current inversion for current pathID for bins generation.
        inversion[pathID] = nodeInversionInPath

        # Get indexes of positions of the node in the path
        
        
        # Check whether the node 
        if nodeInversionInPath<=inversionThreshold:
            # not inverted node
            leftPathStart = leftPathStart or (pathID in pathStarts)
            rightPathEnd = rightPathEnd or (pathID in pathEnds)
        else:
            if debug:
                print(f'Node {nodeIdx}, accession {graph.accessions[pathID]}, inversion {nodeInversionInPath}')
            # inverted node
            rightPathStart = rightPathStart or (pathID in pathStarts)
            
            if pathID in pathStarts:
                invertedStarts.add(pathID)
            
            leftPathEnd = leftPathEnd or (pathID in pathEnds)
            
            if pathID in pathEnds:
                invertedEnds.add(pathID)
        
        for nodePos in nodePositions:
            # Boolean flags to whether check and record incoming and outgoing links
            checkIn = False
            checkOut = False
            # Get the sequencial number of node in the path (for current passing), can be several for duplicated nodes
            nodePositionInPath = nodeSeqInPath[nodePos]
            
            if nodePositionInPath<(pathLengths[pathID]-1):
                # if not last node in the path
                checkOut = True
                nextNode = pathNodeArray[pathID,nodePositionInPath+1]
                nextNodePositions = np.where(pathNodeArray[pathID,:]==nextNode)[0]
                nextNodeInversionInPath = np.sum([int(pathDirArray[pathID,nodeInPathPos]) for nodeInPathPos in nextNodePositions])/len(nextNodePositions)
                if debug:
                    print(f'Node {nodeIdx}, next node {nextNode}, path {pathID}, number in path {nodePositionInPath}, length of path {pathLengths[pathID]}')
            else:
                if debug:
                    print("Last node in path")
            
            if nodePositionInPath>0:
                checkIn = True
                prevNode = pathNodeArray[pathID,nodePositionInPath-1]
                prevNodePositions = np.where(pathNodeArray[pathID,:]==prevNode)[0]
                prevNodeInversionInPath = np.sum([int(pathDirArray[pathID,nodeInPathPos]) for nodeInPathPos in prevNodePositions])/len(prevNodePositions)
                if debug:
                    print(f'Node {nodeIdx}, previous node {nextNode}, path {pathID}, number in path {nodePositionInPath}, length of path {pathLengths[pathID]}')
            else:
                if debug:
                    print("First node in path")
            
            
            if nodeInversionInPath<=inversionThreshold:
            # Not inverted node
            # For the node find the links going to the left of the node
                if checkIn:
                    # node is not the first
                    if debug:
                        print(f"IncomingLinks: {prevNode} -> {nodeIdx}")
                        print(f'Existing known links: {fromLinks.get(prevNode,{})}')

                    if prevNode+1!=nodeIdx:
                        # Link in the path does not go between consecutive nodes (from previous to current)
                        # This means that there is a links coming to the left of the node
                        if (prevNode<nodeIdx and nodeIdx in fromLinks.get(prevNode,{})) or (prevNode>=nodeIdx):
                            # If we are not picking up passing through empty blocks as far link
                            # Tandem duplicates are included as they cause an arrow to the left of the node.
                            leftFarLink = True
                if checkOut:
                    # node is not the last
                    
                    # Find out whether it is a non-linear (e.g. reversed link or link from inversed to non-inversed node).
                    if nextNodeInversionInPath<=inversionThreshold:
                        # The inversion is the same for two consecutive (in path) nodes
                        # Link will be non-linear if previous node is staying further in linearised graph
                        nonLinearCond = (nodeIdx>=nextNode)
                        # Step for checking for jumping over link
                        step = 1
                    else:
                        # The inversion is not the same for two consecutive nodes.
                        nonLinearCond = True
                        # Step is not used and assigned for consistency
                        step =0
                        
                    # Process links from (!) this node.
                    _leftFarLink,_rightFarLink,fromLinks,toLinks = \
                    recordLinks(nodeIdx,nextNode,pathID,
                                step,nodeInversionInPath,
                                nonLinearCond,
                                pathNodeArray,
                                fromLinks,toLinks,
                                debug=debug,
                                inversionThreshold=inversionThreshold)
                    
                    leftFarLink = leftFarLink or _leftFarLink
                    rightFarLink = rightFarLink or _rightFarLink
                            
            else:
                #Inverted node
                # For the node find the links goint from the left of the node
                if checkOut:
                    # node is not the last
                    # Find out whether it is a non-linear (e.g. reversed link or link from inversed to non-inversed node).
                    if nextNodeInversionInPath>inversionThreshold:
                        # The inversion is the same for two consecutive (in path) nodes

                        # Link will be non-linear if previous node is staying further in linearised graph
                        nonLinearCond  = (nodeIdx<=nextNode)
                        # Step for checking for jumping over link
                        step = -1
                    else:
                        # The inversion is not the same for two consecutive nodes.
                        nonLinearCond = True
                        # Step is not used and assigned for consistency
                        step = 0
                        
                    _leftFarLink,_rightFarLink,fromLinks,toLinks = \
                    recordLinks(nodeIdx,nextNode,pathID,
                                step,nodeInversionInPath,
                                nonLinearCond,
                                pathNodeArray,
                                fromLinks,toLinks,
                                debug=debug,
                                inversionThreshold=inversionThreshold)
                    
                    leftFarLink = leftFarLink or _leftFarLink
                    rightFarLink = rightFarLink or _rightFarLink
                
                if checkIn:
                    # The node is not the first
                    if debug:
                        print(f"IncomingLinks: {nodeIdx} -> {nextNode}")
                        print(f'Existing known links: {toLinks.get(nextNode,{})}')

                    if nodeIdx-1!=nextNode:
                        # Link does not go between consecutive blocks (from this to previous one)
                        if (nodeIdx>nextNode and nodeIdx in toLinks.get(nextNode,{})) or (nodeIdx<=nextNode):
                            # If we are not picking up passing through empty blocks as far link
                            # Tandems duplicates are included as they also cause an arrow from the left of the node.
                            rightFarLink = True
        
    return (leftFarLink,rightFarLink,
            leftPathStart or leftPathEnd,
            rightPathStart or rightPathEnd,
            isChangedOccupancy,isChangeInversion,
            isNotFitIntoComponent,
            pathStarts,pathEnds,
            fromLinks,toLinks,
            occupancy,inversion,
            invertedStarts,invertedEnds)

In [ ]:
#export
def createZoomLevelDir(outputPath,outputName,zoomLevel):
    '''
    Creates a directory for zoom level chunks. The function will take care of correct directory level separator.
    
    Parameters
    ==========
    `outputPath`: str. Absolute path to location where the visualisation data should be created
    `outputName`: str. The name of the visualisation case, will be a directory at the path given by `outputPath`
    `zoomLevel`: int or str. zoom level (maximum number of nucleotides in a single column/bin).
                 A directory will be ceated
                 
    Return
    ======
    The function does not return anything, but creates a directory tree `outputPath`, 
    within it creates a directory `outputName` and inside it creates a directory `zoomLevel`
    '''
    
    os.makedirs(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}',exist_ok=True)

In [ ]:
#export
def recordZoomLevelForDebug(zoomNodeToComponent,
                            zoomComponentToNodes,
                            zoomComponentLengths,
                            zoomComponents,
                            zoomCompNucleotides,
                            zoomAccStarts,
                            zoomAccEnds,
                            nodeToComponent,
                            componentToNodes,
                            componentLengths,
                            components,
                            componentNucleotides,
                            accStarts,
                            accEnds,
                            zoomLevel):
    '''
    A function which records result of segmentation to dictionaries, 
    which holds results for all zoom levels.
    It is currently used only for debugging purposes and in normal operation 
    all zoom level dictionaries are not created and used.
    
    Parameters
    ==========
    
    Parameters are self-explanatory. Those starting from `zoom` are dictionary
    holding all zoom levels, whereas the same parameter names without `zoom`
    in the beginning are data for zoom level given in `zoomLevel` parameter.
    
    Returns
    =======
    
    Returns modified dictionaries with `zoom` in the beginning of the names. Theoretically,
    it should work without return at all (through pointers passed to the function), but it
    did not work before and to be on the safe side, this is done through return.
    
    '''
    zoomNodeToComponent[zoomLevel] = nodeToComponent
    zoomComponentToNodes[zoomLevel] = componentToNodes
    zoomComponentLengths[zoomLevel] = componentLengths
    zoomComponents[zoomLevel] = components
    zoomCompNucleotides[zoomLevel] = componentNucleotides
    zoomAccStarts[zoomLevel] = accStarts
    zoomAccEnds[zoomLevel] = accEnds
    
    return  zoomNodeToComponent, \
            zoomComponentToNodes, \
            zoomComponentLengths, \
            zoomComponents, \
            zoomCompNucleotides, \
            zoomAccStarts, \
            zoomAccEnds

In [ ]:
#export
def nodeStat(nodeIdx,pathNodeArray,nodeLengths):
    '''
    Function calculate information about node as part of the overall graph.
    
    Parameters
    ==========
    
    `nodeIdx`: int. Number of the node in the graph (1 based).
    `pathNodeArray`: numpy.array. A 2D array (<number of paths> x <max lengths of path>), 
                     where each line present sequence of node IDs in a particular path.
                     Path ordered as they are present in the graph.
    `nodeLengths`: list or numpy.array. An 1D subscribable structure 
                   (normally, list or 1D numpy.array is expected) where each element i
                   is the length of the node ID (1 based) i+1.
    
    Return
    ======
    
    `nodePathsIdx`: numpy.array. A 1D numpy.array which contain ids of the paths 
                    for each appearance of the node ID in each path.
    `nodeSeqInPath`: numpy.array. A 1D numpy.array with positions of given node in every path. 
                     The path to which this position is related to can be found from `nodePathsIdx`, 
                     e.g. if `nodeSeqInPath[i]` is `k`, then node with node ID `nodeIdx`
                     appear in path `nodePathsIdx[i]` at position `k`.
    `nodeLen`: int. Length of the node (in terms of nucleotides, real or simulated).
    `uniqueNodePathsIDs`: numpy.array. A 1D array containing all path IDs (0 based, non-repeated) 
                          that contains the node.
    `pathNodeCount`: numpy.array.: A 1D array (the same length as `uniqueNodePathsIDs`) which 
                     contains number of times the node appear in each path from `uniqueNodePathsIDs`,
                     e.g. if `pathNodeCount[i]` is `k` then node with ID nodeIdx appears in path
                     `uniqueNodePathsIds[i]` `k` times.
    
    '''
    
    # Find positions of the node in each path
    nodePathsIdx, nodeSeqInPath = np.where(pathNodeArray == nodeIdx)
    # Find node length
    nodeLen = nodeLengths[nodeIdx-1]

    # Find unique path IDs (numerical) and cound of given node in each path.
    uniqueNodePathsIDs,pathNodeCount = np.unique(nodePathsIdx,return_counts=True)
    
    return nodePathsIdx,nodeSeqInPath,nodeLen,uniqueNodePathsIDs,pathNodeCount

In [ ]:
#export
def finaliseComponentBase(component,
                      components,componentNucleotides,
                      matrix,occupants,nBins,componentLengths,nucleotides,zoomLevel,accessions,inversionThreshold=0.5):
    componentLengths.append(nBins)
    component["matrix"].extend([[pathID,
                                 int(matrixPathArray[1][0][1]>inversionThreshold),
                                 matrixPathArray] \
                                for pathID,matrixPathArray in matrix.items()])
    component['binsToCols'] = [1]*nBins
    
    component["occupants"] = sorted(list(occupants))
    component['last_bin'] = component['first_bin'] + nBins - 1
    component['lastCol'] = component['firstCol'] + nBins - 1
    component['binColStarts'] = list(range(component['firstCol'],component['lastCol']+1))
    component['binColEnds'] = list(range(component['firstCol'],component['lastCol']+1))
    componentNucleotides.append(nucleotides)
    
    firstBin = component['last_bin'] + 1
    firstCol = component['lastCol'] + 1
    components.append(component)
    component = deepcopy(componentTemplate)
    component['first_bin'] = firstBin
    component['firstCol'] = firstCol
    return component,components,componentNucleotides,{},set(),0,''

In [ ]:
#export
def processAnnotationInterval(posStart,posEnd,annotation,res):
    for pos in range(posStart,posEnd):
        r = []
        for name,intervals in annotation.items():
            for interval in intervals:
                if pos<=interval[1] and pos>=interval[0]:
                    r.append(name)
        res[pos] = r

In [ ]:
#export
def updateEdges(accEdge,edgeAccessions,compNum):
    '''
    Function fills up either accStarts or accEnds (on which component each accession starts and 
    on which ends). `compNum` is assumed to be 1-based.
    '''
    
    for accID in edgeAccessions:
        accEdge[accID] = compNum
        
    return accEdge

In [ ]:
#export
def baseLayerZoom(graph,
                  outputPath,outputName,
                  pathNodeArray,pathDirArray,
                  pathLengths,nodeLengths,
                  pathNodeLengthsCum,
                  maxLengthComponent,
                  CPUS = cpu_count(),
                  inversionThreshold=0.5,
                  isSeq=True,
                  debug=False,
                  debugTime=False):

    zoomLevel = 1
    
    #Create the directory to hold zoomLevel chunks and fasta files (if available)
#     createZoomLevelDir(outputPath,outputName,zoomLevel)
    

    print('\n===========================')
    print(f'Zoom level {zoomLevel}')
    print('===========================')
    zoom_level_struct = {}
    zoom_level_struct["files"] = []
    
    accStarts = {}
    accEnds = {}
    
    invertedStarts = set()
    invertedEnds = set()
    
    nodeToComponent = []
    componentToNode = []
    componentLengths = []
    components = []
    componentNucleotides = []

    component = deepcopy(componentTemplate)
    
    numNodes = len(graph.nodes)
    numNodesDigits = np.int(np.ceil(np.log10(numNodes)))
    
    fromLinks = {}
    toLinks = {}
    
    fromComponentLinks = {}
    toComponentLinks = {}
    
    occupants = set()
    nucleotides = ''
    matrix = {}

    nodeLinks = []

    nBins = 0
#     previousInv = {}
    binLength = 0
    occupancy = {}
    inversion = {}
    pos = {} #???
    nodesInComp = set()
    annotationNames = {}
#     breakComponentWhenBinEnds = False
#         breakCompBeforeBin = False
#     binOpen = False
#     breakComponent = False
#     forceBreak = False
    
    for nodeIdx in range(1,numNodes+1):
        if debug or debugTime:
            print(f'Processing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}')
        else:
            print(f'\nProcessing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}',end='')
        
        if debugTime:
            startNodeTime = time.time()
        ######################
        # Node preprocessing #
        ######################
        
        # Get some onfo about the node
        nodePathsIdx,nodeSeqInPath,nodeLen,uniqueNodePathsIDs,pathNodeCount = \
        nodeStat(nodeIdx,pathNodeArray,nodeLengths)
        
        
        
        # Check whether the component should be broken before and/or after current node
        (leftFarLink,rightFarLink,
        leftPathStartEnd,
        rightPathStartEnd,
        isChangeOccupancy,isChangeInversion,
        isNotFitIntoComponent,
        pathStarts,pathEnds,
        fromLinks,toLinks,
        occupancy,inversion,
        invertedCompStarts,invertedCompEnds) = checkForBreak(nodeIdx,nodeLen,
                                             nodePathsIdx,nodeSeqInPath,
                                             uniqueNodePathsIDs,pathNodeCount,
                                             pathLengths,
                                             pathNodeArray,pathDirArray,
                                             occupancy,inversion,
                                             fromLinks,toLinks,
                                             nBins,maxLengthComponent,
                                             inversionThreshold=inversionThreshold,
                                             debug=debug)
        
        invertedStarts.update(invertedCompStarts)
        invertedEnds.update(invertedCompEnds)
        
        if debugTime:
            print(f'Checking for breaks node {nodeIdx}. Took {time.time() - startNodeTime}')
        ###################################
        # Breaking component before node. #
        ###################################
        ## Here component should be broken before node (if determined necessary)
        if nodeIdx>1 and \
           (leftPathStartEnd or leftFarLink or isChangeInversion or isChangeOccupancy or isNotFitIntoComponent) and \
           nBins>0:
            # It is not the first component, and there is at least one flag to break before node and 
            # there is something to break (the component was not broken after previous node)
            if debug:
                if leftFarLink:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to left far link.')
                elif isChangeOccupancy or isChangeInversion:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to change in inversion or occupancy in a path.')
                elif isNotFitIntoComponent:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} because it does not fit into current component.')
                else:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to start or end of a path.')

            [component,components,
            componentNucleotides,
            matrix,occupants,
            nBins,
            nucleotides] = finaliseComponentBase(component,components,
                                            componentNucleotides,
                                            matrix,occupants,
                                            nBins,
                                            componentLengths,
                                            nucleotides,
                                            zoomLevel=zoomLevel,
                                            accessions=graph.accessions)
            if len(components) not in nodeToComponent[-1]:
                nodeToComponent[-1].append(len(components))
            componentToNode.append(list(nodesInComp))
            nodesInComp = set()
        
        occupants |= set(uniqueNodePathsIDs)
        nodesInComp.add(nodeIdx)
        nodeToComponent.append([])
        
        
        #######################################################
        # Adding bins and forming components for zoom level 1 #
        #######################################################
        # TODO: This process for each bin should be parallelised
        
        # Building nucleotide (real or dummy) for the component.
        if isSeq:
            nucleotides += graph.nodesData[nodeIdx-1]
        else:
            nucleotides += graph.nodes[nodeIdx-1]
        
        # binLength is removed as this function process only single nucleotide level.
        
        # Preparing annotation for each node and each accession
        curNodeAnnotation = {}
        
        forwardPaths = []
        invertedPaths = []
        
        for j,pathID in enumerate(uniqueNodePathsIDs):
            if debugTime:
                startTime = time.time()
            
            pathName = graph.accessions[pathID]
            
            # get occupancy and inversion for current node in current path.
            occ = occupancy[pathID]
            inv = inversion[pathID]
            
            if inv>inversionThreshold:
                invertedPaths.append(pathID)
            else:
                forwardPaths.append(pathID)
            
            # Creating addition to matrix from this node for this path.
            matrixPath = matrix.get(pathID,[[],[]])
            
            
            # Absolute positinal genomic coordinate for given path for each pass of the node.
            # NodeStarts are 1-based (position of the first node in the path will be 1).
            nodeStarts = np.array([pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1 \
                          for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]])
            
            
            # Generating positional pairs for each nucleotide of the node.
            posPath = [list(zip(nodeStarts+pos,nodeStarts+pos)) for pos in range(nodeLen)]
            
            if debugTime:
                print(f'Preparation for node {nodeIdx} path {pathName} took {time.time() - startTime}')
                annotationTime = time.time()
            
            ###
            # Splitting annotaton into single bins/nucleotides
            ###
            # Do a test for when number of annotation records X number of nucleotides give benefit of parallelising 
            # and do an check. If it is larger, then run in parallel, otherwise, do sequencially.
            if len(graph.nodesAnnotation[nodeIdx-1][pathName])*nodeLengths[nodeIdx-1]>2e6:
                blockSize = int(np.ceil(nodeLen/CPUS))
                bounds = list(range(0,nodeLen + nodeLen%blockSize+1,blockSize))
                intervals = list(zip(bounds[:-1],bounds[1:]))
                intervals[-1] = (intervals[-1][0],nodeLen)
                manager = Manager()
                annotations = manager.list([[]]*nodeLen)
                processList = [Process(target=processAnnotationInterval,
                                       args=(*ints,graph.nodesAnnotation[nodeIdx-1][pathName],
                                             annotations)) for ints in intervals]
                [p.start() for p in processList]
                [p.join() for p in processList]
                annotations = list(annotations)
            else:
                annotations = [[]]*nodeLen
                processAnnotationInterval(0,nodeLen,graph.nodesAnnotation[nodeIdx-1][pathName],annotations)
            
            if debugTime:
                print(f'Annotations for node {nodeIdx} path {pathName} finished in {time.time()-annotationTime}, overall time {time.time()-startTime}')
                matrixTime = time.time()
            
            matrixPath[0].extend(range(nBins,nBins+nodeLen))
            matrixPath[1].extend(zip([occ]*nodeLen,[inv]*nodeLen,posPath,annotations))
#             if inv>inversionThreshold:
#                 matrixPath[1].extend(list(zip([occ]*nodeLen,[inv]*nodeLen,posPath,annotations)))
#             else:
#                 matrixPath[1].extend(zip([occ]*nodeLen,[inv]*nodeLen,posPath,annotations))
            
            matrix[pathID] = matrixPath
            
            if debugTime:
                print(f'MatrixPath for node {nodeIdx} path {pathName} finished in {time.time()-matrixTime}, overall time {time.time()-startTime}')
        
        forwardPaths.extend(set(range(len(graph.accessions))).difference(forwardPaths+invertedPaths))
        
        # After all bins are formed, they should be split among components.
        # If there are more than one component from the node, then the last component 
        # should remain open and the rest of checks should go as normal.
        # This will allow to attach a small fully followed node after 
        # (which should be the single node but due to graph construction error was separated).
        # Otherwise, if only one component is formed, then the normal checks should follow.
        nBinsReduction = 0
        if nodeLen>maxLengthComponent:
            if debugTime:
                postBinTime = time.time()
            for blockStart in range(0,nodeLen+1-maxLengthComponent,maxLengthComponent):
                blockEnd = blockStart + maxLengthComponent
                if len(forwardPaths)>0:
#                     component["rdepartures"].append({
#                         "upstream": component["first_bin"]+maxLengthComponent-1,
#                         "downstream": component["first_bin"]+maxLengthComponent,
#                         "participants": forwardPaths,
#                         'otherSideRight': False
#                     })
                    addLink(len(components)+1,'+',len(components)+2,'+',forwardPaths,fromComponentLinks,toComponentLinks)

                if len(invertedPaths)>0:
#                     component["rarrivals"].append({
#                         "upstream": component["first_bin"]+maxLengthComponent,
#                         "downstream": component["first_bin"]+maxLengthComponent-1,
#                         "participants": invertedPaths,
#                         'otherSideRight': False
#                     })
                    addLink(len(components)+2,'-',len(components)+1,'-',invertedPaths,fromComponentLinks,toComponentLinks)

                if debug:
                    print(f'Node {nodeIdx}: Component broken inside node {nodeIdx} due to max component length.')
                matrixSlice = {}
                for pathID,matrixPath in matrix.items():
                    matrix[pathID][0] = list(range(nodeLen-blockEnd))
                    if inversion[pathID]>inversionThreshold:
                        matrixDataSlice = matrixPath[1][-maxLengthComponent:]
#                         matrix[pathID][0] = [element-maxLengthComponent for element in matrixPath[0][:-maxLengthComponent]]
                        matrix[pathID][1] = matrixPath[1][:-maxLengthComponent]
                    else:
                        matrixDataSlice = matrixPath[1][:maxLengthComponent]
                        #[element-maxLengthComponent for element in matrixPath[0][maxLengthComponent:]]
                        matrix[pathID][1] = matrixPath[1][maxLengthComponent:]
                    matrixSlice[pathID] = [list(range(len(matrixDataSlice))),matrixDataSlice]
                
                if inversion[pathID]>inversionThreshold:
                    nucleotideSlice = nucleotides[-maxLengthComponent:]
                    nucleotides = nucleotides[:-maxLengthComponent]
                else:
                    nucleotideSlice = nucleotides[:maxLengthComponent]
                    nucleotides = nucleotides[maxLengthComponent:]
                
                component,components,componentNucleotides,_,_,_,_ = \
                    finaliseComponentBase(component,components,componentNucleotides,
                                      matrixSlice,occupants,
                                      maxLengthComponent,componentLengths,nucleotideSlice,
                                      zoomLevel=zoomLevel,accessions=graph.accessions)

                nodeToComponent[-1].append(len(components))
                componentToNode.append(list(nodesInComp))
                nodesInComp = set([nodeIdx])
                
                nBinsReduction += maxLengthComponent
            if debugTime:
                print(f'Node splitting for node {nodeIdx} took {time.time()-postBinTime}')
        nBins += nodeLen-nBinsReduction
        
        
        if debugTime:
            postBinTime = time.time()
        #########################################
        # Breaking component after current node #
        #########################################
        # If any path ends on current node, this should be recorded in the component.
        if len(pathEnds)>0:
            if nBins>0:
                component.setdefault("ends",set()).update(pathEnds.tolist())
                accEnds = updateEdges(accEnds,pathEnds,len(components)+1)
            else:
                components[-1].setdefault("ends",set()).update(pathEnds.tolist())
                accEnds = updateEdges(accEnds,pathEnds,len(components))
                
        if len(pathStarts)>0:
            if nBins>0:
                component.setdefault("starts",set()).update(pathStarts.tolist())
                accStarts = updateEdges(accStarts,pathStarts,len(components)+1)
            else:
                components[-1].setdefault("starts",set()).update(pathStarts.tolist())
                accStarts = updateEdges(accStarts,pathStarts,len(components))
        
        if nodeIdx==len(graph.nodes) and nBins>0:
            # Last node in graph.
            if debug:
                print(f'Node {nodeIdx}: Last node in the last component.')
            component,components,componentNucleotides,matrix,occupants,nBins,nucleotides = \
                finaliseComponentBase(component,components,componentNucleotides,matrix,occupants,nBins,componentLengths,nucleotides,
                                  zoomLevel=zoomLevel,accessions=graph.accessions)
            nodeToComponent[-1].append(len(components))
            componentToNode.append(list(nodesInComp))
            nodesInComp = set()

        elif (rightFarLink or rightPathStartEnd) and nBins>0:
            if debug:
                if rightFarLink:
                    print(f'Node {nodeIdx}: Component broken after node {nodeIdx} due to right far link.')
                else:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to start or end of a path.')
            component,components,componentNucleotides,matrix,occupants,nBins,nucleotides = \
                finaliseComponentBase(component,components,componentNucleotides,matrix,occupants,nBins,componentLengths,nucleotides,
                                  zoomLevel=zoomLevel,accessions=graph.accessions)
            nodeToComponent[-1].append(len(components))
            componentToNode.append(list(nodesInComp))
            nodesInComp = set()
        # Not sure why this is needed. It will add com
        elif nBins>0:
            nodeToComponent[-1].append(len(components)+1)
        
        if debugTime:
            print(f'After node {nodeIdx} component breaking took {time.time() - postBinTime}')
            print(f'Processing of node {nodeIdx} took {time.time() - startNodeTime}')
    return (numNodes, # number of nodes
            numNodesDigits, # 
            nodeToComponent,
            componentToNode,
            componentLengths,
            components,
            componentNucleotides,
            fromLinks,toLinks,
            fromComponentLinks,toComponentLinks,
            accStarts,accEnds,
            invertedStarts,invertedEnds)

In [ ]:
#export
def splitforwardInversedNodeComp(pathList,component,isInverse):
    forward = []
    inversed = []
    
    for pathID in pathList:
            try:
                if component["matrix"][component["occupants"].index(pathID)][1]>0:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
            except (IndexError,ValueError):
                # If it is artificial pass link.
                if isInverse:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
                
    return forward,inversed

In [ ]:
#export
def nodeToComponentLinks(components,componentToNode,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,debug=False):
    numComps = len(components)
    numCompsDigits = np.int(np.ceil(np.log10(numComps)))

    for compNum in range(numComps):
        if debug:    
            print(f'Processing component links {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}')
        else:
            print(f'\nProcessing component links {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}',end='')

        component = components[compNum]
        
#         if component['first_bin']==17957:
#             pdb.set_trace()
        
        nodeInComp = componentToNode[compNum]
        nodeInComp.sort()

        if len(nodeInComp)>1:
            fromComponentLinks,toComponentLinks = fillLinksBase(nodeInComp,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,compNum,components)
        elif len(nodeInComp)==1:

            mainNode = nodeInComp[0]
    #             compForNode = nodeToComponent[mainNode-1]
            doLeft = False
            if compNum==0:
                doLeft = True
            elif mainNode not in componentToNode[compNum-1]:
                doLeft = True

            doRight = False # Check this conditions as they may be causing issue!
            if compNum==(len(components)-1):
                doRight = True
            elif mainNode not in componentToNode[compNum+1]:
                doRight = True

            fromComponentLinks,toComponentLinks = fillLinksBase(nodeInComp,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,compNum,components,doLeft,doRight)

        else:
            raise ValueError(f"Component {compNum} does not have any associated nodes!")
    
    print('')
    
    return fromComponentLinks,toComponentLinks

In [ ]:
#export
def fillLinksBase(nodeInComp,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,compNum,components,doLeft=True,doRight=True):
    component = components[compNum]
    for node,isFirstNode in zip([nodeInComp[0],nodeInComp[-1]],[True,False]):
        # Processing all external arrival links
        nodeToLink = toLinks.get(node,{})
        for fromNode in nodeToLink.keys():
            intermediateCondition = (node<fromNode)
            
            la,ra = splitforwardInversedNodeComp(nodeToLink[fromNode],component,intermediateCondition)
            
            fromFirstCompNum = nodeToComponent[fromNode-1][0]
            fromFirstComp = components[fromFirstCompNum-1]
            fromLastCompNum = nodeToComponent[fromNode-1][-1]
            fromLastComp = components[fromLastCompNum-1]
            
            if isFirstNode:
                #left arrivals
                if len(la)>0 and doLeft:
                    frd,fld = splitforwardInversedNodeComp(la,fromFirstComp,intermediateCondition)

                    # from right departure
                    if len(frd)>0 and (fromNode not in nodeInComp) and (fromLastComp['last_bin']+1!=component['first_bin']):
                        addLink(fromLastCompNum,'+',compNum+1,'+',frd,fromComponentLinks,toComponentLinks)
                    #from left departure
                    if len(fld)>0 and (fromNode not in nodeInComp):
                        addLink(fromFirstCompNum,'-',compNum+1,'+',fld,fromComponentLinks,toComponentLinks)
            
            if not isFirstNode:
                #right arrivals
                if len(ra)>0 and doRight:
                    frd,fld = splitforwardInversedNodeComp(ra,fromFirstComp,intermediateCondition)

                    #from right departures
                    if len(frd)>0 and (fromNode not in nodeInComp):
                        addLink(fromLastCompNum,'+',compNum+1,'-',frd,fromComponentLinks,toComponentLinks)
                    #from left departures
                    if len(fld)>0 and (fromNode not in nodeInComp):
                        addLink(fromFirstCompNum,'-',compNum+1,'-',fld,fromComponentLinks,toComponentLinks)
            
        # Processing all external departure links
        nodeFromLink = fromLinks.get(node,{})
        for toNode in nodeFromLink.keys():
            intermediateCondition = (node>toNode)
            
            rd,ld = splitforwardInversedNodeComp(nodeFromLink[toNode],component,intermediateCondition)
            
            toFirstCompNum = nodeToComponent[toNode-1][0]
            toFirstComp = components[toFirstCompNum-1]
            toLastCompNum = nodeToComponent[toNode-1][-1]
            toLastComp = components[toLastCompNum-1]
            
            if not isFirstNode:
                #right departures
                if len(rd)>0 and doRight: # Check if doRight is set incorrectly for our case (121->122 at level 4)
                    tla,tra = splitforwardInversedNodeComp(rd,toFirstComp,intermediateCondition)

                    #to left arrivals
                    if len(tla)>0:
                        addLink(compNum+1,'+',toFirstCompNum,'+',tla,fromComponentLinks,toComponentLinks)

                    #to right arrivals
                    if len(tra)>0: # Most probably the problem is here! Check it!
                        addLink(compNum+1,'+',toLastCompNum,'-',tra,fromComponentLinks,toComponentLinks)
            
            if isFirstNode:
                #left departures
                if len(ld)>0 and doLeft:
                    tla,tra = splitforwardInversedNodeComp(ld,toFirstComp,intermediateCondition)

                    #to left arrivals
                    if len(tla)>0:
                        addLink(compNum+1,'-',toFirstCompNum,'+',tla,fromComponentLinks,toComponentLinks)

                    if len(tra)>0:
                        addLink(compNum+1,'-',toLastCompNum,'-',tra,fromComponentLinks,toComponentLinks)
                        
    return fromComponentLinks,toComponentLinks

In [ ]:
#export
def addLink(fromComp,fromStrand,toComp,toStrand,pathList,fromComponentLinks,toComponentLinks):
    fromComponentLinks.setdefault(fromComp,{}).setdefault(fromStrand,{}).\
                       setdefault(toComp,{}).setdefault(toStrand,set()).\
                       update(pathList)
    toComponentLinks.setdefault(toComp,{}).setdefault(toStrand,{}).\
                     setdefault(fromComp,{}).setdefault(fromStrand,set()).\
                     update(pathList)

In [ ]:
#export
def finaliseChunk(rootStruct,zoomLevel,chunk,nucleotides,nBins,chunkNum,curCompCols,prevTotalCols,outputPath,outputName):
    endChunkBin = chunk['components'][-1]['last_bin']
    endChunkCol = chunk['components'][-1]['lastCol']
    chunk['last_bin'] = endChunkBin
    chunk['last_col'] = endChunkCol
    
    localPath = f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}{os.path.sep}'
    
    fileName = f'chunk{chunkNum}_zoom{zoomLevel}.schematic.json'
    
    with open(f'{localPath}{fileName}','w') as f:
        json.dump(chunk,f,cls=NpEncoder)
    
    rootStruct['zoom_levels'][zoomLevel]['files'].append({
        'file': fileName,
        'first_bin':chunk['first_bin'],
        'first_col':chunk['first_col'],
        'last_bin':chunk['last_bin'],
        'last_col':chunk['last_col'],
#         'x':prevTotalCols
    })
    
    if zoomLevel==1:
        fastaName = f'seq_chunk{chunkNum}_zoom{zoomLevel}.fa'
        rootStruct['zoom_levels'][zoomLevel]['files'][-1]['fasta'] = fastaName
        
        with open(f'{localPath}{fastaName}','w') as f:
            f.write(f'>first_bin:{chunk["first_bin"]} last_bin:{chunk["last_bin"]}\n')
            f.write(nucleotides)
        
    chunk = deepcopy(chunkTemplate)
    chunk['first_bin'] = endChunkBin + 1
    chunk['first_col'] = endChunkCol + 1
    return rootStruct,chunk,0,chunkNum + 1,prevTotalCols+curCompCols,0,'' #rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols

In [ ]:
#export
def addLinksToComp(compNum,components,fromComponentLinks,toComponentLinks):
    component = deepcopy(components[compNum])
    
    toCompDict = toComponentLinks.get(compNum+1,{}) # !!! Check that in fromComponentLinks component numbering is 1-based
    if '+' in toCompDict:
        for fromCompNum,fromCompDict in toCompDict['+'].items():
            fromComp = components[fromCompNum-1]
            for fromStrand,participants in fromCompDict.items():
                doAddArrival = True
                if fromStrand=='+':
                    upstreamBin = fromComp['last_bin']
                    fromRight = True
                    if component['first_bin']-upstreamBin==1:
                        doAddArrival = False
                elif fromStrand=='-':
                    upstreamBin = fromComp['first_bin']
                    fromRight = False
                else:
                    raise ValueError(f'Unrecognised direction of node link {fromStrand}')
                if doAddArrival:
                    component['larrivals'].append({
                            'upstream': upstreamBin,
                            'downstream': component['first_bin'],
                            'otherSideRight': fromRight,
                            'participants': participants
                        })
    
    if '-' in toCompDict:
        for fromCompNum,fromCompDict in toCompDict['-'].items():
            fromComp = components[fromCompNum-1]
            for fromStrand,participants in fromCompDict.items():
                if fromStrand=='+':
                    upstreamBin = fromComp['last_bin']
                    fromRight = True
                elif fromStrand=='-':
                    upstreamBin = fromComp['first_bin']
                    fromRight = False
                else:
                    raise ValueError(f'Unrecognised direction of node link {fromStrand}')
                component['rarrivals'].append({
                        'upstream': upstreamBin,
                        'downstream': component['last_bin'],
                        'otherSideRight': fromRight,
                        'participants': participants
                    })
    
    fromCompDict = fromComponentLinks.get(compNum+1,{}) # !!! Check that in fromComponentLinks component numbering is 1-based
    if '+' in fromCompDict:
        for toCompNum,toCompDict in fromCompDict['+'].items():
            toComp = components[toCompNum-1]
            for toStrand,participants in toCompDict.items():
                if toStrand=='+':
                    downstreamBin = toComp['first_bin']
                    toRight = False
                elif toStrand=='-':
                    downstreamBin = toComp['last_bin']
                    toRight = True
                else:
                    raise ValueError(f'Unrecognised direction of node link {toStrand}')
                component['rdepartures'].append({
                        'upstream': component['last_bin'],
                        'downstream': downstreamBin,
                        'otherSideRight': toRight,
                        'participants': participants
                    })
    if '-' in fromCompDict:
        for toCompNum,toCompDict in fromCompDict['-'].items():
            toComp = components[toCompNum-1]
            for toStrand,participants in toCompDict.items():
                doAddDeparture = True
                if toStrand=='+':
                    downstreamBin = toComp['first_bin']
                    toRight = False
                elif toStrand=='-':
                    downstreamBin = toComp['last_bin']
                    toRight = True
                    if component['first_bin']-downstreamBin==1:
                        doAddDeparture = False
                else:
                    raise ValueError(f'Unrecognised direction of node link {toStrand}')
                if doAddDeparture:
                    component['ldepartures'].append({
                            'upstream': component['first_bin'],
                            'downstream': downstreamBin,
                            'otherSideRight': toRight,
                            'participants': participants
                        })
        
    return component

In [ ]:
#export
def checkLinks(leftComp,rightComp):
    leftRdepCond = np.all([link['upstream']+1==link['downstream'] for link in leftComp['rdepartures']])
    leftRarrCond = np.all([link['upstream']-1==link['downstream'] for link in leftComp['rarrivals']])

    rightRdepCond = np.all([link['upstream']-1==link['downstream'] for link in rightComp['ldepartures']])
    rightRarrCond = np.all([link['upstream']+1==link['downstream'] for link in rightComp['larrivals']])
    
    return leftRdepCond and leftRarrCond and rightRarrCond and rightRdepCond

In [ ]:
#export
def getMatrixPathElement(matrix,pathID):
    res = [el for el in matrix if el[0]==pathID]
    if len(res)==1:
        return res[0]
    elif len(res)>0:
        warnings.warn(f"More than one element for path {pathID} is found!")
    
    return None

In [ ]:
#export
def joinComponents(leftComp,rightComp, maxLengthComponent, inversionThreshold=0.5):
    '''
    If the joining was successful, the function will return a joined component.
    
    If the joining was not successful and was aborted for one of the following reasons, it will return a list of original components. 
    The reasons for aborting the joining can be the following:
    - In one of the paths the invertion is lower than threshold in one component and higher in the other.
    - Left component contains at least one end
    - Right component contains at least one start
    
    The function will not check links for coming or going on the right of the left component and left of the right component. 
    It will just get left links from left component and right links from right component and assign them to the new component.
    '''
    
    if leftComp['last_bin']-leftComp['first_bin']+1 + rightComp['last_bin']-rightComp['first_bin']+1 > maxLengthComponent:
        return [leftComp,rightComp]
    
    if leftComp.get('ends',False):
        # End of a path
        return [leftComp,rightComp]
    
    newComp = {}
    newComp['first_bin'] = min(leftComp['first_bin'],rightComp['first_bin'])
    newComp['last_bin'] = max(leftComp['last_bin'],rightComp['last_bin'])
    newComp['firstCol'] = min(leftComp['firstCol'],rightComp['firstCol'])
    newComp['lastCol'] = max(leftComp['lastCol'],rightComp['lastCol'])
    
    leftCompNumBins = leftComp['last_bin']-leftComp['first_bin']+1
    
    newComp['occupants'] = list(set(leftComp['occupants']).union(rightComp['occupants']))
    
    for pathID in newComp['occupants']:
        leftPathElement = getMatrixPathElement(leftComp['matrix'],pathID)
        rightPathElement = getMatrixPathElement(rightComp['matrix'],pathID)
        if leftPathElement is None and rightPathElement is None:
            continue
        
        if leftPathElement is None:
            if len([el for el in rightPathElement[2][1] if el[2][0][0]==1 or el[2][-1][0]==1])>0:
                # Start of a path
                return [leftComp,rightComp]
            rightPathElement[2][0] = [el+leftCompNumBins for el in rightPathElement[2][0]]
            newComp.setdefault("matrix",[]).append(rightPathElement)
            continue
        
        if rightPathElement is None:
            newComp.setdefault("matrix",[]).append(leftPathElement)
            continue
        
        if (leftPathElement[1]>inversionThreshold and rightPathElement[1]<=inversionThreshold) or \
           (leftPathElement[1]<=inversionThreshold and rightPathElement[1]>inversionThreshold):
            return [leftComp,rightComp]
        
        newPathElement = []
        newPathElement.append(pathID)
        newPathElement.append(leftPathElement[1])
        pathMatrix = []
        pathMatrix.append(leftPathElement[2][0] + [el+leftCompNumBins for el in rightPathElement[2][0]])
        pathMatrix.append(leftPathElement[2][1] + rightPathElement[2][1])
        newPathElement.append(pathMatrix)
        newComp.setdefault("matrix",[]).append(newPathElement)
        
    newComp['larrivals'] = leftComp['larrivals']
    newComp['ldepartures'] = leftComp['ldepartures']
    newComp['rarrivals'] = rightComp['rarrivals']
    newComp['rdepartures'] = rightComp['rdepartures']
    ends = list(set(leftComp.get('ends',[])).union(rightComp.get('ends',[])))
    if len(ends)>0:
        newComp['ends'] = ends
    
    return newComp

In [ ]:
#export
def searchIndicesPosRecord(redisConn,zoomLevel,accessions,posMapping):
    for pathID,posMappingPath in posMapping.items():
        iset_add(redisConn, f'{zoomLevel}.{accessions[pathID]}.Pos',posMappingPath)

def searchIndicesGeneRecord(redisConn,zoomLevel,accessions,geneMapping):
    for pathID,geneMappingPath in geneMapping.items():
        redisConn.hmset( f'{zoomLevel}.{accessions[pathID]}.Gene',geneMappingPath)

In [ ]:
#export
def exportLayer(zoomLevel,components,componentNucleotides,
                fromComponentLinks,toComponentLinks,
                rootStruct,
                outputPath,outputName,
                maxLengthComponent,maxLengthChunk,
                inversionThreshold=0.5,
                redisConn=None,
                accessions=None,
                debug=False):
    
    #Create the directory to hold zoomLevel chunks and fasta files (if available)
    createZoomLevelDir(outputPath,outputName,zoomLevel)

    chunkList = rootStruct["zoom_levels"].setdefault(zoomLevel,{
        "last_bin": components[-1]["last_bin"],
        "last_col": components[-1]["lastCol"],
        "files":[]
    })
    # Recording data to files for zoom level

    #     chunkTemplate = {
    #     "json_version":17,
    #     "bin_width":1,
    #     "first_bin":1,
    #     "includes_connectors": True,
    #     "components": []
    # }    
    chunk = deepcopy(chunkTemplate)
    chunkNum = 0
    prevTotalCols = 0
    curCompCols = 0
    nucleotides = ''
    nBins = 0
    numComps = len(components)
    numCompsDigits = np.int(np.ceil(np.log10(numComps)))
    
    if redisConn:
        accessions = rootStruct["pathNames"]
    posMapping = {}
    geneMapping = {}
    for compNum in range(numComps):
        if debug:    
            print(f'Recording component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}')
        else:
            print(f'\nRecording component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}',end='')
        component = addLinksToComp(compNum,components,fromComponentLinks,toComponentLinks)
#         print(f'Component Length is {component["last_bin"]-component["first_bin"]+1}')
#         redisStartTime = time.time()
        if redisConn is not None:
            
            for pathID,inv,matrixPathArray in component['matrix']:
#                 print(f'Recording search indices for path {accessions[pathID]}')
                pathRedisStartTime = time.time()
                for binNum,binMatrix in zip(*matrixPathArray):
                    if binMatrix[1]>inversionThreshold:
                        overallBin = component['last_bin']-binNum
                    else:
                        overallBin = component['first_bin']+binNum
                    
#                     if pathID==5 and overallBin==20350:
#                         pdb.set_trace()
                    
                    posMapping.setdefault(pathID,{}).update({int(overallBin):[(int(posStart),int(posEnd)) for posStart,posEnd in binMatrix[2]]})
                    
                    

                    geneList = binMatrix[3]
                    for gene in geneList:
                        if gene not in geneMapping.get(pathID,{}):#not redisConn.hexists(f'{zoomLevel}.{accessions[pathID]}.Gene',gene):
                            geneMapping.setdefault(pathID,{}).update({gene:overallBin})
                            
#                 print(f'Recording search indices for path {accessions[pathID]} took {time.time()-pathRedisStartTime}')
#         print(f'Overall recording of search indices for component {compNum+1} is {time.time()-redisStartTime}')
        
#         print(f'Joining component {compNum}')
#         joinStartTime = time.time()
        nBins += component["last_bin"]-component["first_bin"]+1
#         if len(chunk['components'])>0:
#             if checkLinks(chunk["components"][-1],component):
#                 newComp = joinComponents(chunk["components"].pop(),component,maxLengthComponent,inversionThreshold)
#                 if isinstance(newComp,list):
#                     chunk["components"].append(newComp[0])
#                     component = newComp[1]
#                 else:
#                     component = newComp
        if zoomLevel==1:
            nucleotides += componentNucleotides[compNum]
        chunk["components"].append(component)
        
#         print(f'Joining took {time.time() - joinStartTime}')
        
        # End of chunk
        if compNum<len(components)-1:
            if len(chunk['components'])>=maxLengthChunk:
            #nBins+components[compNum+1]["last_bin"]-components[compNum+1]["first_bin"]+1>maxLengthChunk:
#                 print(f'Recording chunk')
#                 chunkStartTime = time.time()
                rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                          rootStruct,
                                                                                          zoomLevel,
                                                                                          chunk,
                                                                                          nucleotides,
                                                                                          nBins,
                                                                                          chunkNum,
                                                                                          curCompCols,
                                                                                          prevTotalCols,
                                                                                          outputPath,
                                                                                          outputName)
#                 print(f'Chunk recorded in {time.time() - chunkStartTime}')
        else:
#             print(f'Recording chunk')
#             chunkStartTime = time.time()
            rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                      rootStruct,
                                                                                      zoomLevel,
                                                                                      chunk,
                                                                                      nucleotides,
                                                                                      nBins,
                                                                                      chunkNum,
                                                                                      curCompCols,
                                                                                      prevTotalCols,
                                                                                      outputPath,
                                                                                      outputName)
#             print(f'Chunk recorded in {time.time() - chunkStartTime}')
    if redisConn is not None:
        searchIndicesPosRecord(redisConn,zoomLevel,accessions,posMapping)
        searchIndicesGeneRecord(redisConn,zoomLevel,accessions,geneMapping)
        try:
            redisConn.bgsave()
        except ResponseError:
            pass
    if not debug:
        print()
    
    print(f'Recording zoom level {zoomLevel} finished.') # Zoom level time is {time.time() - zoomTime}. Time elapsed {time.time() - startTime}')

    return rootStruct

In [ ]:
#export
def combineIntervals(posPath):
    # posPath = pos.get(pathID)
    posArray = np.array(posPath)
    posArray = posArray[np.argsort(posArray[:,0]),:]
    posIntersect = (posArray[1:,1]-(posArray[:-1,0]-1))*\
                    (posArray[:-1,1]-(posArray[1:,0]-1))
    newPos = [[posArray[0,0]]]
    candidates = [posArray[0,1]]
    for jointNum in range(len(posIntersect)):
        if posIntersect[jointNum]>=0:
            candidates.extend(posArray[jointNum+1,:].tolist())
        else:
            newPos[-1].append(np.max(candidates))
            newPos.append([posArray[jointNum+1,0]])
            candidates = [posArray[jointNum+1,1]]

    newPos[-1].append(np.max(candidates))# !!!!    
    return newPos

In [ ]:
#export
def checkLinksZoom(compNum,fromComponentLinks,toComponentLinks):
    # Check only outgoing from the right and incoming to the right
    # and check outgoing from the left on and incoming to the left on the next node.
    if any(['-' in toNodeDicts for toNodeDicts in fromComponentLinks.get(compNum+1,{}).get('+',{}).values()]):
        # If there is an outgoing link from positive block (right) to negative block (anywhere)
        return True

    if any(['+' in fromNodeDicts for fromNodeDicts in toComponentLinks.get(compNum+1,{}).get('-',{}).values()]):
        # If there is an incoming link to the negative block (right) from positive block (anywhere)
        return True
    
    if any([toNodes!=compNum+2 for toNodes in fromComponentLinks.get(compNum+1,{}).get('+',{}).keys()]):
        # If there is an outgoing link from positive block (right) to anywhere 
        # (positive block because negative downstream was excluded above) except the next component
        return True
    
    if any([fromNodes!=compNum+2 for fromNodes in toComponentLinks.get(compNum+1,{}).get('-',{}).keys()]):
        # If there is an incoming link to negative block (right) from anywhere
        # (negavive block because positive upstream was excluded above) except the next component
        return True
    
    if any([fromNodes!=compNum+1 or '-' in fromNodesDict for fromNodes,fromNodesDict in toComponentLinks.get(compNum+2,{}).get('+',{}).items()]):
        # If there is an incoming link to positive block of next component (left) from anywhere except current component
        return True
    
    if any([toNodes!=compNum+1 or '+' in toNodesDict for toNodes,toNodesDict in fromComponentLinks.get(compNum+2,{}).get('-',{}).items()]):
        # If there is an outgoing link from negative block (left) to anywhere except current component
        return True
    
    if not (('+' in fromComponentLinks.get(compNum+1,{}).get('+',{}).get(compNum+2,{})) or \
            ('-' in toComponentLinks.get(compNum+1,{}).get('-',{}).get(compNum+2,{}))):
        return True
    
    return False

In [ ]:
#export
def finaliseComponentZoom(component,components,componentLengths,#componentNucleotides,
                          nBins,nCols,occupants,
                          binBlockLengths,binColStarts,binColEnds,
                          matrix,starts,ends,
                          forwardPaths,invertedPaths,inversionThreshold=0.5):
    
    componentLengths.append(nBins)
    component["matrix"].extend(sorted([[pathID,
                                 int(matrixPathArray[1][0][1]>inversionThreshold),
                                 matrixPathArray] \
                                for pathID,matrixPathArray in matrix.items()],key=lambda el: el[0]))
    
    component['binsToCols'] = binBlockLengths
    component["occupants"] = sorted(list(matrix.keys()))
    
    component['last_bin'] = component['first_bin'] + nBins - 1
    
    component['firstCol'] = min(binColStarts)
    component['lastCol'] = max(binColEnds)
    
    # Is it needed?
    component['binColStarts'] = binColStarts
    component['binColEnds'] = binColEnds
    
    curStarts = starts.intersection(forwardPaths)
    if len(curStarts)>0:
        component['starts'] = list(curStarts)
    starts -= set(forwardPaths)
        
    curEnds = ends.intersection(invertedPaths)
    if len(curEnds)>0:
        component['ends'] = list(curEnds)
    ends -= set(invertedPaths)    
        
#     componentNucleotides.append(nucleotides)
    
    firstBin = component['last_bin'] + 1
#     firstCol = component['lastCol'] + 1
    components.append(component)
    component = deepcopy(componentTemplate)
    component['first_bin'] = firstBin
#     component['firstCol'] = firstCol
    return component,components,componentLengths,0,0,set(),[],[],[],{},starts,ends
    # component,components,componentLengths,(componentNucleotides),nBins,nCols,occupants,
    # binBlockLengths,binColStarts,binColEnds,matrix,starts,ends

Changed how inversion was calculated (below)

In [ ]:
#export
def getOccInvChange(binColLengths,binBlockLength,binOcc,binInv,prevOcc,prevInv,inversionThreshold=0.5):
    occChanged = False
    invChanged = False
    occ = {}
    inv = {}
    for pathID in binOcc:
        # Averaging occupancy
        occ[pathID] = sum([bl*bo for bl,bo in zip(binColLengths,binOcc[pathID])])/binBlockLength
        # Do comparison through floor and then abs difference > 0
        if np.abs(np.floor(occ[pathID]+0.5)-np.floor(prevOcc.get(pathID,occ[pathID])+0.5))>0:
            occChanged = True
        prevOcc[pathID] = occ[pathID]
        
        # Averaging invertion
        inv[pathID] = sum([bl*bo*bi for bl,bo,bi in zip(binColLengths,binOcc[pathID],binInv[pathID])])/(binBlockLength*occ[pathID])
        if (inv[pathID]-inversionThreshold)*(prevInv.get(pathID,inv[pathID])-inversionThreshold)<0 or \
        (inv[pathID]-inversionThreshold)*(prevInv.get(pathID,inv[pathID])-inversionThreshold)==0 and \ 
        inv[pathID]*prevInv.get(pathID,inv[pathID])>inversionThreshold*inversionThreshold:
            # The second comdition after or is taking the case where one is equal to inversionThreshold
            # and another is more than inversionThreshold.
            invChanged = True
        prevInv[pathID] = inv[pathID]
        
    return occChanged,invChanged,occ,inv,prevOcc,prevInv

In [ ]:
#export
def recordBinZoom(occ,inv,binPosArray,binAnn,nBins,nCols,
                  binBlockLength,binBlockLengths,
                  binColLengths,
                  binColStart,binColStarts,binColEnd,binColEnds,
                  matrix,inversionThreshold=0.5):
    # need to check for occupancy and inversion change in comparison with previous bin
    # and if that happens, break the component before this bin (should be special boolean returned.
    
    for pathID in binPosArray:
        pathMatrix = matrix.setdefault(pathID,[[],[]])
        pathMatrix[0].append(nBins)
        
        # Adding everything to matrix element with Combined positions and Annotations (already combined through set)
        if inv[pathID]>inversionThreshold:
            pathMatrix[1].insert(0,[occ[pathID],inv[pathID],combineIntervals(binPosArray[pathID]),list(binAnn[pathID])])
        else:
            pathMatrix[1].append([occ[pathID],inv[pathID],combineIntervals(binPosArray[pathID]),list(binAnn[pathID])])
    
    binBlockLengths.append(binBlockLength)
    
    binColStarts.append(binColStart)
    binColEnds.append(binColEnd)
    
    return 0,[],binColStarts,binColEnds,{},{},{},{},nBins+1,nCols+binBlockLength,binBlockLengths,matrix
    # binBlockLength,binColLengths,binColStarts,binColEnds,
    # binOcc,binInv,binPosArray,binAnn,nBins,nCols,
    # binBlockLengths,matrix

In [ ]:
#export
def finaliseBinZoom(compNum,
                    binOcc,binInv,binPosArray,binAnn,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    prevOcc,prevInv,
                    newComponent,newComponents,newComponentLengths,#compAccDir,#newComponentNucleotides,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,collapsibleBlocks,#nucleotides,
                    starts,ends,
                    forwardPaths,invertedPaths,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=0.5):
    
    preservedPrevInv = deepcopy(prevInv)
    
    occChanged,invChanged,occ,inv,prevOcc,prevInv = \
    getOccInvChange(binColLengths,binBlockLength,binOcc,binInv,prevOcc,prevInv,inversionThreshold=inversionThreshold)
    
    if (occChanged or invChanged) and nBins>0:
        
        # cut the matrix and remove the last added bin for the latest component and 
        # check that there is something to finalise (that it was not the only bin)

        # Add direct link
        if invChanged:
            for pathID in forwardPaths|invertedPaths:
                if preservedPrevInv.get(pathID,0)>inversionThreshold:
                    fromStrand = '-'
                else:
                    fromStrand = '+'
                    
                if inv.get(pathID,0)>inversionThreshold:
                    toStrand = '-'
                else:
                    toStrand = '+'
                
                if fromStrand=='-' and toStrand=='-':
                    addLink(len(newComponents)+2,toStrand,len(newComponents)+1,fromStrand,[pathID],newFromComponentLinks,newToComponentLinks)
                elif fromStrand=='+' and toStrand=='+':
                    addLink(len(newComponents)+1,fromStrand,len(newComponents)+2,toStrand,[pathID],newFromComponentLinks,newToComponentLinks)
                
                if fromStrand!=toStrand:
                    collapsibleBlocks.append((pathID,nCols,[(len(newComponents)+1,len(newComponents)+2)],[len(newComponents)+1]))
                # These links should go into collapsible blocks associated with the new component column length
                # More probably these links will be removed on the next step. But this is happening on the path per path basis
                # and lengths should be calculated on the same path to path basis as well!!! That can be taken from 'binColLengths' without
                # the last element!!!
            
        else:
            if len(forwardPaths)>0:
                addLink(len(newComponents)+1,'+',len(newComponents)+2,'+',forwardPaths,newFromComponentLinks,newToComponentLinks)
            if len(invertedPaths)>0:
                addLink(len(newComponents)+2,'-',len(newComponents)+1,'-',invertedPaths,newFromComponentLinks,newToComponentLinks)
        
        # Add new element to newToOldInd
        newToOldInd.append([compNum])
        # Add number of next element to oldToNewInd
        oldToNewInd[-1].append(len(newComponents))
            
        newComponent,newComponents,newComponentLengths,\
        nBins,nCols,_,binBlockLengths,binColStarts,binColEnds,\
        matrix,starts,ends = \
            finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                  nBins,nCols,occupants,binBlockLengths,
                                  binColStarts,binColEnds,
                                  matrix,starts,ends,
                                  forwardPaths,invertedPaths,inversionThreshold=inversionThreshold)

        

    binBlockLength,binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,binAnn,nBins,nCols,binBlockLengths,matrix = \
    recordBinZoom(occ,inv,binPosArray,binAnn,
                    nBins,nCols,binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    inversionThreshold=inversionThreshold)
    
    return binColLengths,binColStarts,binColEnds,\
            binOcc,binInv,binPosArray,binAnn,nBins,nCols,\
            binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,\
            newComponent,newComponents,newComponentLengths,\
            newFromComponentLinks,newToComponentLinks,\
            occupants,collapsibleBlocks,\
            starts,ends,newToOldInd,oldToNewInd

In [ ]:
#export
def checkChange(compNum,components,zoomLevel):
    doBreak = False
    curComp = components[compNum]
    nextComp = components[compNum+1]
    
    if sum(curComp['binsToCols'])<zoomLevel or sum(nextComp['binsToCols'])<zoomLevel:
        return False
    
    paths = list(set(curComp['occupants'])|set(nextComp['occupants']))
    for pathID in paths:
        curPathMatrix = getMatrixPathElement(curComp['matrix'],pathID)
        nextPathMatrix = getMatrixPathElement(nextComp['matrix'],pathID)

        
        if curPathMatrix is not None and nextPathMatrix is not None:
            curEdgeBinOcc = curPathMatrix[2][1][-1 if curPathMatrix[1]==0 else 0][0] # the last index==1 gives inversion
            nextEdgeBinOcc = nextPathMatrix[2][1][0 if nextPathMatrix[1]==0 else -1][0]
            # Checking for breaking in occupancy
            if np.abs(np.floor(nextEdgeBinOcc+0.5)-np.floor(curEdgeBinOcc+0.5))>0:
                doBreak = True
            
            # Checking for breaking in inversion (using recorded binary inversion)
            if curPathMatrix[1]!=nextPathMatrix[1]:
                doBreak = True
        
        else:
            # If one of the component in the path has empty block.
            # At the moment nothing is happening here, but potentially we can enforce break if that is the case.
            pass
    return doBreak

In [ ]:
#export
def checkForBreaksZoom(zoomLevel,compNum,components,fromComponentLinks,toComponentLinks):
    if compNum<len(components)-1:
        breakByLinks = checkLinksZoom(compNum,fromComponentLinks,toComponentLinks)
        breakByChange = checkChange(compNum,components,zoomLevel)
    else:
        return [false]
    return [breakByLinks,breakByChange]

In [ ]:
#export
def isStartEnd(compNum,components):
    isBreak = False
    leftComp = components[compNum]
    if compNum<len(components)-1:
        rightComp = components[compNum+1]
    else:
        rightComp = None
    
    
    for path,compInvBin,pathMatrix in leftComp['matrix']:
        if compInvBin==1:
            if path in leftComp.get('starts',[]):
                isBreak = True
        else:
            if path in leftComp.get('ends',[]):
                isBreak = True
    
    if rightComp is not None:
        for path,compInvBin,pathMatrix in rightComp['matrix']:
            if compInvBin==1:
                if path in rightComp.get('ends',[]):
                    isBreak = True
            else:
                if path in rightComp.get('starts',[]):
                    isBreak = True
    
    return isBreak

In [ ]:
#export
def nextLayerZoom(zoomLevel,components,componentLengths,#componentNucleotides,
                  fromComponentLinks,toComponentLinks,graph,
                  collapsibleBlocks,accStarts,accEnds,
                  maxLengthComponent,inversionThreshold=0.5,
                  debug=False,debugTime=False):

    print('\n===========================')
    print(f'Zoom level {zoomLevel}')
    print('===========================')
    
    # removing links and isolated components
    ### !!! Function to process these things should go here.
    
    
    numComponents = len(components)
    numComponentsDigits = np.int(np.ceil(np.log10(numComponents)))
    
    newComponent = deepcopy(componentTemplate)
    newComponents = []
    newComponentLengths = []
    newComponentNucleotides = []

    newFromComponentLinks = {}
    newToComponentLinks = {}
    
    collapsibleBlocksUpdate = []

    occupants = set()
    #nucleotides = '' 
    newToOldInd = [[]]
    oldToNewInd = []
    
    binMeanOcc = {}
    binMeanInv = {}
    binOcc = {}
    binInv = {}
    binPosArray = {}
    binAnn = {}
    
    binColStarts = []
    binColEnds = []
    binColLengths = []
    binBlockLength = 0
    binBlockLengths = []

    matrix = {}
    nBins = 0
    nCols = 0

    starts = set()
    ends = set()

    prevOcc = {}
    prevInv = {}
    
    compAccDir = {}
    
    for compNum,component in enumerate(components):
        
#         if zoomLevel==16:
#             pdb.set_trace()
        
        if debug or debugTime:
            print(f'Processing component {compNum+1:0{numComponentsDigits}}/{numComponents:0{numComponentsDigits}}')
        else:
            print(f'\nProcessing component {compNum+1:0{numComponentsDigits}}/{numComponents:0{numComponentsDigits}}',end='')
        
        forwardPaths = set(range(len(graph.accessions)))
        invertedPaths = set()
        
        oldToNewInd.append([]) # len(newComponents)
        newToOldInd[-1].append(compNum)
        occupants.update(component['occupants'])
        starts.update(component.get('starts',[]))
        ends.update(component.get('ends',[]))
        for binNum in range(0,component['last_bin']-component['first_bin']+1):
            if binBlockLength>0:
                binColStart = min(binColStart,component['binColStarts'][binNum])
                binColEnd = max(binColEnd,component['binColEnds'][binNum])
            else:
                binColStart = component['binColStarts'][binNum]
                binColEnd = component['binColEnds'][binNum]

            binColLengths.append(component['binsToCols'][binNum])
            binBlockLength += binColLengths[-1]
            for pathID,compInvBin,pathMatrix in component['matrix']:
                if compInvBin==1:
                    invertedPaths.add(pathID)
                    forwardPaths -= set([pathID])
                    compAccDir.setdefault(compNum,{})[pathID]='-'
                else:
                    compAccDir.setdefault(compNum,{})[pathID]='+'
                occupiedBins = pathMatrix[0]
                binsMatrix = pathMatrix[1]
                try:
                    # TODO: For inverted columns reading should happen from the other side to the front.
                    if compInvBin==1:
                        binPos = occupiedBins.index(componentLengths[compNum] - 1 - binNum)
                    else:
                        binPos = occupiedBins.index(binNum)
                        
                    binOcc.setdefault(pathID,[]).append(binsMatrix[binPos][0])
                    binInv.setdefault(pathID,[]).append(binsMatrix[binPos][1])
                    binPosArray.setdefault(pathID,[]).extend(binsMatrix[binPos][2])
                    binAnn.setdefault(pathID,set()).update(binsMatrix[binPos][3])
                    
#                     binColLengths.setdefault(pathID,[]).append(binColLength)
#                     binBlockLengthAddition = max([binBlockLengthAddition,binColLength])
                except ValueError:
                    continue
            
            if binBlockLength >= zoomLevel:
                # Probably this needs to be changed to binBlockLength + current bin size >= zoomLevel and 
                # not include the current bin into the new bin.
                
                # If bin got equal or larger than target zoom level bin size 
                # (it can grow over by less than previous zoom level)
                # then bin is closing and new bin will be formed.
                [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,binAnn,nBins,nCols,
                    binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,collapsibleBlocksUpdate,
                    starts,ends,newToOldInd,oldToNewInd] = \
                finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    prevOcc,prevInv,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,collapsibleBlocksUpdate,
                    starts,ends,
                    forwardPaths,invertedPaths,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=inversionThreshold)

            if nBins==maxLengthComponent:
#                 if component['first_bin']==14889:
#                     pdb.set_trace()
                # Break component due to length
#                 pdb.set_trace()
                # Finalise bin
#                 if binBlockLength>0:
#                     [binColLengths,binOcc,binInv,binPosArray,binAnn,nBins,nCols,binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
#                         newComponent,newComponents,newComponentLengths,
#                         newFromComponentLinks,newToComponentLinks,
#                         occupants,collapsibleBlocksUpdate,
#                         starts,ends,newToOldInd,oldToNewInd] = \
#                     finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
#                         nBins,nCols,
#                         binBlockLength,binColLengths,binBlockLengths,
#                         matrix,
#                         prevOcc,prevInv,
#                         newComponent,newComponents,newComponentLengths,
#                         newFromComponentLinks,newToComponentLinks,
#                         occupants,collapsibleBlocksUpdate,
#                         starts,ends,
#                         forwardPaths,invertedPaths,
#                         newToOldInd,oldToNewInd,
#                         inversionThreshold=inversionThreshold)
                
                if nBins>0:
                
                    # Add links from current component to the next one
                    if len(forwardPaths)>0:
                        addLink(len(newComponents)+1,'+',len(newComponents)+2,'+',forwardPaths,newFromComponentLinks,newToComponentLinks)
                    if len(invertedPaths)>0:
                        addLink(len(newComponents)+2,'-',len(newComponents)+1,'-',invertedPaths,newFromComponentLinks,newToComponentLinks)
                    # Add new element to newToOldInd
                    
                    if binNum==componentLengths[compNum]-1:
                        newToOldInd.append([])
                    else:
                        newToOldInd.append([compNum])
                    # Add number of next element to oldToNewInd
                    oldToNewInd[-1].append(len(newComponents))
                    
#                     keepNucl = nucleotides[-1]
                    # close component

                    newComponent,newComponents,newComponentLengths,\
                    nBins,nCols,occupants,binBlockLengths,binColStarts,binColEnds,\
                    matrix,starts,ends = \
                        finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                              nBins,nCols,occupants,binBlockLengths,
                                              binColStarts,binColEnds,
                                              matrix,starts,ends,
                                              forwardPaths,invertedPaths,inversionThreshold=inversionThreshold)
#                     nucleotides = keepNucl
                    occupants = set(component['occupants'])
#         pdb.set_trace()

        isEndBreak = isStartEnd(compNum,components)
        
        if isEndBreak and binBlockLength>0:
            [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,binAnn,nBins,nCols,
            binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
            newComponent,newComponents,newComponentLengths,
            newFromComponentLinks,newToComponentLinks,
            occupants,collapsibleBlocksUpdate,
            starts,ends,newToOldInd,oldToNewInd] = \
                finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    prevOcc,prevInv,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,collapsibleBlocksUpdate,
                    starts,ends,
                    forwardPaths,invertedPaths,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=inversionThreshold)
            
        if compNum==len(components)-1:
            # Close the bin
            if binBlockLength>0:
                [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,binAnn,nBins,nCols,
                binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
                newComponent,newComponents,newComponentLengths,
                newFromComponentLinks,newToComponentLinks,
                occupants,collapsibleBlocksUpdate,
                starts,ends,newToOldInd,oldToNewInd] = \
                    finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
                        nBins,nCols,
                        binBlockLength,binBlockLengths,binColLengths,
                        binColStart,binColStarts,binColEnd,binColEnds,
                        matrix,
                        prevOcc,prevInv,
                        newComponent,newComponents,newComponentLengths,
                        newFromComponentLinks,newToComponentLinks,
                        occupants,collapsibleBlocksUpdate,
                        starts,ends,
                        forwardPaths,invertedPaths,
                        newToOldInd,oldToNewInd,
                        inversionThreshold=inversionThreshold)
#                 [binColLengths,binOcc,binInv,binPosArray,binAnn,nBins,nCols,binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
#                     newComponent,newComponents,newComponentLengths,
#                     newFromComponentLinks,newToComponentLinks,
#                     occupants,collapsibleBlocksUpdate,
#                     starts,ends,newToOldInd,oldToNewInd] = \
#                 finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
#                     nBins,nCols,
#                     binBlockLength,binColLengths,binBlockLengths,
#                     matrix,
#                     prevOcc,prevInv,
#                     newComponent,newComponents,newComponentLengths,
#                     newFromComponentLinks,newToComponentLinks,
#                     occupants,collapsibleBlocksUpdate,
#                     starts,ends,
#                     forwardPaths,invertedPaths,
#                     newToOldInd,oldToNewInd,
#                     inversionThreshold=inversionThreshold)
            
            # break the last component
            if nBins>0:
                
                oldToNewInd[-1].append(len(newComponents))
                
                newComponent,newComponents,newComponentLengths,\
                nBins,nCols,occupants,binBlockLengths,binColStarts,binColEnds,\
                matrix,starts,ends = \
                    finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                          nBins,nCols,occupants,binBlockLengths,
                                          binColStarts,binColEnds,
                                          matrix,starts,ends,
                                          range(len(graph.accessions)),range(len(graph.accessions)),inversionThreshold=inversionThreshold)

        elif any(checkForBreaksZoom(zoomLevel,compNum,components,fromComponentLinks,toComponentLinks)):
            # Break at the end of the component because of the links
#             pdb.set_trace()
            if binBlockLength>0:
                [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,binAnn,nBins,nCols,
                binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
                newComponent,newComponents,newComponentLengths,
                newFromComponentLinks,newToComponentLinks,
                occupants,collapsibleBlocksUpdate,
                starts,ends,newToOldInd,oldToNewInd] = \
                finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    prevOcc,prevInv,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,collapsibleBlocksUpdate,
                    starts,ends,
                    forwardPaths,invertedPaths,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=inversionThreshold)
            
#                 [binColLengths,binOcc,binInv,binPosArray,binAnn,nBins,nCols,binBlockLength,binBlockLengths,matrix,prevOcc,prevInv,
#                     newComponent,newComponents,newComponentLengths,
#                     newFromComponentLinks,newToComponentLinks,
#                     occupants,collapsibleBlocksUpdate,
#                     starts,ends,newToOldInd,oldToNewInd] = \
#                 finaliseBinZoom(compNum,binOcc,binInv,binPosArray,binAnn,
#                     nBins,nCols,
#                     binBlockLength,binColLengths,binBlockLengths,
#                     matrix,
#                     prevOcc,prevInv,
#                     newComponent,newComponents,newComponentLengths,
#                     newFromComponentLinks,newToComponentLinks,
#                     occupants,collapsibleBlocksUpdate,
#                     starts,ends,
#                     forwardPaths,invertedPaths,
#                     newToOldInd,oldToNewInd,
#                     inversionThreshold=inversionThreshold)
            
            # Break component
            if nBins>0:
                
                newToOldInd.append([])
                oldToNewInd[-1].append(len(newComponents))
                
                newComponent,newComponents,newComponentLengths,\
                nBins,nCols,occupants,binBlockLengths,binColStarts,binColEnds,\
                matrix,starts,ends = \
                    finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                          nBins,nCols,occupants,binBlockLengths,
                                          binColStarts,binColEnds,
                                          matrix,starts,ends,
                                          range(len(graph.accessions)),range(len(graph.accessions)),inversionThreshold=inversionThreshold)
                
        elif nBins>0 or binBlockLength>0:
            oldToNewInd[-1].append(len(newComponents))
    
#     pdb.set_trace()
    
    newFromComponentLinks,newToComponentLinks,accStarts,accEnds,collapsibleBlocks = \
        updateLinks(newToOldInd,oldToNewInd,fromComponentLinks,toComponentLinks,collapsibleBlocks,accStarts,accEnds,
                    newComponents,compAccDir,newFromComponentLinks,newToComponentLinks)
    
    collapsibleBlocks.extend(collapsibleBlocksUpdate)
    
    print()
    
    return newComponents,newComponentLengths,\
            newFromComponentLinks,newToComponentLinks,\
            accStarts,accEnds,collapsibleBlocks
#,newToOldInd,oldToNewInd

In [ ]:
#export
def splitPositiveNegative(compID,accs,components):
    '''
    This function simply pulls all accession presented in the component and split them into forward and inversed.
    
    Parameters
    ==========
    
    `compID`: int. Number of the component in the current zoom layer (0-based).
    `accs`: int or Iterable. Should provide either overall number of accession/paths in the graph or 
            a list of all (intended) accessions for the given component. It is used only for carrying over links 
            through empty components for some accessions.
    `components`: list[dict]. List of component dictionaries, one of the main data structure representing zoom layer.
    
    Returns
    =======
    
    `posAcc`: list[int]. IDs of accession which has forward direction in given component.
    `negAcc`: list[int]. IDs of accession which has inverse direction in given component.
    
    '''
    if isinstance(accs,int):
        emptyAcc = set(range(accs))
    elif isinstance(accs,Iterable):
        emptyAcc = set(accs)
    else:
        raise TypeError(f'`accs` should be either int or Iterable, but {type(accs)} was given.')
    posAcc = []
    negAcc = []
    for pathID,pathInversion,_ in components[compID]['matrix']:
        emptyAcc -= set([pathID])
        if pathInversion==1:
            negAcc.append(pathID)
        else:
            posAcc.append(pathID)
    
#     for pathID in allAcc:
#         negAcc.append(pathID)
#         posAcc.append(pathID)
    
    return posAcc,negAcc,emptyAcc

In [ ]:
#export
def intersectAccLists(accList,dirDict):
    overallLinkAccList = set()
    for linkAccList in dirDict.values():
        overallLinkAccList.update(linkAccList)
    return set(accList).intersection(overallLinkAccList)

In [ ]:
#export
def updateLinks(newToOldInd,oldToNewInd,
                fromComponentLinks,toComponentLinks,
                collapsibleBlocks,accStarts,accEnds,components,compAccDir,
                newFromComponentLinks={},newToComponentLinks={}):
    '''
    newToOldInd and oldToNewInd: both index and values are 0-based numbers of components 
    in previous and current zoomlayer.
    
    
    '''
    
    for newComp,oldCompList in enumerate(newToOldInd):
        
        
        leftOldCompId = oldCompList[0] + 1
        rightOldCompId = oldCompList[-1] + 1
        newCompId = newComp + 1
        
        newCompPosAcc,newCompNegAcc,newCompEmptyAcc = splitPositiveNegative(newComp,list(accStarts.keys()),components)
        
        # The next two if blocks set conditions for processing or not processing left and right of the current new component
        # The side should not be processed if the end of the previous node is coming from the same component 
        # (on lower zoom level) as the begininng of the current component.
        if newComp>0:
            if oldCompList[0]==newToOldInd[newComp-1][-1]:
                doLeft = False
            else:
                doLeft = True
        else:
            doLeft = True
            
        if newComp<len(newToOldInd)-1:
            if oldCompList[-1]==newToOldInd[newComp+1][0]:
                doRight = False
            else:
                doRight = True
        else:
            doRight = True
        
        # Departure on the right (from positive block)
        if doRight:
#             for fromComp in [leftOldCompId,rightOldCompId]:
            fromComp = rightOldCompId
            for toCompDict in fromComponentLinks.get(fromComp,{}).values():
                for toOldComp,toOldCompDirDict in toCompDict.items():
                    # To positive strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][0]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[toNewCompId][0]==toOldComp-1:
                        # Getting accession list which come out from positive and goes to positive.
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(toCompPosAcc)),
                                                    toOldCompDirDict)
    #                     accList = list(set(newCompPosAcc).intersection(toCompPosAcc).intersection(toOldCompDirDict.get('+',[])))

                        if len(accList)>0:
                            if not(toNewCompId+1==newCompId and \
                                (fromComp-1!=newToOldInd[toNewCompId][-1] or toOldComp-1!=newToOldInd[toNewCompId][0])):
                                # If the link is self loop (which happens sometimes), then it has to come out from right component
                                # and go to the left component, otherwise it should not be carried forward.
                                addLink(newCompId,'+',toNewCompId+1,'+',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)
                        
                        ### Wrap creating empty links into a separate function!!!
                        if fromComp+1==toOldComp and newCompId+1==toNewCompId+1:
                            continuityLinks = set(fromComponentLinks.get(fromComp,{}).get('+',{}).get(fromComp+1,{}).get('+',[]))
                            emptyLinks = (set(newCompPosAcc)|set(newCompEmptyAcc)).intersection(set(toCompPosAcc)|set(toCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(newCompId,'+',toNewCompId+1,'+',emptyLinks,newFromComponentLinks,newToComponentLinks)

                    # To negative strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][-1]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the right of the new comp
                    if newToOldInd[toNewCompId][-1]==toOldComp-1:
                        # Getting accession list which come out from positive and goes to negative.
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(toCompNegAcc)),
                                                    toOldCompDirDict)
    #                     accList = list(set(newCompPosAcc).intersection(toCompNegAcc).intersection(toOldCompDirDict.get('-',[])))
                        if len(accList)>0:
                            addLink(newCompId,'+',toNewCompId+1,'-',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)
        
        # Departure on the left (from negative block)
        if doLeft:
#             for fromComp in [leftOldCompId,rightOldCompId]:
            fromComp = leftOldCompId
#             toCompDict = fromComponentLinks.get(fromComp,{}).get('-',{})
            for toCompDict in fromComponentLinks.get(fromComp,{}).values():
                for toOldComp,toOldCompDirDict in toCompDict.items():
                    # To positive strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][0]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[toNewCompId][0]==toOldComp-1:
                        # Getting accession list which come out from negative and goes to positive.
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(toCompPosAcc)),
                                                    toOldCompDirDict)
    #                     accList = list(set(newCompNegAcc).intersection(toCompPosAcc).intersection(toOldCompDirDict.get('+',[])))
                        if len(accList)>0:
                            addLink(newCompId,'-',toNewCompId+1,'+',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)

                    # to Negative strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][-1]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the right of the new comp
                    if newToOldInd[toNewCompId][-1]==toOldComp-1:
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(toCompNegAcc)),
                                                    toOldCompDirDict)
    #                     accList = list(set(newCompNegAcc).intersection(toCompNegAcc).intersection(toOldCompDirDict.get('-',[])))
                        if len(accList)>0:
                            if not (toNewCompId+1==newCompId and \
                                (fromComp-1!=newToOldInd[toNewCompId][0] or toOldComp-1!=newToOldInd[toNewCompId][-1])):
                                # If the link is self loop (which happens sometimes), then it has to come out from left component
                                # and go to the right component, otherwise it should not be carried forward.
                                addLink(newCompId,'-',toNewCompId+1,'-',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)

                        if fromComp-1==toOldComp and newCompId-1==toNewCompId+1:
                            continuityLinks = set(fromComponentLinks.get(fromComp,{}).get('-',{}).get(fromComp+1,{}).get('-',[]))
                            emptyLinks = (set(newCompNegAcc)|set(newCompEmptyAcc)).intersection(set(toCompNegAcc)|set(toCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(newCompId,'-',toNewCompId+1,'-',emptyLinks,newFromComponentLinks,newToComponentLinks)


        # Arrival on the left (to positive block)
        if doLeft:
#             for toComp in [leftOldCompId,rightOldCompId]:
            toComp = leftOldCompId
#             fromCompDict = toComponentLinks.get(toComp,{}).get('+',{})
            for fromCompDict in toComponentLinks.get(toComp,{}).values():
                for fromOldComp,fromOldCompDirDict in fromCompDict.items():
                    # From positive strand
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the from comp (old) is on the right of the new comp
                    if newToOldInd[fromNewCompId][-1]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(fromCompPosAcc)),
                                                    fromOldCompDirDict)
    #                     accList = list(set(newCompPosAcc).intersection(fromCompPosAcc).intersection(fromOldCompDirDict.get('+',[])))
                        if len(accList)>0:
                            if not (fromNewCompId+1==newCompId and \
                                (fromOldComp-1!=newToOldInd[fromNewCompId][-1] or toComp-1!=newToOldInd[fromNewCompId][0])):
                                # If the link is self loop (which happens sometimes), then it has to come out from right component
                                # and go to the left component, otherwise it should not be carried forward.
                                addLink(fromNewCompId+1,'+',newCompId,'+',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)
                        if toComp-1==fromOldComp and newCompId-1==fromNewCompId+1:
                            continuityLinks = set(toComponentLinks.get(toComp,{}).get('+',{}).get(toComp-1,{}).get('+',[]))
                            emptyLinks = (set(newCompPosAcc)|set(newCompEmptyAcc)).intersection(set(fromCompPosAcc)|set(fromCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(fromNewCompId+1,'+',newCompId,'+',emptyLinks,newFromComponentLinks,newToComponentLinks)

                    # From negative strand
                    # Getting to comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][0]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[fromNewCompId][0]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(fromCompNegAcc)),
                                                    fromOldCompDirDict)
    #                     accList = list(set(newCompPosAcc).intersection(fromCompNegAcc).intersection(fromOldCompDirDict.get('-',[])))
                        if len(accList)>0:
                            addLink(fromNewCompId+1,'-',newCompId,'+',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)

        if doRight:
            # Arrival on the right (to negative block)
#             for toComp in [leftOldCompId,rightOldCompId]:
            toComp = rightOldCompId
#             fromCompDict = toComponentLinks.get(toComp,{}).get('-',{})
            for fromCompDict in toComponentLinks.get(toComp,{}).values():
                for fromOldComp,fromOldCompDirDict in fromCompDict.items():
                    # From positive strand
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the from comp (old) is on the right of the new comp
                    if newToOldInd[fromNewCompId][0]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(fromCompPosAcc)),
                                                    fromOldCompDirDict)
    #                     accList = list(set(newCompNegAcc).intersection(fromCompPosAcc).intersection(fromOldCompDirDict.get('+',[])))
                        if len(accList)>0:
                            addLink(fromNewCompId+1,'+',newCompId,'-',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)

                    # From negative strand
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][0]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the from comp (old) is on the left of the new comp
                    if newToOldInd[fromNewCompId][-1]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(fromCompNegAcc)),
                                                    fromOldCompDirDict)
    #                     accList = list(set(newCompNegAcc).intersection(fromCompNegAcc).intersection(fromOldCompDirDict.get('-',[])))
                        if len(accList)>0:
                            if not (fromNewCompId+1==newCompId and \
                                (fromOldComp-1!=newToOldInd[fromNewCompId][0] or toComp-1!=newToOldInd[fromNewCompId][-1])):
                                # If the link is self loop (which happens sometimes), then it has to come out from left component
                                # and go to the right component, otherwise it should not be carried forward.
                                addLink(fromNewCompId+1,'-',newCompId,'-',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)

                        if toComp+1==fromOldComp and newCompId+1==fromNewCompId+1:
                            continuityLinks = set(toComponentLinks.get(toComp,{}).get('-',{}).get(toComp-1,{}).get('-',[]))
                            emptyLinks = (set(newCompNegAcc)|set(newCompEmptyAcc)).intersection(set(fromCompNegAcc)|set(fromCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(fromNewCompId+1,'-',newCompId,'-',emptyLinks,newFromComponentLinks,newToComponentLinks)
    
    for acc in accStarts.keys():
        accStarts[acc] = oldToNewInd[accStarts[acc]-1][0]+1
        accEnds[acc] = oldToNewInd[accEnds[acc]-1][0]+1
        
    for blockID,[accessionID,blockLength,associatedLinks,associatedComponents] in enumerate(collapsibleBlocks):
#         if zoomLevel==144 and accessionID==13 and blockLength==378:
#             pdb.set_trace()
        collapsibleBlocks[blockID] = (accessionID,blockLength,
                  [(oldToNewInd[link[0]-1][0 if compAccDir[link[0]-1].get(accessionID,'+')=='-' else -1]+1,\
                    oldToNewInd[link[1]-1][0 if compAccDir[link[1]-1].get(accessionID,'+')=='+' else -1]+1) \
                   for link in associatedLinks],
                  [compID+1 for comp in associatedComponents for compID in oldToNewInd[comp-1]])
    
    return newFromComponentLinks,newToComponentLinks,accStarts,accEnds,collapsibleBlocks

In [ ]:
#export
def calcLengthBlock(startComp,endComp,components):
    blockLength = 0
    for comp in range(startComp,endComp+1):
        blockLength += compLength(comp,components)
        
    return blockLength

In [ ]:
#export
def processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components):
    
    # Potentially, we need to change this process to use actual paths and take into account the cardinalities/relation of the arrows
    # This will make it slower, but more accurate. That also will make linking through removed links more accurate as well.
    
    blockSet = set()
    
    accessionFromLinks = fromComponentLinksAcc[accID]
    compDir = accCompDir[accID]
    endComp = accEnds[accID]
    
    block = [toComp]
    blockLength = compLength(toComp,components)
    associatedLinks = set()
    if fromComp is not None:
        associatedLinks.add((fromComp,toComp))
    
    curComp = toComp
    
    while True:
        nextCompList = np.array(list(accessionFromLinks.get(curComp,set())))
#         distToNextComp = nextCompList-curComp # Do I need it?
        
        nextCompFollow = None
        linksToAssociate = set()
        
        if compDir[curComp]=='+':
            for nextComp in nextCompList:
                if compDir[nextComp]=='+' and nextComp==curComp+1:
                    nextCompFollow = nextComp
                else:
                    linksToAssociate.add((curComp,nextComp))
        elif compDir[curComp]=='-':
            for nextComp in nextCompList:
              
                if compDir[nextComp]=='-' and nextComp==curComp-1:
                    nextCompFollow = nextComp
                else:
                    linksToAssociate.add((curComp,nextComp))

        if nextCompFollow:
#             if len(linksToAssociate)==0:
#                 pdb.set_trace()
            if len(linksToAssociate)>0:
                # This row adds collapsible block for the case where we can continue to follow the block, but the there is an outgoing arrow
                # This usually happens in case of partial repeats.
                # There two options here:
                # - add associatedLinks, which contains original incoming arrow. In this case original incoming arrow will be removed with the smallest block
                # - do not add associatedLinks. In this case, the original incoming arrow will be removed only with the largest block.
                blockSet.add((accID,blockLength,tuple(associatedLinks | linksToAssociate),tuple(deepcopy(block))))
        else:
            # Potentially, we would want to stop the block following if we found another incoming arrow, 
            # but that is not that simple as this can cause breaking of large repeat due to small extra repeat inside.
            associatedLinks.update(linksToAssociate)
            break
        
        blockLength += compLength(nextCompFollow,components)
        block.append(nextCompFollow)
        curComp = nextCompFollow
    
#     if len(associatedLinks)==0:
#         pdb.set_trace()
    if curComp!=endComp:
        blockSet.add((accID,blockLength,tuple(associatedLinks),tuple(block)))
        
    if fromComp is not None:
        if np.abs(toComp-fromComp)>1:
            if fromComp<toComp:
                blockSet.add((accID,calcLengthBlock(fromComp+1,toComp-1,components),tuple([(fromComp,toComp)]),tuple(list(range(fromComp+1,toComp)))))
            elif fromComp>toComp:
                blockSet.add((accID,calcLengthBlock(toComp+1,fromComp-1,components),tuple([(fromComp,toComp)]),tuple(list(range(toComp+1,fromComp)))))
    
    return blockSet

In [ ]:
#export
def updateCollapsibleBlockDict(collapsibleBlocks,listOfUpdates):
    for accessionID,blockLength,associatedLinks,associatedBlocks in listOfUpdates:
        lengthAccessionComb = collapsibleBlocks.setdefault(blockLength,{}).setdefault(accessionID,(set(),set()))
        lengthAccessionComb[0].update(associatedLinks)
        lengthAccessionComb[1].update(associatedBlocks)

In [ ]:
#export
def identifyCollapsibleBlocks(toComponentLinks,fromComponentLinksAcc,components,accStarts,accEnds,accCompDir):
    print('Identifying collapsible blocks!')
    
    collapsibleBlocks = {}
#     collapsibleBlocks = set()
#     blockStarts = set()
    for toComp,toStrandDict in toComponentLinks.items():
        for toStrand,fromCompDict in toStrandDict.items():
            for fromComp,fromStrandDict in fromCompDict.items():
                for fromStrand,accessionSet in fromStrandDict.items():
                    if toStrand!=fromStrand:
                        for accID in accessionSet:
                            # collapsibleBlocks.update(processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
                            updateCollapsibleBlockDict(collapsibleBlocks,
                                                       processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
                                                                                                              
                    elif toStrand=='+' and fromComp+1!=toComp:
                        for accID in accessionSet:
                            # collapsibleBlocks.update(processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
                            updateCollapsibleBlockDict(collapsibleBlocks,
                                                       processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
                    elif toStrand=='-' and fromComp-1!=toComp:
                        for accID in accessionSet:
                            # collapsibleBlocks.update(processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
                            updateCollapsibleBlockDict(collapsibleBlocks,
                                                       processBlock(fromComp,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
                            
#     for accID,toComp in accStarts.items():
#         # collapsibleBlocks.update(processBlock(None,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
#         updateCollapsibleBlockDict(collapsibleBlocks,
#                                    processBlock(None,toComp,accID,accEnds,accCompDir,fromComponentLinksAcc,components))
    
    _collapsibleBlocks = []
    for blockLength,accessionDict in collapsibleBlocks.items():
        for accessionID,[associatedLinksSet,associatedCompSet] in accessionDict.items():
            _collapsibleBlocks.append((accessionID,blockLength,list(associatedLinksSet),list(associatedCompSet)))
    
    print(f'Identified {len(_collapsibleBlocks)} collapsible block lengths')
    return _collapsibleBlocks

In [ ]:
#export
def compLinksToAccCompLinks(compLinks,doCompDir=False):
    accCompLinks = {}
    if doCompDir:
        accCompDir = {}
    
    for comp1,comp1Dict in compLinks.items():
        for strand1,strand1Dict in comp1Dict.items():
            for comp2,comp2Dict in strand1Dict.items():
                for strand2,accList in comp2Dict.items():
                    for acc in accList:
                        # Do we need to include strand or component only is enough. Here are two options, just choose one:
#                         accCompLinks.setdefault(acc,{}).setdefault((comp1,strand1),set()).add((comp2,strand2))
                        accCompLinks.setdefault(acc,{}).setdefault(comp1,set()).add(comp2)
                        
                        if doCompDir:
                            accCompDir.setdefault(acc,{})[comp1] = strand1
                            accCompDir.setdefault(acc,{})[comp2] = strand2
    if doCompDir:
        return accCompLinks,accCompDir
    else:
        return accCompLinks

In [ ]:
#export
def compLength(compID,components):
    return components[compID-1]['lastCol']-components[compID-1]['firstCol'] + 1

In [ ]:
#export
def removeLink(fromComp,toComp,accessionID,fromComponentLinks,toComponentLinks):
    '''
    At the moment this function will remove link from `fromComp` to 'toComp' for 
    specific accession `accessionID` irrespective of strands.
    
    If any dict on any hierarchical level gets empty, it will be removed as well.
    
    
    '''
    
    # removing links from fromComponentLinks
    fromStrandsToRemove = []
    for fromStrand,fromStrandDict in fromComponentLinks.get(fromComp,{}).items():
        toStrandsToRemove = []
            
        for toStrand,accessionList in fromStrandDict.get(toComp,{}).items():
            try:
                accessionList.remove(accessionID)
            except KeyError:
                pass
            if len(accessionList)==0:
                toStrandsToRemove.append(toStrand)
        
        for toStrand in toStrandsToRemove:
            del fromStrandDict[toComp][toStrand]
        if toComp in fromStrandDict:
            if len(fromStrandDict[toComp])==0:
                del fromStrandDict[toComp]
        
        if len(fromStrandDict)==0:
            fromStrandsToRemove.append(fromStrand)
    
    for fromStrand in fromStrandsToRemove:
        del fromComponentLinks[fromComp][fromStrand]
    
    if fromComp in fromComponentLinks:
        if len(fromComponentLinks[fromComp])==0:
            del fromComponentLinks[fromComp]
            
    # removing links from toComponentLinks
    toStrandsToRemove = []
    for toStrand,toStrandDict in toComponentLinks.get(toComp,{}).items():
        fromStrandsToRemove = []
            
        for fromStrand,accessionList in toStrandDict.get(fromComp,{}).items():
            try:
                accessionList.remove(accessionID)
            except KeyError:
                pass
            if len(accessionList)==0:
                fromStrandsToRemove.append(fromStrand)
        
        for fromStrand in fromStrandsToRemove:
            del toStrandDict[fromComp][fromStrand]
        if fromComp in toStrandDict:
            if len(toStrandDict[fromComp])==0:
                del toStrandDict[fromComp]
        
        if len(toStrandDict)==0:
            toStrandsToRemove.append(toStrand)
    
    for toStrand in toStrandsToRemove:
        del toComponentLinks[toComp][toStrand]
    
    if toComp in toComponentLinks:
        if len(toComponentLinks[toComp])==0:
            del toComponentLinks[toComp]

In [ ]:
#export
def processCollapsibleBlocks(zoomLevel,collapsibleBlocks,fromComponentLinks,toComponentLinks):
#     if zoomLevel==576:
#         pdb.set_trace()
    
    idToRemove = []
    for blockID,[accessionID,blockLength,linkList,componentList] in enumerate(collapsibleBlocks):
        if blockLength<zoomLevel:
            idToRemove.append(blockID)
            for link in linkList:
                removeLink(*link,accessionID,fromComponentLinks,toComponentLinks)

    # Remove blocks that already been cleared
    for blockID in sorted(idToRemove,reverse=True):
        del collapsibleBlocks[blockID]

In [ ]:
#export
def testStartEnd(compNum,isLeft,components,accStarts,accEnds):
    if isLeft:
        startInv = 0
        endInv = 1
    else:
        startInv = 1
        endInv = 0
        
    for accStart,startComp in accStarts.items():
        if startComp == compNum:
            comp = components[compNum-1]
            try:
                accessionInd = comp['occupants'].index(accStart)
            except ValueError:
                continue
            if comp['matrix'][accessionInd][1]==startInv:
                return True
    
    for accEnd,endComp in accEnds.items():
        if endComp == compNum:
            comp = components[compNum-1]
            try:
                accessionInd = comp['occupants'].index(accStart)
            except ValueError:
                continue
            if comp['matrix'][accessionInd][1]==endInv:
                return True
        
    return False

In [ ]:
#export
def findEmptyEdges(fromComponentLinks,toComponentLinks,accStarts,accEnds,components):
    '''
    Identify all empty edges by simply finding components that do not appear either in toComponentLinks (left empty)
    or fromComponentLinks (right empty)
    '''
    
    allCompSet = set((np.array(range(len(components)))+1).tolist())
    toLinkedSet = set(toComponentLinks.keys())
    fromLinkedSet = set(fromComponentLinks.keys())

    leftEmptyCandidates = list(allCompSet - toLinkedSet)

    leftEmptyList = []

    for leftEmptyCand in leftEmptyCandidates:
        if testStartEnd(leftEmptyCand,True,components,accStarts,accEnds):
            continue

        if '-' in fromComponentLinks.get(leftEmptyCand,{}):
            continue

        leftEmptyList.append(leftEmptyCand)

    rightEmptyCandidates = list(allCompSet - fromLinkedSet)

    rightEmptyList = []

    for rightEmptyCand in rightEmptyCandidates:
        if testStartEnd(rightEmptyCand,False,components,accStarts,accEnds):
            continue

        if '-' in toComponentLinks.get(rightEmptyCand,{}):
            continue

        rightEmptyList.append(rightEmptyCand)
        
    return leftEmptyList,rightEmptyList

In [ ]:
#export
def checkExternalLinks(blockStart,blockEnd,fromComponentLinks,toComponentLinks,components):
    '''
    
    
    Return
    ======
    
    componentID: int. If an external links found, then a list of components inside the block involved 
                    in external links returned. If no external links found, then empty list is returned.
    '''
    
    externalLinksComponents = []
    
    for compID in range(blockStart,blockEnd+1):
        passToNext = False
        
        if len(components[compID-1].get('ends',[]))>0 or len(components[compID-1].get('starts',[]))>0:
            externalLinksComponents.append(compID)
            continue
        
        for fromStrandDict in fromComponentLinks.get(compID,{}).values():
            for toComp in fromStrandDict.keys():
                if toComp>blockEnd or toComp<blockStart:
                    externalLinksComponents.append(compID)
                    passToNext = True
                    break
            if passToNext:
                break
        
        if passToNext:
            continue
            
        for toStrandDict in toComponentLinks.get(compID,{}).values():
            for fromComp in toStrandDict.keys():
                if fromComp>blockEnd or fromComp<blockStart:
                    externalLinksComponents.append(compID)
                    passToNext = True
                    break
            if passToNext:
                break
    
    return externalLinksComponents

In [ ]:
#export
def createNewBoundaries(blockStart,blockEnd,externalLinksComps):
    blockSplit = [[blockStart]]
    
    previousBreakPoint = blockStart
    for breakPoint in externalLinksComps:
        
        if breakPoint==blockSplit[-1][0]:
            blockSplit[-1][0] = breakPoint+1
            
        else:
            blockSplit[-1].append(breakPoint-1)
            blockSplit.append([breakPoint+1])
        
    if blockEnd>=blockSplit[-1][0]:
        blockSplit[-1].append(blockEnd)
    else:
        del blockSplit[-1]

    return blockSplit

In [ ]:
#export
def identifyIsolatedBlocks(leftEmptyList,rightEmptyList,fromComponentLinks,toComponentLinks,components):
    if len(leftEmptyList)==0 or len(rightEmptyList)==0:
        return []
    
    leftmostLeftEmpty = min(leftEmptyList)
    rightmostRightEmpty = max(rightEmptyList)

    isolatedBlockCandidates = []
    isolatedBlockList = []

    if leftmostLeftEmpty<=rightmostRightEmpty:
        isolatedBlockCandidates.append([leftmostLeftEmpty,rightmostRightEmpty])

    while len(isolatedBlockCandidates)>0:

        isolatedBlock = isolatedBlockCandidates.pop()

        externalLinksComps = checkExternalLinks(*isolatedBlock,fromComponentLinks,toComponentLinks,components)

        if len(externalLinksComps)==0:
            isolatedBlockList.append(isolatedBlock)
        else:
            isolatedBlockCandidates.extend(createNewBoundaries(*isolatedBlock,externalLinksComps))
    
    return isolatedBlockList

In [ ]:
#export
def updateLinksRemoveComp(oldToNewInd,fromComponentLinks,toComponentLinks,collapsibleBlocks,accStarts,accEnds):
    
    newFromComponentLinks={}
    newToComponentLinks={}
    
    for oldFromComp,fromCompDict in fromComponentLinks.items():
        if oldFromComp-1 in oldToNewInd:
            newFromCompDict = newFromComponentLinks.setdefault(oldToNewInd[oldFromComp-1][0]+1,{})
            for fromStrand,fromStrandDict in fromCompDict.items():
                newFromStrandDict = newFromCompDict.setdefault(fromStrand,{})
                for toComp,toCompDict in fromStrandDict.items():
                    if toComp-1 in oldToNewInd:
                        newFromStrandDict[oldToNewInd[toComp-1][0]+1] = toCompDict
                
    for oldToComp,toCompDict in toComponentLinks.items():
        if oldToComp-1 in oldToNewInd:
            newToCompDict = newToComponentLinks.setdefault(oldToNewInd[oldToComp-1][0]+1,{})
            for toStrand,toStrandDict in toCompDict.items():
                newToStrandDict = newToCompDict.setdefault(toStrand,{})
                for fromComp,fromCompDict in toStrandDict.items():
                    if fromComp-1 in oldToNewInd:
                        newToStrandDict[oldToNewInd[fromComp-1][0]+1] = fromCompDict
    
    for acc in accStarts.keys():
        accStarts[acc] = oldToNewInd[accStarts[acc]-1][0]+1
        accEnds[acc] = oldToNewInd[accEnds[acc]-1][0]+1
    
    collapsibleBlocksToRemove = []
    for blockID,[accessionID,blockLength,associatedLinks,associatedComponents] in enumerate(collapsibleBlocks[::-1]):
        collapsibleBlocks[blockID] = (accessionID,blockLength,
                                      [[oldToNewInd[link[0]-1][0]+1,oldToNewInd[link[1]-1][0]+1] for link in associatedLinks \
                                           if link[0]-1 in oldToNewInd and link[1]-1 in oldToNewInd],
                                      [oldToNewInd[comp-1][0]+1 for comp in associatedComponents \
                                           if comp-1 in oldToNewInd])
        if len(collapsibleBlocks[blockID][2])==0:
            collapsibleBlocksToRemove.append(blockID)
    
    if len(collapsibleBlocksToRemove)>0:
        for blockID in sorted(collapsibleBlocksToRemove,reverse=True):
            del collapsibleBlocks[blockID]
    
    return newFromComponentLinks,newToComponentLinks,accStarts,accEnds,collapsibleBlocks

In [ ]:
#export
def removeIsolatedBlocks(isolatedBlockList,components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks):
    if len(isolatedBlockList)==0:
        return components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks
    
    newToOldInd = list(range(len(components)))
    
    for isolatedBlockStart,isolatedBlockEnd in sorted(isolatedBlockList,key=lambda el: el[1],reverse=True):
        # deleting components and creating translation from new index to old index
        for compToRemove in range(isolatedBlockEnd,isolatedBlockStart-1,-1):
            del components[compToRemove-1]
            del componentLengths[compToRemove-1]
            del newToOldInd[compToRemove-1]
    
    # Converting new to old index to old to new index
    oldToNewInd = {el:[ind] for ind,el in enumerate(newToOldInd)}
    # Converting new to old index from list to dict (not sure it is needed in the first place)
#     newToOldInd = {ind:[el] for ind,el in enumerate(newToOldInd)}
    
    
    # updating links
    fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks = \
        updateLinksRemoveComp(oldToNewInd,fromComponentLinks,toComponentLinks,collapsibleBlocks,accStarts,accEnds)
    return components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks

In [ ]:
#export
def clearInvisible(zoomLevel,collapsibleBlocks,fromComponentLinks,toComponentLinks,accStarts,accEnds,components,componentLengths):
    print('Removing links according to collapsible blocks')
    numOfCollapsibleBlocks = len(collapsibleBlocks)
    numOfComponents = len(components)
    # Link removal
    processCollapsibleBlocks(zoomLevel,collapsibleBlocks,fromComponentLinks,toComponentLinks)
    
    # Identify empty (without links) edges
    leftEmptyList,rightEmptyList = findEmptyEdges(fromComponentLinks,toComponentLinks,accStarts,accEnds,components)

    # Identify isolated blocks ,i.e. without any links on both sides and no outside links from inside
    isolatedBlockList = identifyIsolatedBlocks(leftEmptyList,rightEmptyList,fromComponentLinks,toComponentLinks,components)

    # Remove identified isolated blocks
    components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks = \
    removeIsolatedBlocks(isolatedBlockList,components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks)
    
    print(f'All links associated with collapsibleComponents <{zoomLevel} were removed. Used {numOfCollapsibleBlocks - len(collapsibleBlocks)}, \
    {numOfComponents-len(components)} were deleted as isolated.')
    
    return components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks

In [ ]:
#export
def exportToPantograph(graph=None, inputPath=None, GenomeGraphParams={}, 
                                    outputPath=None, outputName=None, outputSuffix=None, 
                                    isSeq=True,
                                    redisConn=None,
                                    listOfExports=['schematise', 'genomeToPangenome', 'annotationToGenome'],
                                    zoomLevels=[1], fillZoomLevels=True, maxLengthComponent=100, maxLengthChunk=20, inversionThreshold=0.5,
                                    debug=False, returnDebugData=False):
    
    if graph is None:
        if inputPath is not None:
            print('Loading Genome')
            graph = GenomeGraph(gfaPath=inputPath, isGFASeq=isSeq, **GenomeGraphParams)
        else:
            raise ValueError("Either graph or inputpath to GFA file should be provided")

    if outputPath is None and outputName is None:
        if inputPath is not None:
            if outputSuffix is not None:
                outputPath, outputName = pathConvert(inputPath, suffix = outputSuffix)
            else:
                outputPath, outputName = pathConvert(inputPath)

        else:
            raise ValueError("If inputPath is not given, then outputPath and outputName should be provided.")
    else:
        if outputSuffix is not None:
            outputName = outputName + outputSuffix
    print(f'Recording Pantograph data to {outputPath}{os.path.sep}{outputName}')

#     numNodes = len(graph.nodes)
#     numNodesDigits = np.int(np.ceil(np.log10(numNodes)))
#     fromLinks = {}
#     toLinks = {}

    rootStruct = deepcopy(rootStructTemplate)
    rootStruct["pathNames"] = graph.accessions

    if returnDebugData:
        # temporary structure for testing, in normal mode each zoomlevel 
        # should be just saved as soon as it is processed and only previous and current zoom level should be preserved.
        zoomComponentLengths = {}
        zoomNodeToComponent = {}
        zoomComponentToNodes = {}
        zoomComponents = {}  
        zoomCompNucleotides = {}
        zoomAccStarts = {}
        zoomAccEnds = {}

    nodeLengths = calcNodeLengths(graph)

    pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(graph,nodeLengths)
    
    #!!!!!! Process lowest level zoom to create component structure.
    [numNodes, # number of nodes
    numNodesDigits, # 
    nodeToComponent,
    componentToNode,
    componentLengths,
    components,
    componentNucleotides,
    fromLinks,toLinks,
    fromComponentLinks,toComponentLinks,
    accStarts,accEnds,invertedStarts,invertedEnds] = baseLayerZoom(graph,
                                      outputPath,outputName,
                                      pathNodeArray,pathDirArray,
                                      pathLengths,nodeLengths,
                                      pathNodeLengthsCum,
                                      maxLengthComponent,
                                      inversionThreshold=inversionThreshold,
                                      isSeq=isSeq)
    print()
    if returnDebugData:
        zoomNodeToComponent,zoomComponentToNodes,zoomComponentLengths, \
        zoomComponents,zoomCompNucleotides,zoomAccStarts,zoomAccEnds = \
        recordZoomLevelForDebug(zoomNodeToComponent,zoomComponentToNodes,zoomComponentLengths,zoomComponents,zoomCompNucleotides,zoomAccStarts,zoomAccEnds,
                                nodeToComponent,componentToNode,componentLengths,components,componentNucleotides,accStarts,accEnds,
                                1)
    
    fromComponentLinks,toComponentLinks = \
    nodeToComponentLinks(components,
                         componentToNode,
                         nodeToComponent,
                         fromLinks,
                         toLinks,
                         fromComponentLinks,
                         toComponentLinks)
    
    
    rootStruct = deepcopy(rootStructTemplate)
    rootStruct["pathNames"] = graph.accessions
    rootStruct["pangenome_length"] = np.sum(componentLengths)

    exportLayer(1,components,componentNucleotides,
                fromComponentLinks,toComponentLinks,
                rootStruct,
                outputPath,outputName,
                maxLengthComponent,maxLengthChunk,
                inversionThreshold=inversionThreshold,
                redisConn=redisConn)
    
    if len(zoomLevels)>1 or fillZoomLevels:
        # Converting link dicts from component first to path/accession first. 
        # It is needed only for collapsible block identification which happens only once.
        # So, they should be removed after collapsible blocks are removed.
        # First line also returns dictionary giving direction (inversion) of each component in each accession.
        fromComponentLinksAcc,accCompDir = compLinksToAccCompLinks(fromComponentLinks,doCompDir=True)
#         toComponentLinksAcc = compLinksToAccCompLinks(toComponentLinks)

#         # This identifies collapsible block starts based on links.
#         # For forward components
#         positiveBlockStarts = getBackwardPositiveLinks(toComponentLinks,toComponentLinksAcc,accStarts,accEnds)
#         # For inversed components
#         negativeBlockStarts = getForwardNegativeLinks(toComponentLinks,toComponentLinksAcc,accStarts,accEnds,invertedStarts)

        # Identifying all collapsible blocks.
        # The return is a list of tuples containg the following elements:
        # accession ID, length of block, list of associated links, list of associated components.
        collapsibleBlocks = identifyCollapsibleBlocks(toComponentLinks,fromComponentLinksAcc,components,accStarts,accEnds,accCompDir)
#         collapsibleBlocks = identifyCollapsibleBlocks(positiveBlockStarts,negativeBlockStarts,
#                                                       accStarts,accEnds,
#                                                       fromComponentLinksAcc,toComponentLinksAcc,accCompDir,
#                                                       components)

        if returnDebugData:
            retCollapsibleBlocks = deepcopy(collapsibleBlocks)

        del fromComponentLinksAcc
#         del toComponentLinksAcc
        del accCompDir
#         del positiveBlockStarts
#         del negativeBlockStarts
    elif returnDebugData:
        retCollapsibleBlocks = []

    _zoomLevels = deepcopy(zoomLevels)
    
    if fillZoomLevels:
        maxBlock = max(collapsibleBlocks,key=lambda bl: bl[1])

        while _zoomLevels[-1]<maxBlock[1]:
            _zoomLevels.append(_zoomLevels[-1]*2)
    
    for zoomLevel in _zoomLevels[1:]:
        
        # Collapsing is done through removing some particular links (not satisfying specific conditions)
        components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks = \
        clearInvisible(zoomLevel,collapsibleBlocks,fromComponentLinks,toComponentLinks,
                       accStarts,accEnds,components,componentLengths)
        
        # Process next zoom level
        components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,collapsibleBlocks = \
        nextLayerZoom(zoomLevel,
                      components,componentLengths,#componentNucleotides,
                      fromComponentLinks,toComponentLinks,graph,
                      collapsibleBlocks,accStarts,accEnds,
                      maxLengthComponent,inversionThreshold=inversionThreshold)
        
        if returnDebugData:
            zoomNodeToComponent,zoomComponentToNodes,zoomComponentLengths, \
            zoomComponents,zoomCompNucleotides,zoomAccStarts,zoomAccEnds = \
            recordZoomLevelForDebug(zoomNodeToComponent,zoomComponentToNodes,zoomComponentLengths,zoomComponents,zoomCompNucleotides,zoomAccStarts,zoomAccEnds,
                                    [],[],componentLengths,components,[],[],[],
                                    zoomLevel)
        # Zoom level should be split into chunks and recorded.
        exportLayer(zoomLevel,components,[],#componentNucleotides,
                fromComponentLinks,toComponentLinks,
                rootStruct,
                outputPath,outputName,
                maxLengthComponent,maxLengthChunk,
                inversionThreshold=inversionThreshold,
                redisConn=redisConn)
        
    
    with open(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}bin2file.json','w') as f:
        json.dump(rootStruct,f,cls=NpEncoder)
    
    if returnDebugData:
        return zoomComponentLengths,zoomNodeToComponent,zoomComponentToNodes,zoomComponents,zoomCompNucleotides,zoomAccStarts,zoomAccEnds, \
                invertedStarts,invertedEnds,toComponentLinks,fromComponentLinks,retCollapsibleBlocks,fromLinks,toLinks,graph,rootStruct

components: keys: 0-based, values: occupants: 0-based, binNumbers: 0-based  
componentToNode: keys: 0-based, values: 1-based  
nodeToComponent: keys: 0-based, values: 1-based  
fromLinks: top level keys (from nodes): 1-based, bottom level keys (to nodes): 1-based, values (list of participants): 0-based  
toLinks: top level keys (to nodes): 1-based, bottom level keys (from nodes): 1-based, values (list of participants): 0-based  
fromComponentLinks: top level keys (from components): 1-based, bottom level keys (to components): 1-based, values (set of participants): 0-based  
toComponentLinks: top level keys (to components): 1-based, bottom level keys (from components): 1-based, values (set of participants): 0-based  

## Old version of the exporting

### Old vertions of functions of new routine

In [ ]:
# To be deleted
def getBackwardPositiveLinks(toComponentLinks,toComponentLinksAcc,accStarts,accEnds):
    '''
    Function gets toComponentLinks dict structure (output from `baseLayerZoom` (partially) and 
    `nodeToComponentLinks` (finalised) on base layer and by `nextLayerZoom` on every other zoom layer).
    
    Parameters
    ==========
    
    toComponentLinks: dict{int:dict{str:dict{int:dict{str:list[int]}}}}. This input is an hierarchical structure of dicts
                    which holds all links between components on the current zoom layer. This structure is internal and 
                    produced by `baseLayerZoom` (partially) and `nodeToComponentLinks` (finalised) on base layer and 
                    by `nextLayerZoom` on every other zoom layer.
    
    Return
    ======
    
    compAccessionBackLinks: list[tuple(int)]. A list of tuples of component ids (1-based) (outgoing and incoming) and accessions 
                            where reversed (backward) links are coming.
    
    '''
    allBlockStarts = []
#     backLinksThroughStart = []
#     forwardLinksThroughEnd = []
    
    for toComp,toCompDict in toComponentLinks.items():

        # We are interested only in backward links coming to forward components (per accession, not overall)
        for fromComp,fromCompDict in toCompDict.get('+',{}).items():
            if fromComp>=toComp:
                for accList in fromCompDict.values():
                    for acc in accList:
                        leftEnd = min(accStarts[acc],accEnds[acc])
                        rightEnd = max(accStarts[acc],accEnds[acc])
                        if fromComp>rightEnd and toComp<=rightEnd and not (fromComp>=leftEnd and toComp<leftEnd):
                            if np.all(np.array(list(toComponentLinksAcc[acc][toComp]))>=toComp):
                                continue
                        
                        
#                         if fromComp-1 in toComponentLinksAcc[acc].get(toComp-1,set()):
#                             continue
                        
                        if fromComp>=leftEnd and toComp<leftEnd:
                            allBlockStarts.append((fromComp,toComp,acc))
                        else:
                            allBlockStarts.append((fromComp,toComp,acc))
            else:
                # Identify forward links through end
                for accList in fromCompDict.values():
                    for acc in accList:
                        rightEnd = max(accStarts[acc],accEnds[acc])
                        if fromComp<rightEnd and toComp>rightEnd:
                            allBlockStarts.append((fromComp,toComp,acc))
    
    
    return allBlockStarts        

In [ ]:
# To be deleted
def getForwardNegativeLinks(toComponentLinks,toComponentLinksAcc,accStarts,accEnds,invertedStarts):
    '''
    Function gets toCOmponentLinks dict structure (output from `baseLayerZoom` (partially) and 
    `nodeToComponentLinks` (finalised) on base layer and by `nextLayerZoom` on every other zoom layer).
    
    Parameters
    ==========
    
    toComponentLinks: dict{int:dict{str:dict{int:dict{str:list[int]}}}}. This input is an hierarchical structure of dicts
                    which holds all links between components on the current zoom layer. This structure is internal and 
                    produced by `baseLayerZoom` (partially) and `nodeToComponentLinks` (finalised) on base layer and 
                    by `nextLayerZoom` on every other zoom layer.
    
    Return
    ======
    
    compAccessionBackLinks: list[tuple(int)]. A list of tuples of component ids (1-based) (outgoing and incoming) and accessions 
                            where reversed (backward) links are coming.
    
    '''
    allBlockStarts = []
#     backwardNegLinks = []
#     backLinksThroughStart = []
#     forwardLinksThroughEnd = []
    
    for toComp,toCompDict in toComponentLinks.items():

        # We are interested only in backward links coming to forward components (per accession, not overall)
        for fromComp,fromCompDict in toCompDict.get('-',{}).items():
            if fromComp<=toComp:
                for accList in fromCompDict.values():
                    for acc in accList:
                        leftEnd = min(accStarts[acc],accEnds[acc])
                        rightEnd = max(accStarts[acc],accEnds[acc])
                        
                        if fromComp<leftEnd and toComp>=leftEnd and not (fromComp<rightEnd and toComp>=rightEnd):
                            if np.all(np.array(list(toComponentLinksAcc[acc][toComp]))<=toComp):
                                continue
                            
                        if fromComp<=rightEnd and toComp>rightEnd:
                            allBlockStarts.append((fromComp,toComp,acc))
                        else:
                            allBlockStarts.append((fromComp,toComp,acc))
            else:
                
                # Identify backward links through start
                outsideLinkAdded = False
                for fromStrand,accList in fromCompDict.items():
                    for acc in accList:
                        leftEnd = min(accStarts[acc],accEnds[acc])
                        adjustment = int(fromStrand=='-')
                        if fromComp-adjustment>=leftEnd and toComp<leftEnd:
                            outsideLinkAdded = True
                            allBlockStarts.append((fromComp,toComp,acc))
                        elif fromStrand=='+':
                            allBlockStarts.append((fromComp,toComp,acc))
                        else:
                            if all([fComp>=toComp for fComp,fCompDict in toCompDict.get('-',{}).items() if acc in fCompDict.get('+',[])]) \
                            and not (fromComp+1 in toComponentLinksAcc[acc].get(toComp+1,set())): # Not sure why this condition is here...
                                allBlockStarts.append((fromComp,toComp,acc)) #backwardNegativeLinks were appended here originally
                
#                 if not outsideLinkAdded:
#                     for acc in fromCompDict.get('+',[]):
                        
    
    # Adding inverted starts as the start of the inverted blocks as they cannot be picked up by any links.
    for accInvStart in invertedStarts:
        allBlockStarts.append((None,accStarts[accInvStart],accInvStart))
    
    return allBlockStarts#,backwardNegLinks   

In [ ]:
# To be deleted
def checkOvercoming(block,accessionFromLinks,accessionToLinks):
    '''
    Probably, it is not a correct way of doing this, although, we can still leave this, 
    but also follow the following rule (this should be implemented in `processNegativeLink`):
    
    If inverted block start have at least one backward link incoming to it, 
    we need to look for arrow straight "before" the "end" (left end) of this block,
    which goes forward beyond the start (right start) of this block and add this arrow to associated links.
    '''
    minComp = min(block)
    maxComp = max(block)
    
    associatedLinks = []
    
    if maxComp+1 in accessionFromLinks.get(minComp-1, set()):
        associatedLinks.append((minComp-1,maxComp+1))
    if maxComp+1 in accessionToLinks.get(minComp-1, set()):
        associatedLinks.append((maxComp+1,minComp-1))
                                         
    return associatedLinks

In [ ]:
# To be deleted
def addToNewBlockStarts(extendedLinks,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts):
    newPositiveBlockStarts.update([(fromComp,toComp,accessionID) for fromComp,toComp in extendedLinks if compDir[toComp]=='+'])
    newNegativeBlockStarts.update([(fromComp,toComp,accessionID) for fromComp,toComp in extendedLinks if compDir[toComp]=='-'])
    return newPositiveBlockStarts,newNegativeBlockStarts

In [ ]:
# To be deleted
def processBackwardLink(fromComp,toComp,accessionID,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,accCompDir,components):
    accessionLinks = fromComponentLinksAcc[accessionID]
    accessionToLinks = toComponentLinksAcc[accessionID]
    
    compDir = accCompDir[accessionID]
    
    leftEnd = min(accStarts[accessionID],accEnds[accessionID])
    rightEnd = max(accStarts[accessionID],accEnds[accessionID])
    isOverEnd = (fromComp<rightEnd and toComp>rightEnd)
    isOverStart = (fromComp>=leftEnd and toComp<leftEnd)
#     if fromComp==4 and toComp==2:
#         pdb.set_trace()

    # This list will include tuples in the following structure:
    # (accessionID, blockLength, associatedLinks, includedComponents)
    blockList = []
    block = [toComp]
    blockLength = compLength(toComp,components)
    associatedLinks = [(fromComp,toComp)]
    newPositiveBlockStarts = set()
    newNegativeBlockStarts = set()
    if fromComp>=toComp:
        # If it is forward link, then we need to check whether there are any backward link
        # coming to the same component and associate them with this block as well
        for fc in accessionToLinks[toComp]:
            if fc<toComp:
                isBackward = True
                if (fc,toComp) not in associatedLinks:
                    associatedLinks.append((fc,toComp))
    curComp = toComp
    
    isBackward = False
    doBreak = False
    
#     if accessionID==3 and fromComp==3 and toComp==2:
#         pdb.set_trace()
    
    while True:
        
        # Include any forward links coming inside the block
        extraLinks = []
        prevCompList = np.array(list(accessionToLinks.get(curComp,set())))
        for prevComp in prevCompList:
            if (prevComp<curComp and prevComp<rightEnd or prevComp>rightEnd) and ((prevComp,curComp) not in associatedLinks):
                if not ((prevComp==curComp-1 and compDir[prevComp]=='+' and compDir[curComp]=='+') or \
                         (prevComp==curComp+1 and compDir[prevComp]=='-' and compDir[curComp]=='-')):
                    associatedLinks.append((prevComp,curComp))
        
        nextCompList = np.array(list(accessionLinks.get(curComp,set())))
        distToNextComp = nextCompList-curComp
        if len(nextCompList)==0:
            break
        if np.all(distToNextComp<=0):
            isBackward = True
            # add all backwards link jumping over the end to associated links
            # This link is just a return from the block that started and ended beyond 
            # the right end of the genome and everything there is in reverse.
            # That is why these links are listed and associated with this block 
            # beyond the right end.
            associatedLinks.extend([(curComp,nextComp) for nextComp in nextCompList if not \
                                   (nextComp==curComp-1 and compDir[nextComp]=='-' and compDir[curComp]=='-')])
            break
        elif (np.all(nextCompList>fromComp) and (toComp<=fromComp)) or (np.all(nextCompList<=toComp) and (toComp>fromComp)):
            extendedLinks = [(curComp,nextComp) for nextComp in nextCompList if not \
                                    ((nextComp==curComp+1 and compDir[nextComp]=='+' and compDir[curComp]=='+') or \
                                     (nextComp==curComp-1 and compDir[nextComp]=='-' and compDir[curComp]=='-'))]
            associatedLinks.extend(extendedLinks)
            newPositiveBlockStarts,newNegativeBlockStarts = addToNewBlockStarts(extendedLinks,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
            break
        elif np.any(distToNextComp<=0) or (np.any(nextCompList>fromComp) and toComp<=rightEnd) or (np.any(nextCompList<toComp) and (toComp>rightEnd)):
            if np.any(distToNextComp<=0):
                isBackward = True
                
            if (np.any(np.logical_and(nextCompList>toComp,nextCompList<=fromComp)) and toComp<=rightEnd) or \
                (np.any(nextCompList>toComp) and toComp>rightEnd):
                linksToAssociate = []
                for nextComp in nextCompList:
                    if not ((curComp==nextComp-1 and compDir[curComp]=='+' and compDir[nextComp]=='+') or \
                         (curComp==nextComp+1 and compDir[curComp]=='-' and compDir[nextComp]=='-')) and \
                        ((curComp,nextComp) not in associatedLinks):
                        if (((nextComp>=fromComp or nextComp<=toComp) and toComp<=rightEnd) or \
                            (toComp>rightEnd)):
                            linksToAssociate.append((curComp,nextComp))
                        else:
                            extraLinks.append((curComp,nextComp))
                        
                # These auxiliary blocks should be associated only with the links that create this break 
                # (recorded in `linksToAssociate`), but not with the links which creates the main block 
                # (which recorded in `associatedLinks`).
                if len(linksToAssociate)>0:
                    newPositiveBlockStarts,newNegativeBlockStarts = addToNewBlockStarts(linksToAssociate,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
                    blockList.append((accessionID,blockLength,tuple(linksToAssociate),tuple(deepcopy(block))))
            else:
                extendedLinks = [(curComp,nextComp) for nextComp in nextCompList if not \
                                    ((nextComp==curComp+1 and compDir[nextComp]=='+' and compDir[curComp]=='+') or \
                                     (nextComp==curComp-1 and compDir[nextComp]=='-' and compDir[curComp]=='-'))]
                associatedLinks.extend(extendedLinks)
                newPositiveBlockStarts,newNegativeBlockStarts = \
                    addToNewBlockStarts([link for link in extendedLinks if not (link[0]>=link[1] and link[1]<=rightEnd or link[0]<link[1] and link[1]>rightEnd)],
                                        compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
                break
        else:
            for nextCompInd in np.where(distToNextComp>1)[0]:
                nextComp = nextCompList[nextCompInd]
                if nextComp>rightEnd and curComp<rightEnd:
                    isBackward = True
                    doBreak = True
                    break
                    
                if nextComp<=fromComp:
                    smallBlock = []
                    smallAssociatedLinks = [(curComp,nextComp)]
                    smallBlockSize = 0
                    for compInd in range(curComp+1,nextComp):
                        if accessionID in components[compInd-1]['occupants']:
                            smallBlock.append(compInd)
                            smallBlockSize += compLength(compInd,components)
                    blockList.append((accessionID,smallBlockSize,tuple(smallAssociatedLinks),tuple(smallBlock)))
#                 else:
#                     associatedLinks.append((curComp,nextComp))
            if doBreak:
                break
            
        nextComp = nextCompList[np.argmin(np.where(distToNextComp>0,distToNextComp,np.max(distToNextComp)+np.abs(distToNextComp)+1))]
        if ((nextComp-curComp>1 and compDir[nextComp]=='+' and compDir[curComp]=='+') or \
           (curComp-nextComp>1 and compDir[nextComp]=='-' and compDir[curComp]=='-')):
            associatedLinks.append((curComp,nextComp))
        if (curComp,nextComp) in extraLinks:
            extraLinks.remove((curComp,nextComp))
        newPositiveBlockStarts,newNegativeBlockStarts = \
            addToNewBlockStarts(extraLinks,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
        associatedLinks.extend(extraLinks)
        blockLength += compLength(nextComp,components)
        block.append(nextComp)
        curComp = nextComp
    
    blockList.append((accessionID,blockLength,tuple(associatedLinks+checkOvercoming(block,accessionLinks,accessionToLinks)),tuple(block)))
    
    if not isBackward and not isOverStart and not isOverEnd:

        stopSearch = False

        
        for comp in range(toComp-1,0,-1):
            compLinks = np.array(list(accessionLinks.get(comp,set())))
            if len(accessionToLinks.get(comp,set()))==0 and accessionID in components[comp-1].get('starts',[]):
                stopSearch = True

            if len(compLinks)==0:
                continue

            distToNextComp = compLinks - comp

            if np.all(distToNextComp==1):
                break

            for linkEnd in np.where(distToNextComp>1)[0]:
                stopSearch = True
                if not (comp<accEnds[accessionID] and (compLinks[linkEnd]>accEnds[accessionID])):
                    blockList.append(processForwardLink(comp,compLinks[linkEnd],accessionLinks,accessionToLinks,accessionID,components))

            if stopSearch:
                break
    
    return blockList,newPositiveBlockStarts,newNegativeBlockStarts

In [ ]:
# To be deleted
def processForwardLink(fromComp,toComp,accessionLinks,accessionToLinks,accessionID,components):
    
    block = [toComp]
    associatedLinks = [(fromComp,toComp)]
    blockLength = compLength(toComp, components)
    curComp = toComp
    
    stopSearch = False
    
    while True:
        nextCompList = np.array(list(accessionLinks.get(curComp,set())))
        
        if len(nextCompList)==0 and accessionID in components[curComp-1].get('ends',[]):
            stopSearch = True
        
        if np.any(nextCompList<toComp):
            break
        
        distToNextComp = nextCompList-curComp
        
        if np.all(distToNextComp<=0):
            break
        
        nextCompList = nextCompList[distToNextComp>0]
        distToNextComp = distToNextComp[distToNextComp>0]
        
        nextComp = nextCompList[np.argmin(distToNextComp)]
        if (nextComp-curComp>1):
            associatedLinks.append((curComp,nextComp))
        blockLength += compLength(nextComp,components)
        block.append(nextComp)
        curComp = nextComp
        
        if stopSearch:
            break
    
    return (accessionID,blockLength,tuple(associatedLinks+checkOvercoming(block,accessionLinks,accessionToLinks)),tuple(block))

In [ ]:
# To be deleted
def processNegativeLink(fromComp,toComp,accessionID,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,accCompDir,components):
    accessionFromLinks = fromComponentLinksAcc[accessionID]
    accessionToLinks = toComponentLinksAcc[accessionID]
    
    compDir = accCompDir[accessionID]
    
    leftEnd = min(accStarts[accessionID],accEnds[accessionID])
    rightEnd = min(accStarts[accessionID],accEnds[accessionID])
    
    if fromComp is not None:
        isOverEnd = (fromComp<accEnds[accessionID] and toComp>accEnds[accessionID])
        isOverStart = (fromComp>=accStarts[accessionID] and toComp<accStarts[accessionID])
    else:
        isOverStart = (accStarts[accessionID]<accEnds[accessionID])
        isOverEnd = False
    
    isBackward = False
    
    # This list will include tuples in the following structure:
    # (accessionID, blockLength, associatedLinks, includedComponents)
    blockList = []
    block = [toComp]
    blockLength = compLength(toComp,components)
    
    newPositiveBlockStarts = set()
    newNegativeBlockStarts = set()
    
    if fromComp is not None:
        associatedLinks = [(fromComp,toComp)]
        if fromComp<=toComp:
            # If it is forward link, then we need to check whether there are any backward link
            # coming to the same component and associate them with this block as well
            for fc in accessionToLinks[toComp]:
                if fc>=toComp:
                    isBackward = True
                    if (fc,toComp) not in associatedLinks:
                        associatedLinks.append((fc,toComp))
        fComp = fromComp
    else:
        associatedLinks = []
        fComp = 0
        # Not sure whether it is correct to do like this or it should be taken into account
        # when the block ends and we hit another forward link. Possibly, it will be more universal solution.
#         if toComp+1 in accessionFromLinks.get(fromComp+1,set()):
#             associatedLinks.append((fromComp+1,toComp+1))
    curComp = toComp
    
    doBreak = False
#     pdb.set_trace()
    while True:
#         if curComp==2958 and accessionID==17:
#             pdb.set_trace()
        extraLinks = []
        
        nextCompList = np.array(list(accessionFromLinks.get(curComp,set())))
        distToNextComp = nextCompList-curComp
        if len(nextCompList)==0:
            break
        if np.all(distToNextComp>=0) or (np.any(np.logical_and(distToNextComp>0,nextCompList>toComp)) and (fComp>toComp)):
            
            # add all forward link jumping over the start to associated links
            associatedLinks.extend([(curComp,nextComp) for nextComp in nextCompList if not \
                                    ((nextComp==curComp+1 and compDir[nextComp]=='+' and compDir[curComp]=='+'))])
            break
        elif np.all(nextCompList<fComp) and (fComp<toComp):
            extendedLinks = [(curComp,nextComp) for nextComp in nextCompList if not \
                                    ((nextComp==curComp+1 and compDir[nextComp]=='+' and compDir[curComp]=='+') or \
                                     (nextComp==curComp-1 and compDir[nextComp]=='-' and compDir[curComp]=='-'))]
            associatedLinks.extend(extendedLinks)
            newPositiveBlockStarts,newNegativeBlockStarts = \
                    addToNewBlockStarts(extendedLinks,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
            break
        elif np.any(distToNextComp>=0) or np.any(nextCompList<fComp) and (fComp<toComp):
            
            if np.any(np.logical_and(nextCompList>=fComp,nextCompList<toComp)):
                linksToAssociate = []
                for nextComp in nextCompList:
                    if not ((curComp==nextComp-1 and compDir[curComp]=='+' and compDir[nextComp]=='+') or \
                         (curComp==nextComp+1 and compDir[curComp]=='-' and compDir[nextComp]=='-')) and \
                        ((curComp,nextComp) not in associatedLinks):
                            if (nextComp<=fComp or nextComp>toComp):
                                linksToAssociate.append((curComp,nextComp))
                            else:
                                extraLinks.append((curComp,nextComp))
                
                # These auxiliary blocks should be associated only with the links that create this break 
                # (recorded in `linksToAssociate`), but not with the links which creates the main block 
                # (which recorded in `associatedLinks`).
                if len(linksToAssociate)>0:
                    blockList.append((accessionID,blockLength,tuple(linksToAssociate),tuple(deepcopy(block))))
                    newPositiveBlockStarts,newNegativeBlockStarts = \
                        addToNewBlockStarts(linksToAssociate,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
#                 else:
#                     pdb.set_trace()
            else:
                extendedLinks = [(curComp,nextComp) for nextComp in nextCompList if not \
                                        ((nextComp==curComp+1 and compDir[nextComp]=='+' and compDir[curComp]=='+') or \
                                         (nextComp==curComp-1 and compDir[nextComp]=='-' and compDir[curComp]=='-'))]
                associatedLinks.extend(extendedLinks)    
                newPositiveBlockStarts,newNegativeBlockStarts = \
                    addToNewBlockStarts([link for link in extendedLinks if not (link[0]>=link[1] and link[1]<=leftEnd or link[0]<link[1] and link[1]>leftEnd)],
                                        compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
                break
        elif np.any(np.logical_and(distToNextComp>=0,nextCompList>toComp)) and (fComp>toComp):
            
            if np.any(np.logical_and(nextCompList<toComp)):
                linksToAssociate = []
                for nextComp in nextCompList:
                    if (nextComp>toComp) and \
                    not ((curComp==nextComp-1 and compDir[curComp]=='+' and compDir[nextComp]=='+') or \
                         (curComp==nextComp+1 and compDir[curComp]=='-' and compDir[nextComp]=='-')) and \
                    ((curComp,nextComp) not in associatedLinks):
                        linksToAssociate.append((curComp,nextComp))
                
                if len(linksToAsociate)>0:
                    newPositiveBlockStarts,newNegativeBlockStarts = \
                        addToNewBlockStarts(linksToAssociate,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
                blockList.append((accessionID,blockLength,tuple(associatedLinks+linksToAssociate),tuple(deepcopy(block))))
            else:
                break
        else:
            for nextCompInd in np.where(distToNextComp<-1)[0]:
                nextComp = nextCompList[nextCompInd]
                if nextComp<leftEnd and curComp>leftEnd:
                    doBreak = True
                    break
                    
                if nextComp>=toComp:
                    smallBlock = []
                    smallAssociatedLinks = [(curComp,nextComp)]
                    smallBlockSize = 0
                    for compInd in range(curComp+1,nextComp):
                        if accessionID in components[compInd-1]['occupants']:
                            smallBlock.append(compInd)
                            smallBlockSize += compLength(compInd,components)
                    blockList.append((accessionID,smallBlockSize,tuple(smallAssociatedLinks),tuple(smallBlock)))
#                 else:
#                     associatedLinks.append((curComp,nextComp))
            if doBreak:
                break
            
        nextComp = nextCompList[np.argmin(np.where(distToNextComp<0,distToNextComp,np.max(distToNextComp)+np.abs(distToNextComp)+1))]
        if not ((nextComp-curComp==1 and compDir[nextComp]=='+' and compDir[curComp]=='+') or \
           (curComp-nextComp==1 and compDir[nextComp]=='-' and compDir[curComp]=='-')):
            associatedLinks.append((curComp,nextComp))
        if (curComp,nextComp) in extraLinks:
            extraLinks.remove((curComp,nextComp))
        newPositiveBlockStarts,newNegativeBlockStarts = \
            addToNewBlockStarts(extraLinks,compDir,accessionID,newPositiveBlockStarts,newNegativeBlockStarts)
        associatedLinks.extend(extraLinks)
        blockLength += compLength(nextComp,components)
        block.append(nextComp)
        curComp = nextComp
    
    leftmostBlockComp = min(block)
    
    
    # If there was at least one link to the start of the block from the right (backward link),
    # we should expect that there are links from the component just before 
    # (!!! is it just before or we should look further or hope that it will be picked up by other blocks?) 
    # the block going over it and all of them should also be associated with this block.
    if (fComp>toComp or isBackward) and leftmostBlockComp>1:
        for jumpToComp in accessionFromLinks.get(leftmostBlockComp-1,set()):
            if jumpToComp>toComp:
                associatedLinks.append((leftmostBlockComp-1,jumpToComp))
    
    blockList.append((accessionID,blockLength,tuple(associatedLinks),tuple(block)))
    
    # At the moment checkOvercoming was substituted by the routine just above, but there is a possibility that it may be needed.
    # Double check!
    #+checkOvercoming(block,accessionFromLinks,accessionToLinks)
    
    return blockList,newPositiveBlockStarts,newNegativeBlockStarts

In [ ]:
# To be deleted
def identifyCollapsibleBlocks(positiveBlockStarts,negativeBlockStarts,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,accCompDir,components):
    collapsibleBlocks = []
    collectivePositiveBlockStarts = set(positiveBlockStarts)
    collectiveNegativeBlockStarts = set(negativeBlockStarts)
    _positiveBlockStarts = deepcopy(collectivePositiveBlockStarts)
    _negativeBlockStarts = deepcopy(collectiveNegativeBlockStarts)
    counter = 1
    while len(_positiveBlockStarts)>0 or len(_negativeBlockStarts)>0:
        print(f'Iteration {counter}, {len(_positiveBlockStarts)} positive starts, {len(_negativeBlockStarts)} negative starts')
        counter += 1
        for backLink in _positiveBlockStarts:
            res,newPosBlockStarts,newNegBlockStarts = processBackwardLink(*backLink,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,accCompDir,components)
    #         print(f'Back Link {backLink[0]} -> {backLink[1]} for accession {backLink[2]} ({graph.accessions[backLink[2]]})')
    #         print(res)
            collapsibleBlocks.extend(res)
        
        _positiveBlockStarts = newPosBlockStarts.difference(collectivePositiveBlockStarts)
        _negativeBlockStarts.update(newNegBlockStarts.difference(collectiveNegativeBlockStarts))
        
        collectivePositiveBlockStarts.update(_positiveBlockStarts)
        collectiveNegativeBlockStarts.update(_negativeBlockStarts)
        
        for forwardLink in _negativeBlockStarts:
            res,newPosBlockStarts,newNegBlockStarts = processNegativeLink(*forwardLink,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,accCompDir,components)
    #         print(f'Back Link {backLink[0]} -> {backLink[1]} for accession {backLink[2]} ({graph.accessions[backLink[2]]})')
    #         print(res)
            collapsibleBlocks.extend(res)
            
            
        _positiveBlockStarts.update(newPosBlockStarts.difference(collectivePositiveBlockStarts))
        _negativeBlockStarts = newNegBlockStarts.difference(collectiveNegativeBlockStarts)

        collectivePositiveBlockStarts.update(_positiveBlockStarts)
        collectiveNegativeBlockStarts.update(_negativeBlockStarts)
    return collapsibleBlocks

In [ ]:
def getBackwardPositiveLinks(toComponentLinks,toComponentLinksAcc,accStarts,accEnds):
    '''
    Function gets toCOmponentLinks dict structure (output from `baseLayerZoom` (partially) and 
    `nodeToComponentLinks` (finalised) on base layer and by `nextLayerZoom` on every other zoom layer).
    
    Parameters
    ==========
    
    toComponentLinks: dict{int:dict{str:dict{int:dict{str:list[int]}}}}. This input is an hierarchical structure of dicts
                    which holds all links between components on the current zoom layer. This structure is internal and 
                    produced by `baseLayerZoom` (partially) and `nodeToComponentLinks` (finalised) on base layer and 
                    by `nextLayerZoom` on every other zoom layer.
    
    Return
    ======
    
    compAccessionBackLinks: list[tuple(int)]. A list of tuples of component ids (1-based) (outgoing and incoming) and accessions 
                            where reversed (backward) links are coming.
    
    '''
    allBlockStarts = []
#     backLinksThroughStart = []
#     forwardLinksThroughEnd = []
    
    for toComp,toCompDict in toComponentLinks.items():

        # We are interested only in backward links coming to forward components (per accession, not overall)
        for fromComp,fromCompDict in toCompDict.get('+',{}).items():
            if fromComp>=toComp:
                for accList in fromCompDict.values():
                    for acc in accList:
                        if fromComp>accEnds[acc] and toComp<=accEnds[acc] and not (fromComp>=accStarts[acc] and toComp<accStarts[acc]):
                            if np.all(np.array(list(toComponentLinksAcc[acc][toComp]))>=toComp):
                                continue
                            
                        if fromComp>=accStarts[acc] and toComp<accStarts[acc]:
                            allBlockStarts.append((fromComp,toComp,acc))
                        else:
                            allBlockStarts.append((fromComp,toComp,acc))
            else:
                # Identify forward links through end
                for accList in fromCompDict.values():
                    for acc in accList:
                        if fromComp<accEnds[acc] and toComp>accEnds[acc]:
                            allBlockStarts.append((fromComp,toComp,acc))
    
    
    return allBlockStarts

In [ ]:
def processBackwardLink(fromComp,toComp,accessionID,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,components):
    accessionLinks = fromComponentLinksAcc[accessionID]
    accessionToLinks = toComponentLinksAcc[accessionID]
    
    isOverEnd = (fromComp<accEnds[accessionID] and toComp>accEnds[accessionID])
    isOverStart = (fromComp>=accStarts[accessionID] and toComp<accStarts[accessionID])
#     if fromComp==4 and toComp==2:
#         pdb.set_trace()

    # This list will include tuples in the following structure:
    # (accessionID, blockLength, associatedLinks, includedComponents)
    blockList = []
    block = [toComp]
    blockLength = compLength(toComp,components)
    associatedLinks = [(fromComp,toComp)]
    curComp = toComp
    
    isBackward = False
    
    while True:
        nextCompList = np.array(list(accessionLinks.get(curComp,set())))
        distToNextComp = nextCompList-curComp
        if len(nextCompList)==0:
            break
        if np.any(distToNextComp<=0):
            isBackward = True
            # add all backwards link jumping over the end to associated links
            for nextCompID in np.where(distToNextComp<=0)[0]:
                nextComp = nextCompList[nextCompID]
                if nextComp<=accEnds[accessionID] and curComp>accEnds[accessionID] and \
                   np.all(np.array(list(accessionToLinks[nextComp]))>=nextComp):
                    associatedLinks.append((curComp,nextComp))
            break
        elif np.any(nextCompList>fromComp):
            associatedLinks.extend([(curComp,nextComp) for nextComp in nextCompList if nextComp>curComp+1])
            break
        else:
            
            for nextCompInd in np.where(distToNextComp>1)[0]:
                nextComp = nextCompList[nextCompInd]
                if nextComp>accEnds[accessionID] and curComp<accEnds[accessionID]:
                    isBackward = True
                    break
                    
                if nextComp<=fromComp:
                    smallBlock = []
                    smallAssociatedLinks = [(curComp,nextComp)]
                    smallBlockSize = 0
                    for compInd in range(curComp+1,nextComp):
                        if accessionID in components[compInd-1]['occupants']:
                            smallBlock.append(compInd)
                            smallBlockSize += compLength(compInd,components)
                    blockList.append((accessionID,smallBlockSize,smallAssociatedLinks,smallBlock))
#                 else:
#                     associatedLinks.append((curComp,nextComp))
            if isBackward:
                break
            
            nextComp = nextCompList[np.argmin(distToNextComp)]
            if (nextComp-curComp>1):
                associatedLinks.append((curComp,nextComp))
            blockLength += compLength(nextComp,components)
            block.append(nextComp)
            curComp = nextComp
    
    blockList.append((accessionID,blockLength,associatedLinks,block))
    
    if not isBackward and not isOverStart and not isOverEnd:

        stopSearch = False

        
        for comp in range(toComp-1,0,-1):
            compLinks = np.array(list(accessionLinks.get(comp,set())))
            if len(accessionToLinks.get(comp,set()))==0 and accessionID in components[comp-1].get('starts',[]):
                stopSearch = True

            if len(compLinks)==0:
                continue

            distToNextComp = compLinks - comp

            if np.all(distToNextComp==1):
                break

            for linkEnd in np.where(distToNextComp>1)[0]:
                stopSearch = True
                if not (comp<accEnds[accessionID] and (compLinks[linkEnd]>accEnds[accessionID])):
                    blockList.append(processForwardLink(comp,compLinks[linkEnd],accessionLinks,accessionID,components))

            if stopSearch:
                break
    
    return blockList

In [ ]:
def processForwardLink(fromComp,toComp,accessionLinks,accessionID,components):
    
    block = [toComp]
    associatedLinks = [(fromComp,toComp)]
    blockLength = compLength(toComp, components)
    curComp = toComp
    
    stopSearch = False
    
    while True:
        nextCompList = np.array(list(accessionLinks.get(curComp,set())))
        
        if len(nextCompList)==0 and accessionID in components[curComp-1].get('ends',[]):
            stopSearch = True
        
        if np.any(nextCompList<toComp):
            break
        
        distToNextComp = nextCompList-curComp
        
        if np.all(distToNextComp<=0):
            break
        
        nextCompList = nextCompList[distToNextComp>0]
        distToNextComp = distToNextComp[distToNextComp>0]
        
        nextComp = nextCompList[np.argmin(distToNextComp)]
        if (nextComp-curComp>1):
            associatedLinks.append((curComp,nextComp))
        blockLength += compLength(nextComp,components)
        block.append(nextComp)
        curComp = nextComp
        
        if stopSearch:
            break
    
    return (accessionID,blockLength,associatedLinks,block)

In [ ]:
def identifyCollapsibleBlocks(allBlockStarts,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,components):
    collapsibleBlocks = []
    for backLink in allBlockStarts:
        res = processBackwardLink(*backLink,accStarts,accEnds,fromComponentLinksAcc,toComponentLinksAcc,components)
#         print(f'Back Link {backLink[0]} -> {backLink[1]} for accession {backLink[2]} ({graph.accessions[backLink[2]]})')
#         print(res)
        collapsibleBlocks.extend(res)
    return collapsibleBlocks

In [ ]:
#old
def updateLinks(newToOldInd,oldToNewInd,
                fromComponentLinks,toComponentLinks,
                collapsibleBlocks,accStarts,accEnds,components,
                newFromComponentLinks={},newToComponentLinks={}):
    
    for newComp,oldCompList in enumerate(newToOldInd):
        leftOldCompId = oldCompList[0] + 1
        rightOldCompId = oldCompList[-1] + 1
        newCompId = newComp + 1
        
        # The next two if blocks set conditions for processing or not processing left and right of the current new component
        # The side should not be processed if the end of the previous node is coming from the same component 
        # (on lower zoom level) as the begininng of the current component.
        if newComp>0:
            if oldCompList[0]==newToOldInd[newComp-1][-1]:
                doLeft = False
            else:
                doLeft = True
        else:
            doLeft = True
            
        if newComp<len(newToOldInd)-1:
            if oldCompList[-1]==newToOldInd[newComp+1][0]:
                doRight = False
            else:
                doRight = True
        else:
            doRight = True

        # Departure on the right (from positive block)
        if doRight:
            toCompDict = fromComponentLinks.get(rightOldCompId,{}).get('+',{})
            for toOldComp,toOldCompDirDict in toCompDict.items():
                # to positive strand
                if '+' in toOldCompDirDict:
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][0]
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[toNewCompId][0]==toOldComp-1:
                        addLink(newCompId,'+',toNewCompId+1,'+',toOldCompDirDict['+'],newFromComponentLinks,newToComponentLinks)
                # to negative strand
                if '-' in toOldCompDirDict:
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][-1]
                    # Check whether the to comp (old) is on the right of the new comp
                    if newToOldInd[toNewCompId][-1]==toOldComp-1:
                        addLink(newCompId,'+',toNewCompId+1,'-',toOldCompDirDict['-'],newFromComponentLinks,newToComponentLinks)

        # Departure on the left (from negative block)
        if doLeft:
            toCompDict = fromComponentLinks.get(leftOldCompId,{}).get('-',{})
            for toOldComp,toOldCompDirDict in toCompDict.items():
                # to positive strand
                if '+' in toOldCompDirDict:
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][0]
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[toNewCompId][0]==toOldComp-1:
                        addLink(newCompId,'-',toNewCompId+1,'+',toOldCompDirDict['+'],newFromComponentLinks,newToComponentLinks)
                # to negative strand
                if '-' in toOldCompDirDict:
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][-1]
                    # Check whether the to comp (old) is on the right of the new comp
                    if newToOldInd[toNewCompId][-1]==toOldComp-1:
                        addLink(newCompId,'-',toNewCompId+1,'-',toOldCompDirDict['-'],newFromComponentLinks,newToComponentLinks)


        # Arrival on the left (to positive block)
        if doLeft:
            fromCompDict = toComponentLinks.get(leftOldCompId,{}).get('+',{})
            for fromOldComp,fromOldCompDirDict in fromCompDict.items():
                # from positive strand
                if '+' in fromOldCompDirDict:
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    # Check whether the from comp (old) is on the right of the new comp
                    if newToOldInd[fromNewCompId][-1]==fromOldComp-1:
                        addLink(fromNewCompId+1,'+',newCompId,'+',fromOldCompDirDict['+'],newFromComponentLinks,newToComponentLinks)
                # to negative strand
                if '-' in fromOldCompDirDict:
                    # Getting to comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][0]
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[fromNewCompId][0]==fromOldComp-1:
                        addLink(fromNewCompId+1,'-',newCompId,'+',fromOldCompDirDict['-'],newFromComponentLinks,newToComponentLinks)

        if doRight:
            # Arrival on the right (to negative block)
            fromCompDict = toComponentLinks.get(rightOldCompId,{}).get('-',{})
            for fromOldComp,fromOldCompDirDict in fromCompDict.items():
                # from positive strand
                if '+' in fromOldCompDirDict:
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    # Check whether the from comp (old) is on the right of the new comp
                    if newToOldInd[fromNewCompId][0]==fromOldComp-1:
                        addLink(fromNewCompId+1,'+',newCompId,'-',fromOldCompDirDict['+'],newFromComponentLinks,newToComponentLinks)
                # from negative strand
                if '-' in fromOldCompDirDict:
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    # Check whether the from comp (old) is on the left of the new comp
                    if newToOldInd[fromNewCompId][-1]==fromOldComp-1:
                        addLink(fromNewCompId+1,'-',newCompId,'-',fromOldCompDirDict['-'],newFromComponentLinks,newToComponentLinks)
    
    for acc in accStarts.keys():
        accStarts[acc] = oldToNewInd[accStarts[acc]-1][0]+1
        accEnds[acc] = oldToNewInd[accEnds[acc]-1][0]+1
        
    for blockID,[accessionID,blockLength,associatedLinks,associatedComponents] in enumerate(collapsibleBlocks):
        collapsibleBlocks[blockID] = (accessionID,blockLength,
                                      [[oldToNewInd[link[0]-1][0]+1,oldToNewInd[link[1]-1][0]+1] for link in associatedLinks],
                                      [compID+1 for comp in associatedComponents for compID in oldToNewInd[comp-1]])
    
    return newFromComponentLinks,newToComponentLinks,accStarts,accEnds,collapsibleBlocks

### Old routine

In [ ]:
def exportToPantograph(graph=None, inputPath=None, GenomeGraphParams={}, outputPath=None, outputName=None, outputSuffix=None, isSeq=True,
                       redisConn=None,
                       listOfExports=['schematise', 'genomeToPangenome', 'annotationToGenome'],
                       zoomLevels=[1], maxLengthComponent=100, maxLengthChunk=20, inversionThreshold=0.5,
                       debug=False, returnDebugData=False):

    if graph is None:
        if inputPath is not None:
            print('Loading Genome')
            graph = GenomeGraph(gfaPath=inputPath, isGFASeq=isSeq, **GenomeGraphParams)
        else:
            raise ValueError("Either graph or inputpath to GFA file should be provided")

    if outputPath is None and outputName is None:
        if inputPath is not None:
            if outputSuffix is not None:
                outputPath, outputName = pathConvert(inputPath, suffix = outputSuffix)
            else:
                outputPath, outputName = pathConvert(inputPath)

        else:
            raise ValueError("If inputPath is not given, then outputPath and outputName should be provided.")
    else:
        if outputSuffix is not None:
            outputName = outputName + outputSuffix
    print(f'Recording Pantograph data to {outputPath}{os.path.sep}{outputName}')
    
    if redisConn is not None:
        redisConn.flushdb()
    
    numNodes = len(graph.nodes)
    numNodesDigits = np.int(np.ceil(np.log10(numNodes)))
    fromLinks = {}
    toLinks = {}

    startTime = time.time()

    rootStruct = deepcopy(rootStructTemplate)
    rootStruct["pathNames"] = graph.accessions

    if returnDebugData:
        zoomComponentLengths = {}
        zoomNodeToComponent = {}

        zoomComponentToNodes = {}
        zoomComponents = {}  # temporary structure for testing, later each zoomlevel should be just saved as soon as it is processed.
        zoomCompNucleotides = {}

    nodeLengths = calcNodeLengths(graph)

    pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(graph,nodeLengths)

    for zoomLevel in zoomLevels:

        zoomTime = time.time()

        os.makedirs(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}',exist_ok=True)

        print('\n===========================')
        print(f'Zoom level {zoomLevel}')
        print('===========================')
        zoom_level_struct = {}
        zoom_level_struct["files"] = []

        if returnDebugData:
            nodeToComponent = zoomNodeToComponent.setdefault(zoomLevel,[])
            componentToNode = zoomComponentToNodes.setdefault(zoomLevel,[])
            componentLengths = zoomComponentLengths.setdefault(zoomLevel,[])
            components = zoomComponents.setdefault(zoomLevel,[])
            componentNucleotides = zoomCompNucleotides.setdefault(zoomLevel,[])
        else:
            nodeToComponent = []
            componentToNode = []
            componentLengths = []
            components = []
            componentNucleotides = []

        component = deepcopy(componentTemplate)

        occupants = set()
        nucleotides = ''
        matrix = {}

        
        nodeLinks = []

        nBins = 0
        previousInv = {}
        nCols = 0
        binLength = 0
        occupancy = {}
        inversion = {}
        pos = {}
        nodesInComp = set()
        annotationNames = {}
        breakComponentWhenBinEnds = False
#         breakCompBeforeBin = False
        binOpen = False
        breakComponent = False
        forceBreak = False

        for nodeIdx in range(1,numNodes+1):
            if debug:
                print(f'Processing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}')
            else:
                print(f'\nProcessing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}',end='')

            # Find positions of the node in each path
            nodePathsIdx, nodeSeqInPath = np.where(pathNodeArray == nodeIdx)
            # Find node length
            nodeLen = nodeLengths[nodeIdx-1]

            # Find unique path IDs (numerical) and cound of given node in each path.
            uniqueNodePathsIDx,pathNodeCount = np.unique(nodePathsIdx,return_counts=True)

            ####################################
            # Block to check whether the component
            # should be broken before this node.
            ####################################
            # TO CHECK: Should it happen only if binOpen is False???

            # Boolean indicator of far incoming link
            # TO CHECK: Is it only incoming as incoming to the left
            # or incoming to the left and outgoing from the left?
            incomingFarLink = False

            # Boolean if at least one path starts from this node
            pathStart = False
            # Boolean if at least one path ends on this node.
            pathEnd = False
            # TO CHECK: Does it take into account inverted nodes in
            #           some accessions?

            # Indicator whether inversion is changed from one side of
            # `inversionThreshold` to the other.
            isChangeInversion = False

            # Check whether component should be broken before current node.
            for j, pathID in enumerate(uniqueNodePathsIDx):
                
                # calculating occupancy for current node
                # Do we need it?
                pOcc = occupancy.setdefault(pathID, 0)
                pOcc += pathNodeCount[j]*nodeLen
                occupancy[pathID] = pOcc  # Occupancy

                # Calculate inversion of current node
                localInv = 0
                for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
                    localInv += int(pathDirArray[pathID,nodeNumInPath])

                pInv = inversion.get(pathID,0)
                # Do we need to multiply it by node length?
                # Most probably it is needed for larger components
                # Bigger question is why we add it to the pInv, not just assign?
                pInv += localInv*nodeLen/pathNodeCount[j]
                inversion[pathID] = pInv
                
                # Check whether inversion is switched from one side of `inversionThreshold` to the other side.
                # Should not pInv be divided by node length to get actual inversion fraction? 
                # Does it ever work and whether it is repeated after previous node?
                if (pInv/nodeLen-inversionThreshold)*(previousInv.setdefault(pathID, pInv)/nodeLen-inversionThreshold)<0:
                    isChangeInversion = True
                # Record current inversion for current pathID as previous (already been analysed).
#                 previousInv[pathID] = pInv
                
                # Get indexes of positions of the node in the path
                nodePositions = np.where(nodePathsIdx==pathID)[0]
                # Get node inversion
                # Is it necessary to divide by node length???
                nodeInversionInPath = inversion[pathID]/nodeLengths[nodeIdx-1]
                #np.sum([pathDirArray[pathID, nodeSeqInPath[nodePos]] for nodePos in nodePositions])/len(nodePositions)
                # loop over node position sin path
                for nodePos in nodePositions:
                    # Get actual position in path of the node
                    nodePositionInPath = nodeSeqInPath[nodePos]
                    if debug:
                        print(f'Node {nodeIdx}, Path {pathID} inversion {nodeInversionInPath}')
                    if nodeInversionInPath<=inversionThreshold:
                        # Not inverted node
                        # For the node find the links goint to the left of the node
                        if nodePositionInPath>0 and nodePositionInPath<pathLengths[pathID]-1:
                            # node is not the first or last
                            if debug:
                                print(f"IncomingLinks: {pathNodeArray[pathID,nodePositionInPath-1]} -> {pathNodeArray[pathID,nodePositionInPath]}")
                                print(f'Existing known links: {fromLinks.get(pathNodeArray[pathID,nodePositionInPath-1],{})}')
    
                            # Get previous and current nodes (IDs, 1 based) in the path
                            previousNode = pathNodeArray[pathID,nodePositionInPath-1] # previous node
                            nextNode = pathNodeArray[pathID,nodePositionInPath] # current node
                            if previousNode+1!=nextNode:
                                # Link in the path does not go between consecutive nodes (from previous to current)
                                # This means that there is a links coming to the left of the node
                                if (previousNode<nextNode and nextNode in fromLinks.get(previousNode,{})) or (previousNode>=nextNode):
                                    # If we are not picking up passing through empty blocks as far link
                                    # !!! Not sure why this condition is needed and what it excludes.
                                    # Why tandems are excluded from here? 
                                    # They should be included as they also cause an arrow to the start of the node.
                                    incomingFarLink = True
                                    break
                        elif nodePositionInPath==0:
                            pathStart = True
                    else:
                        #Inverted node
                        # For the node find the links goint from the left of the node
                        if nodePositionInPath>0 and nodePositionInPath<pathLengths[pathID]-1:
                            # node is not the first or last
                            if debug:
                                print(f"IncomingLinks: {pathNodeArray[pathID,nodePositionInPath]} -> {pathNodeArray[pathID,nodePositionInPath+1]}")
                                print(f'Existing known links: {toLinks.get(pathNodeArray[pathID,nodePositionInPath+1],{})}')
                            
                            # Get previous and current nodes (IDs, 1 based) in the path
                            previousNode = pathNodeArray[pathID,nodePositionInPath]
                            nextNode = pathNodeArray[pathID,nodePositionInPath+1]
                            
                            if previousNode-1!=nextNode:
                                # Link does not go between consecutive blocks (from this to previous one)
                                if (previousNode>nextNode and previousNode in toLinks.get(nextNode,{})) or (previousNode<=nextNode):
                                    # If we are not picking up passing through empty blocks as far link
                                    # !!! Not sure why this condition is needed and what it excludes.
                                    # Why tandems are excluded from here? 
                                    # They should be included as they also cause an arrow to the start of the node.
                                    incomingFarLink = True
                                    break
                        elif nodePositionInPath==pathLengths[pathID]-1:
                            # if last node, mark it as path end.
                            pathEnd = True
#                 if incomingFarLink or pathStart or pathEnd:
#                     break
            
            #Assign current node inversion to previousInv
            previousInv = inversion
        
            if nodeIdx>1 and (pathStart or pathEnd or incomingFarLink or isChangeInversion) and len(matrix)>0:
                if debug:
                    if incomingFarLink:
                        print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to incoming far link.')
                    else:
                        print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to start or end of a path.')
                component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                    finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,
                                      zoomLevel=zoomLevel,accessions=graph.accessions,redisConn=redisConn)
                componentToNode.append(list(nodesInComp))
                nodesInComp = set([nodeIdx])
                breakComponent = False
                forceBreak = False
                isChangeInversion = False

            occupants |= set(uniqueNodePathsIDx)

            nodeToComponent.append([len(components)])
            nodesInComp.add(nodeIdx)

            if zoomLevel<nodeLen:
                for posInd,i in enumerate(range(0,nodeLen,zoomLevel)):
                    
                    if isSeq:
                        nucleotides += graph.nodesData[nodeIdx-1][i:i+zoomLevel]
                    else:
                        nucleotides += graph.nodes[nodeIdx-1][i:i+zoomLevel]

                    if (i+zoomLevel>nodeLen):
                        binLength = nodeLen-i
                    else:
                        binLength = zoomLevel

                    for j,pathID in enumerate(uniqueNodePathsIDx):
                        annotationNames.setdefault(graph.accessions[pathID],{}).\
                            update({k:None for k,v in graph.nodesAnnotation[nodeIdx-1][graph.accessions[pathID]].items() \
                                    for interval in v \
                                    if (interval[1]-i)*(i+zoomLevel-1-interval[0])>=0})
                        posPath = pos.setdefault(pathID,[])
                        for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
                            offset = np.min([i+zoomLevel,nodeLen])
                            nodeStart = pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1
                            posPath.append([nodeStart+i,nodeStart+offset-1])
                    
                    matrix,pos,binLength,_,_,nBins,annotationNames,nCols,binOpen = \
                        finaliseBin(matrix,pos,binLength,
                                    nodeLengths[nodeIdx-1],
                                    occupancy,inversion,
                                    nBins,nCols,posInd,
                                    annotationNames,
                                    graph.accessions)

                    if nBins+1>maxLengthComponent and (i+zoomLevel<nodeLen) and len(matrix)>0:
                        # Because this is intra-node break, all connectors will be handled by rdepartures without separating into inverted or non-inverted. 
                        # All inter-node links will be handled through links later when chunks are being recorded.
                        
                        forwardPaths = []
                        invertedPaths = []
                        for pathID in range(len(graph.paths)):
                            if pathID in occupants:
                                if matrix[pathID][1][0][1]>inversionThreshold:
                                    invertedPaths.append(pathID)
                                else:
                                    forwardPaths.append(pathID)
                            else:
                                forwardPaths.append(pathID)
                        
                        if len(forwardPaths)>0:
                            
                            component["rdepartures"].append({
                                "upstream": component["first_bin"]+nBins-1,
                                "downstream": component["first_bin"]+nBins,
                                "participants": forwardPaths,
                                'otherSideRight': False
                            })
                        if len(invertedPaths)>0:
                            component["rarrivals"].append({
                                "upstream": component["first_bin"]+nBins,
                                "downstream": component["first_bin"]+nBins-1,
                                "participants": invertedPaths,
                                'otherSideRight': False
                            })

                        if debug:
                            print(f'Node {nodeIdx}: Component broken inside node {nodeIdx} due to max component length.')

                        component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                            finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,
                                              zoomLevel=zoomLevel,accessions=graph.accessions,redisConn=redisConn)

                        nodeToComponent[-1].append(len(components))
                        componentToNode.append(list(nodesInComp))
                        nodesInComp = set([nodeIdx])
                        breakComponent = False
                        forceBreak = False
                binLength = 0
                occupancy = {}
                inversion = {}
            else:
                if nBins==0 and not binOpen:
                    if debug:
                        print(f'Before processing node {nodeIdx} the nodeLinks was cleared')
                    nodeLinks = []
                binOpen = True
                binLength += nodeLen

                for j,pathID in enumerate(uniqueNodePathsIDx):
                    annotationNames.setdefault(graph.accessions[pathID],{}).\
                        update(graph.nodesAnnotation[nodeIdx-1][graph.accessions[pathID]])
                        
                    posPath = pos.setdefault(pathID,[])
                    for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]:
                        nodeStart = pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1
                        posPath.append([nodeStart,nodeStart+nodeLen-1])
            
                if nodeIdx<len(graph.nodes):
                    if binLength+nodeLengths[nodeIdx]>zoomLevel:
                        matrix,pos,binLength,occupancy,inversion,nBins,annotationNames,nCols,binOpen = \
                        finaliseBin(matrix,pos,binLength,
                                    binLength,
                                    occupancy,inversion,
                                    nBins,nCols,0,
                                    annotationNames,
                                    graph.accessions)
                else:
                    matrix,pos,binLength,occupancy,inversion,nBins,annotationNames,nCols,binOpen = \
                    finaliseBin(matrix,pos,binLength,
                                binLength,
                                occupancy,inversion,
                                nBins,nCols,0,
                                annotationNames,
                                graph.accessions)

            # marking path ends and breaking component if any exist
            pathEnds = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if pathLengths[pathIdx]==nodeInSeq+1]))
            if len(pathEnds)>0:
                if binOpen:
                    component.setdefault("ends",set()).update(pathEnds.tolist())
                else:
                    if nBins>0:

                        component.setdefault("ends",set()).update(pathEnds.tolist())

                    else:
                        components[-1].setdefault("ends",set()).update(pathEnds.tolist())

            pathEndsRelevant = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if pathLengths[pathIdx]==nodeInSeq+1 and (not pathDirArray[pathIdx,nodeInSeq])]))
            pathStartsRelevant = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if nodeInSeq==0 and pathDirArray[pathIdx,nodeInSeq]]))
            if len(pathEndsRelevant)>0 or len(pathStartsRelevant)>0:
                if binOpen:
                    if debug:
                        print(f'Node {nodeIdx}: Component will be broken due to start or end of a path at node {nodeIdx} when bin is filled.')
                    breakComponent = True
                    forceBreak = True
                elif nBins>0:
                    if debug:
                        print(f'Node {nodeIdx}: Component broken after node {nodeIdx} due to start or end of a path.')
                    component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                        finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,
                                          zoomLevel=zoomLevel,accessions=graph.accessions,redisConn=redisConn)
                    componentToNode.append(list(nodesInComp))

                    nodesInComp = set()
                    breakComponent = False
                    forceBreak = False

            if nodeIdx<len(graph.nodes) and nBins>0:
                if nBins+nodeLengths[nodeIdx]/zoomLevel>maxLengthComponent:

                    if debug:
                        print(f'Node {nodeIdx}: Component broken after node {nodeIdx} because next component will not fit in this component.')
                    component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                        finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,
                                          zoomLevel=zoomLevel,accessions=graph.accessions,redisConn=redisConn)
                    componentToNode.append(list(nodesInComp))

                    nodesInComp = set()
                    breakComponent = False
                    forceBreak = False
            elif nBins>0:

                if debug:
                    print(f'Node {nodeIdx}: Last node in the last component.')
                component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                    finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,
                                      zoomLevel=zoomLevel,accessions=graph.accessions,redisConn=redisConn)
                componentToNode.append(list(nodesInComp))

                nodesInComp = set()
                breakComponent = False


            nodeFromLink = fromLinks.setdefault(nodeIdx,{})

            if zoomLevel==1:
                for j,pathID in enumerate(uniqueNodePathsIDx):
                    nodePositions = np.where(nodePathsIdx==pathID)[0]
                    nodeInversionInPath = np.sum([pathDirArray[pathID,nodeSeqInPath[nodePos]] for nodePos in nodePositions])/len(nodePositions)
                    for nodePos in nodePositions:
                        nodePositionInPath = nodeSeqInPath[nodePos]
                        if nodePositionInPath<(pathLengths[pathID]-1):
                            nextNode = pathNodeArray[pathID,nodePositionInPath+1]
                            nextNodePathsIdx,nextNodeSeqInPath = np.where(pathNodeArray==nextNode)
                            nextNodePositions = np.where(nextNodePathsIdx==pathID)[0]
                            nextNodeInversionInPath = np.sum([pathDirArray[pathID,nextNodeSeqInPath[nodePos]] for nodePos in nextNodePositions])/len(nextNodePositions)
                            if debug:
                                print(f'Node {nodeIdx}, next node {nextNode}, path {pathID}, number in path {nodePositionInPath}, length of path {pathLengths[pathID]}')
                            
                            if nodeInversionInPath>inversionThreshold and nextNodeInversionInPath>inversionThreshold:
                                reverseCond  = (nodeIdx<=nextNode)
                                step = -1
                            else:
                                reverseCond = (nodeIdx>=nextNode)
                                step = 1

                            if reverseCond:
                                if debug:
                                    print(f'Reverse link for path {pathID} from node {nodeIdx} to node {nextNode}')

                                nodeFromLink.setdefault(nextNode,[]).append(pathID)
                                toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                                if (nBins>0 or binOpen) and (nodeInversionInPath<=inversionThreshold):

                                    if debug:
                                        print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due to backward link to node {nextNode}.')
                                    nodeLinks.append(nextNode)
                                    breakComponent = True
                                    forceBreak = True
                            else:
                                if np.any([node in pathNodeArray[pathID,:] for node in range(nodeIdx+1*step,nextNode,step)]):
                                    if debug:
                                        print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

                                    nodeFromLink.setdefault(nextNode,[]).append(pathID)
                                    toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                                    if (nBins>0 or binOpen) and (nodeInversionInPath<=inversionThreshold): 

                                        if debug:
                                            print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due forward jumping link to node {nextNode}.')
                                        nodeLinks.append(nextNode)
                                        breakComponent = True
                                else:


                                    startNode = None
                                    endNode = None
                                    step = None

                                    if nodeInversionInPath>inversionThreshold and nextNodeInversionInPath>inversionThreshold:
                                        startNode = nodeIdx
                                        endNode = nextNode
                                        step = -1
                                        if debug:
                                            print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')

                                    elif nodeInversionInPath<=inversionThreshold and nextNodeInversionInPath<=inversionThreshold:
                                        startNode = nodeIdx
                                        endNode = nextNode
                                        step = 1
                                        if debug:
                                            print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')
                                    else:
                                        if debug:
                                            print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

                                        nodeFromLink.setdefault(nextNode,[]).append(pathID)
                                        toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                                        if nBins>0 or binOpen: 

                                            if debug:
                                                print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due forward jumping link to node {nextNode}.')
                                            nodeLinks.append(nextNode)
                                            breakComponent = True

                                    if startNode is not None and endNode is not None and step is not None:
                                        for intermediateNodeIdx in range(startNode,endNode,step):
                                            if debug:
                                                print(f'Adding link from node {intermediateNodeIdx} to node {intermediateNodeIdx+1*step} for path {pathID}')
                                            fromLinks.setdefault(intermediateNodeIdx,{}).setdefault(intermediateNodeIdx+1*step,[]).append(pathID)
                                            toLinks.setdefault(intermediateNodeIdx+1*step,{}).setdefault(intermediateNodeIdx,[]).append(pathID)
                        else:
                            if debug:
                                print("Last node in path")
            else:#if not breakComponent:
                if nBins>0 or binOpen:
                    breakLink,nnodes = checkLinksForBreak(nodeIdx,nodeFromLink,toLinks.setdefault(nodeIdx,{}),matrix,inversion,binOpen,inversionThreshold,True)
                    if breakLink==2 or (not binOpen and breakLink==1):
                        if debug:
                            print(f'Node {nodeIdx}: Component will be broken after node {nodeIdx} due a link from that node (res code {breakLink}) to nodes {nnodes}.')
                        breakComponent = True
                        nodeLinks.extend(nnodes)
            
            if breakComponent and not binOpen:
                if np.any([node not in nodesInComp for node in nodeLinks]) or len(nodeLinks)==0 or forceBreak:
                    
                    if debug:
                        print(f'Node {nodeIdx}: Component broken after node {nodeIdx} because of previously set flag.')
                        print(f'Nodes in component: {nodesInComp}')
                        print(f'Nodes linked from component: {nodeLinks}')
                    component,components,componentNucleotides,matrix,occupants,nBins,nCols,nucleotides = \
                        finaliseComponent(component,components,componentNucleotides,matrix,occupants,nBins,nCols,componentLengths,nucleotides,
                                          zoomLevel=zoomLevel,accessions=graph.accessions,redisConn=redisConn)
                    componentToNode.append(list(nodesInComp))
                    nodesInComp = set()
                    breakComponent = False
                    forceBreak = False
                    nodeLinks = []
                else:
                    if debug:
                        print(f'Node {nodeIdx}: Component is not broken as all links are internal, flag reset.')
                        print(f'Nodes in component: {nodesInComp}')
                        print(f'Nodes linked from component: {nodeLinks}')
                    breakComponent = False;

        print('\nNodes processed, recording component to files.')

        if zoomLevel==1:
            rootStruct["pangenome_length"] = np.sum(componentLengths)

        chunkList = rootStruct["zoom_levels"].setdefault(zoomLevel,{
            "last_bin": components[-1]["last_bin"],
            "files":[]
        })
        # Recording data to files for zoom level

    #     chunkTemplate = {
    #     "json_version":17,
    #     "bin_width":1,
    #     "first_bin":1,
    #     "includes_connectors": True,
    #     "components": []
    # }    
        chunk = deepcopy(chunkTemplate)
        chunkNum = 0
        prevTotalCols = 0
        curCompCols = 0
        nucleotides = ''
        nBins = 0
        numComps = len(components)
        numCompsDigits = np.int(np.ceil(np.log10(numComps)))
        
        for compNum in range(numComps):
            if debug:    
                print(f'Processing component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}')
            else:
                print(f'\nProcessing component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}',end='')
                
            component = components[compNum]

            nodeInComp = componentToNode[compNum]
            nodeInComp.sort()

            if len(nodeInComp)>1:
                component = addLinksToComp(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components)
            elif len(nodeInComp)==1:

                mainNode = nodeInComp[0]
    #             compForNode = nodeToComponent[mainNode-1]
                doLeft = False
                if compNum==0:
                    doLeft = True
                elif componentToNode[compNum-1][-1]!=mainNode:
                    doLeft = True

                doRight = False # Check this conditions as they may be causing issue!
                if compNum==(len(components)-1):
                    doRight = True
                elif componentToNode[compNum+1][0]!=mainNode:
                    doRight = True

                component = addLinksToComp(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft,doRight)

            else:
                raise ValueError(f"Component {compNum} does not have any associated nodes!")
            
            nBins += component["last_bin"]-component["first_bin"]+1
            if len(chunk['components'])>0:
                if checkLinks(chunk["components"][-1],component):
                    
                    newComp = joinComponents(chunk["components"].pop(),component,maxLengthComponent,inversionThreshold)
                    if isinstance(newComp,list):
                        chunk["components"].append(newComp[0])
                        component = newComp[1]
                    else:
                        component = newComp
            
            nucleotides += componentNucleotides[compNum]
            chunk["components"].append(component)
            

            # End of chunk
            if compNum<len(components)-1:

                if len(chunk['components'])>=maxLengthChunk:
                #nBins+components[compNum+1]["last_bin"]-components[compNum+1]["first_bin"]+1>maxLengthChunk:
                    rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                              rootStruct,
                                                                                              zoomLevel,
                                                                                              chunk,
                                                                                              nucleotides,
                                                                                              nBins,
                                                                                              chunkNum,
                                                                                              curCompCols,
                                                                                              prevTotalCols,
                                                                                              outputPath,
                                                                                              outputName)
            else:
                rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                          rootStruct,
                                                                                          zoomLevel,
                                                                                          chunk,
                                                                                          nucleotides,
                                                                                          nBins,
                                                                                          chunkNum,
                                                                                          curCompCols,
                                                                                          prevTotalCols,
                                                                                          outputPath,
                                                                                          outputName)
        if not debug:
            print()
        print(f'Recording finished. Zoom level time is {time.time() - zoomTime}. Time elapsed {time.time() - startTime}')

    with open(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}bin2file.json','w') as f:
        json.dump(rootStruct,f,cls=NpEncoder)
    
    if returnDebugData:
        return zoomComponentLengths,zoomNodeToComponent,zoomComponentToNodes,zoomComponents,zoomCompNucleotides,toLinks,fromLinks,graph

In [ ]:
def checkLinks(leftComp,rightComp):
    leftRdepCond = np.all([link['upstream']+1==link['downstream'] for link in leftComp['rdepartures']])
    leftRarrCond = np.all([link['upstream']-1==link['downstream'] for link in leftComp['rarrivals']])

    rightRdepCond = np.all([link['upstream']-1==link['downstream'] for link in rightComp['ldepartures']])
    rightRarrCond = np.all([link['upstream']+1==link['downstream'] for link in rightComp['larrivals']])
    
    return leftRdepCond and leftRarrCond and rightRarrCond and rightRdepCond

In [ ]:
def getMatrixPathElement(matrix,pathID):
    res = [el for el in matrix if el[0]==pathID]
    if len(res)==1:
        return res[0]
    elif len(res)>0:
        warnings.warn(f"More than one element for path {pathID} is found!")
    
    return None
        

In [ ]:
def joinComponents(leftComp,rightComp, maxLengthComponent, inversionThreshold=0.5):
    '''
    If the joining was successful, the function will return a joined component.
    
    If the joining was not successful and was aborted for one of the following reasons, it will return a list of original components. 
    The reasons for aborting the joining can be the following:
    - In one of the paths the invertion is lower than threshold in one component and higher in the other.
    - Left component contains at least one end
    - Right component contains at least one start
    
    The function will not check links for coming or going on the right of the left component and left of the right component. 
    It will just get left links from left component and right links from right component and assign them to the new component.
    '''
    
    if leftComp['last_bin']-leftComp['first_bin']+1 + rightComp['last_bin']-rightComp['first_bin']+1 > maxLengthComponent:
        return [leftComp,rightComp]
    
    if leftComp.get('ends',False):
        # End of a path
        return [leftComp,rightComp]
    
    newComp = {}
    newComp['first_bin'] = min(leftComp['first_bin'],rightComp['first_bin'])
    newComp['last_bin'] = max(leftComp['last_bin'],rightComp['last_bin'])
    newComp['firstCol'] = min(leftComp['firstCol'],rightComp['firstCol'])
    newComp['lastCol'] = max(leftComp['lastCol'],rightComp['lastCol'])
    
    leftCompNumBins = leftComp['last_bin']-leftComp['first_bin']+1
    
    newComp['occupants'] = list(set(leftComp['occupants']).union(rightComp['occupants']))
    
    for pathID in newComp['occupants']:
        leftPathElement = getMatrixPathElement(leftComp['matrix'],pathID)
        rightPathElement = getMatrixPathElement(rightComp['matrix'],pathID)
        if leftPathElement is None:
            if len([el for el in rightPathElement[2][1] if el[2][0][0]==1 or el[2][-1][0]==1])>0:
                # Start of a path
                return [leftComp,rightComp]
            rightPathElement[2][0] = [el+leftCompNumBins for el in rightPathElement[2][0]]
            newComp.setdefault("matrix",[]).append(rightPathElement)
            continue
        
        if rightPathElement is None:
            newComp.setdefault("matrix",[]).append(leftPathElement)
            continue
        
        if (leftPathElement[1]>inversionThreshold and rightPathElement[1]<=inversionThreshold) or \
           (leftPathElement[1]<=inversionThreshold and rightPathElement[1]>inversionThreshold):
            return [leftComp,rightComp]
        
        newPathElement = []
        newPathElement.append(pathID)
        newPathElement.append(leftPathElement[1])
        pathMatrix = []
        pathMatrix.append(leftPathElement[2][0] + [el+leftCompNumBins for el in rightPathElement[2][0]])
        pathMatrix.append(leftPathElement[2][1] + rightPathElement[2][1])
        newPathElement.append(pathMatrix)
        newComp.setdefault("matrix",[]).append(newPathElement)
        
    newComp['larrivals'] = leftComp['larrivals']
    newComp['ldepartures'] = leftComp['ldepartures']
    newComp['rarrivals'] = rightComp['rarrivals']
    newComp['rdepartures'] = rightComp['rdepartures']
    ends = list(set(leftComp.get('ends',[])).union(rightComp.get('ends',[])))
    if len(ends)>0:
        newComp['ends'] = ends
    
    return newComp

In [ ]:
def finaliseBin(matrix,pos,binLength,occInvAdj,occupancy,inversion,nBins,nCols,posInd,annotationNames,accessions):
    for pathID in pos.keys():
        matrixPath = matrix.setdefault(pathID,[[],[]])
        posPath = pos.get(pathID)
        posArray = np.array(posPath)
#         print(f'[finaliseBin] original pos {posArray}')
        posArray = posArray[np.argsort(posArray[:,0]),:]
        posIntersect = (posArray[1:,1]-(posArray[:-1,0]-1))*\
                        (posArray[:-1,1]-(posArray[1:,0]-1))
#         print(f'[finaliseBin] pos intersections {posIntersect}')
        newPos = [[posArray[0,0]]]
        candidates = [posArray[0,1]]
        for jointNum in range(len(posIntersect)):
            if posIntersect[jointNum]>=0:
                candidates.extend(posArray[jointNum+1,:].tolist())
            else:
                newPos[-1].append(np.max(candidates))
                newPos.append([posArray[jointNum+1,0]])
                candidates = [posArray[jointNum+1,1]]
            
        newPos[-1].append(np.max(candidates))# !!!!        
        
#         posMapping = {int(overallNBins+1):(int(posStart),int(posEnd)) for posStart,posEnd in newPos}
#         iset_add(redisConn, f'{zoomLevel}.{accessions[pathID]}.Pos',posMapping)
        
#         geneList = annotationNames[accessions[pathID]]
#         if len(geneList)>0:
#             geneMapping = {gene:int(overallNBins+1) for gene in geneList}
#             redisConn.hmset( f'{zoomLevel}.{accessions[pathID]}.Gene',geneMapping)
        
        inversionRate = inversion[pathID]/occInvAdj
#         pdb.set_trace()
        matrixPathRecord = [occupancy[pathID]/occInvAdj, # occupancy
                            inversionRate, # inversion
                            newPos, # genomic positions
                            list(annotationNames[accessions[pathID]].keys())] # annotation
        matrixPath[0].append(nBins)
#         matrixPathRecord[0] = occupancy[pathID]/occInvAdj # Occupancy
#         matrixPathRecord[1] = inversionRate # Inversion rate
#         matrixPathRecord[2] = newPos
        if inversionRate>0.5:
            matrixPath[1].insert(posInd,matrixPathRecord)
        else:
            matrixPath[1].append(matrixPathRecord)
        
    return matrix,{},0,{},{},nBins+1,{},nCols+binLength,False #matrix,pos,binLength,occupancy,inversion,nBins,annotationNames,nCols,overallNBins,binOpen

In [ ]:
def adjustMatrixPathArray(matrixPathArray,isInverted,nBins):
    if isInverted:
        _tempArray = [nBins-binNum-1 for binNum in matrixPathArray[0]]
        _tempArray.sort()
        matrixPathArray[0] = _tempArray
    return matrixPathArray

In [ ]:
def finaliseComponent(component,
                      components,componentNucleotides,
                      matrix,occupants,nBins,nCols,componentLengths,nucleotides,zoomLevel,accessions,inversionThreshold=0.5,redisConn=None):
    componentLengths.append(nBins)
    component["matrix"].extend([[pathID,
                                 int(matrixPathArray[1][0][1]>inversionThreshold),
                                 adjustMatrixPathArray(matrixPathArray,int(matrixPathArray[1][0][1]>inversionThreshold),nBins)] \
                                for pathID,matrixPathArray in matrix.items()])
    
    
    component["occupants"] = sorted(list(occupants))
    component['last_bin'] = component['first_bin'] + nBins - 1
    component['lastCol'] = component['firstCol'] + nCols - 1
    componentNucleotides.append(nucleotides)
    if redisConn is not None:
        for pathID,matrixPathArray in matrix.items():
            for binNum,binMatrix in zip(*matrixPathArray):
                if binMatrix[1]>inversionThreshold:
                    overallBin = component['last_bin']-binNum
                else:
                    overallBin = component['first_bin']+binNum
                
                posMapping = {int(overallBin):(int(posStart),int(posEnd)) for posStart,posEnd in binMatrix[2]}
                iset_add(redisConn, f'{zoomLevel}.{accessions[pathID]}.Pos',posMapping)

                geneList = binMatrix[3]
                for gene in geneList:
                    if not redisConn.hexists(f'{zoomLevel}.{accessions[pathID]}.Gene',gene):
                        redisConn.hset( f'{zoomLevel}.{accessions[pathID]}.Gene',gene,overallBin)
    
    firstBin = component['last_bin'] + 1
    firstCol = component['lastCol'] + 1
    components.append(component)
    component = deepcopy(componentTemplate)
    component['first_bin'] = firstBin
    component['firstCol'] = firstCol
    return component,components,componentNucleotides,{},set(),0,0,''

In [ ]:
def checkLinksForBreak(nodeIdx,nodeFromLinks,nodeToLinks,matrix,inversion,binOpen,inversionThreshold,isCurrent):
    res = 0
    nodes = set()
    if isCurrent:
        for node,pathList in nodeFromLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel<=inversionThreshold:
                            # Forward path
                            if nodeIdx+1!=node:
                                res = max(2,res)
                                nodes.add(node)
        for node,pathList in nodeToLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel>inversionThreshold:
                            # Inverted path
                            if nodeIdx+1!=node:
                                res = max(2,res)
                                nodes.add(node)
            
    else:
        # Do not use isCurrent=False !!!
        for node,pathList in nodeFromLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel>inversionThreshold:
                            # Inverted path
                            if nodeIdx-1!=node:
                                res = max(2,res)
                                nodes.add(node)
        for node,pathList in nodeToLinks.items():
            if nodeIdx==node:
                res = max(1,res)
                nodes.add(node)
            else:
                for pathID in pathList:
                    if binOpen:
                        invLevel = inversion.get(pathID)
                    else:
                        invLevel = matrix.get(pathID)
                        if invLevel is not None:
                            invLevel = invLevel[1][0][1]
                    if invLevel is not None:
                        if invLevel<=inversionThreshold:
                            # Inverted path
                            if nodeIdx+1!=node:
                                res = max(2,res)
                                nodes.add(node)
        
    return res,list(nodes)

In [ ]:
def finaliseChunk(rootStruct,zoomLevel,chunk,nucleotides,nBins,chunkNum,curCompCols,prevTotalCols,outputPath,outputName):
    endChunkBin = chunk['components'][-1]['last_bin']
    chunk['last_bin'] = endChunkBin
    
    localPath = f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}{os.path.sep}'
    
    fileName = f'chunk{chunkNum}_zoom{zoomLevel}.schematic.json'
    
    with open(f'{localPath}{fileName}','w') as f:
        json.dump(chunk,f,cls=NpEncoder)
    
    rootStruct['zoom_levels'][zoomLevel]['files'].append({
        'file': fileName,
        'first_bin':chunk['first_bin'],
        'last_bin':chunk['last_bin'],
#         'x':prevTotalCols
    })
    
    if zoomLevel==1:
        fastaName = f'seq_chunk{chunkNum}_zoom{zoomLevel}.fa'
        rootStruct['zoom_levels'][zoomLevel]['files'][-1]['fasta'] = fastaName
        
        with open(f'{localPath}{fastaName}','w') as f:
            f.write(f'>first_bin:{chunk["first_bin"]} last_bin:{chunk["last_bin"]}\n')
            f.write(nucleotides)
        
    chunk = deepcopy(chunkTemplate)
    chunk['first_bin'] = endChunkBin + 1
    return rootStruct,chunk,0,chunkNum + 1,prevTotalCols+curCompCols,0,'' #rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols

In [ ]:
def addLinksToComp(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft=True,doRight=True):
    compLinks = fillLinks(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft,doRight)
    if "lArrivals" in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["lArrivals"].items():
            if otherSide=='fr':
                fromRight = True
            else:
                fromRight = False
            component['larrivals'].append({
                    'upstream': upstream,
                    'downstream': downstream,
                    'otherSideRight': fromRight,
                    'participants': participants
                })
    
    if 'rArrivals' in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["rArrivals"].items():
            if otherSide == 'fr':
                fromRight = True
            else:
                fromRight = False
            component['rarrivals'].append({
                    'upstream': upstream,
                    'downstream': downstream,
                    'otherSideRight': fromRight,
                    'participants': participants
                })
            
    if 'rDepartures' in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["rDepartures"].items():
            if otherSide == 'tl':
                toRight = False
            else:
                toRight = True
            component['rdepartures'].append({
                'upstream': upstream,
                'downstream': downstream,
                'otherSideRight': toRight,
                'participants': participants
            })
    
    if 'lDepartures' in compLinks:
        for (upstream,downstream,otherSide),participants in compLinks["lDepartures"].items():
            if otherSide == 'tl':
                toRight = False
            else:
                toRight = True
            component['ldepartures'].append({
                'upstream': upstream,
                'downstream': downstream,
                'otherSideRight': toRight,
                'participants': participants
            })
        
    return component

In [ ]:
def splitforwardInversedNodeComp(pathList,component,isInverse):
#     if component['firstCol']==460:
#         pdb.set_trace()
    
    forward = []
    inversed = []
    
    for pathID in pathList:
            try:
                if component["matrix"][component["occupants"].index(pathID)][1]>0:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
            except (IndexError,ValueError):
                # If it is artificial pass link.
                if isInverse:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
                
    return forward,inversed

In [ ]:
def fillLinks(nodeInComp,nodeToComponent,toLinks,fromLinks,component,components,doLeft=True,doRight=True):
    compLinks = {}
    
    
    
#     if component['first_bin']==2 and len(nodeInComp)==2:
#         pdb.set_trace()
    
    for node in nodeInComp:
        # Processing all external arrival links
        nodeToLink = toLinks.get(node,{})
        for fromNode in nodeToLink.keys():
            intermediateCondition = (node<fromNode)
            
            la,ra = splitforwardInversedNodeComp(nodeToLink[fromNode],component,intermediateCondition)
            
            fromFirstComp = components[nodeToComponent[fromNode-1][0]]
            fromLastComp = components[nodeToComponent[fromNode-1][-1]]
            
            #left arrivals
            if len(la)>0 and doLeft:
                frd,fld = splitforwardInversedNodeComp(la,fromFirstComp,intermediateCondition)
                
                complArrivals = compLinks.setdefault('lArrivals',{})
                # from right departure
                if len(frd)>0 and (fromNode not in nodeInComp) and (fromLastComp['last_bin']+1!=component['first_bin']):
                    complArrivals.setdefault((fromLastComp['last_bin'],component['first_bin'],'fr'),set()).update(frd)
                #from left departure
                if len(fld)>0 and (fromNode not in nodeInComp):
                    complArrivals.setdefault((fromFirstComp['first_bin'],component['first_bin'],'fl'),set()).update(fld)
            
            #right arrivals
            if len(ra)>0 and doRight:
                frd,fld = splitforwardInversedNodeComp(ra,fromFirstComp,intermediateCondition)
                
                comprArrivals = compLinks.setdefault('rArrivals',{})
                #from right departures
                if len(frd)>0 and (fromNode not in nodeInComp):
                    comprArrivals.setdefault((fromLastComp['last_bin'],component['last_bin'],'fr'),set()).update(frd)
                #from left departures
                if len(fld)>0 and (fromNode not in nodeInComp):
                    comprArrivals.setdefault((fromFirstComp['first_bin'],component['last_bin'],'fl'),set()).update(fld)
            
        # Processing all external departure links
        nodeFromLink = fromLinks.get(node,{})
        for toNode in nodeFromLink.keys():
            intermediateCondition = (node>toNode)
            
            rd,ld = splitforwardInversedNodeComp(nodeFromLink[toNode],component,intermediateCondition)
            
            toFirstComp = components[nodeToComponent[toNode-1][0]]
            toLastComp = components[nodeToComponent[toNode-1][-1]]
            
            #right departures
            if len(rd)>0 and doRight: # Check if doRight is set incorrectly for our case (121->122 at level 4)
                tla,tra = splitforwardInversedNodeComp(rd,toFirstComp,intermediateCondition)
                
                comprDepartures = compLinks.setdefault('rDepartures',{})
                #to left arrivals
                if len(tla)>0:
                    if toNode not in nodeInComp:
                         comprDepartures.setdefault((component['last_bin'],toFirstComp['first_bin'],'tl'),set()).update(tla)
                    elif node==nodeInComp[-1] and toNode==nodeInComp[0]:
                        # Check that repeated element is actually contiguous inside otherwise it is not a loop and should not be visible.
                        tla_update = list(set(tla).intersection(*[set(fromLinks.get(nodeInComp[i],{}).get(nodeInComp[i+1],[])) for i in range(len(nodeInComp)-1)]))
                        
                        if len(tla_update)>0:
                            #Loop link, can only happen when with left arrival and right departure (or vice versa).
                            complArrivals = compLinks.setdefault('lArrivals',{})

                            complArrivals.setdefault((component['last_bin'],component['first_bin'],'fr'),set()).update(tla)
                            comprDepartures.setdefault((component['last_bin'],component['first_bin'],'tl'),set()).update(tla)
                
                #to right arrivals
                if len(tra)>0: # Most probably the problem is here! Check it!
                    if toNode not in nodeInComp:
                        comprDepartures.setdefault((component['last_bin'],toLastComp['last_bin'],'tr'),set()).update(tra)
            
            #left departures
            if len(ld)>0 and doLeft:
                tla,tra = splitforwardInversedNodeComp(ld,toFirstComp,intermediateCondition)
                
                complDepartures = compLinks.setdefault('lDepartures',{})
                #to left arrivals
                if len(tla)>0:
                    if toNode not in nodeInComp:
                        complDepartures.setdefault((component['first_bin'],toFirstComp['first_bin'],'tl'),set()).update(tla)
                
                if len(tra)>0:
                    if toNode not in nodeInComp:
                        if component['first_bin']-1!=toFirstComp['last_bin']:
                            complDepartures.setdefault((component['first_bin'],toLastComp['last_bin'],'tr'),set()).update(tra)
                    elif node==nodeInComp[0] and toNode==nodeInComp[-1]:
                        # Check that repeated element is actually contiguous inside otherwise it is not a loop and should not be visible.
                        tla_update = list(set(tra).intersection(*[set(fromLinks.get(nodeInComp[i],{}).get(nodeInComp[i-1],[])) for i in range(len(nodeInComp)-1,0,-1)]))
                        
                        if len(tla_update)>0:
                            #Loop link, can only happen when with right arrival and left departure (or vice versa).
                            comprArrivals = compLinks.setdefault('rArrivals',{})

                            comprArrivals.setdefault((component['first_bin'],component['last_bin'],'fl'),set()).update(tra)
                            complDepartures.setdefault((component['first_bin'],component['last_bin'],'tr'),set()).update(tra)
                        
    return compLinks